[kaggle home](https://www.kaggle.com/c/tmdb-box-office-prediction)

In [0]:
# import numpy as np
# import pandas as pd
# def read_data():
#     df_train = pd.read_csv(
#         "https://storage.googleapis.com/kaggle-competitions-data/kaggle/10300/291543/train.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1550508517&Signature=oPLa63IVYX704jidaPG6ACfWshoP7ntNv9%2BFKefjSiiADoUtp2oCYy0sfmc0JdUcw9E3COn7vgPHOcnAzAPjSSsnP1NUGXAIbMqvLAO7mVpWxmnfpZVwg8UZXqH2B%2BCPG%2BGwJrCRQBGuiIdw0UxyRyR6w97yGDEJjsP4c7HdqcnDVKJG%2BjZWlr6JJPMqhWUiGmvZdPED0y7fm1DRV%2BxT3c43eU5npy9kowseOsNwp2tP43LwavwVAjOeRNbCqqCFkFmCM%2BjcO9CgBDwwsB1oY7dgA69L5WKz1IGbSTFr%2FfhkeibQFjQqqCu%2BS4I0q1UwqEUEfZZ5NtD918kX01htug%3D%3D",
#         low_memory = False,
#         compression = "zip",
#         parse_dates = ["release_date"])
#     df_test = pd.read_csv(
#         "https://storage.googleapis.com/kaggle-competitions-data/kaggle/10300/291543/test.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1550508498&Signature=B3yh8PATtePDgbBZ4KHlUur2jt4%2Bk%2BWJrlBVuW%2F25waxgim5cSr6a5J8Xw2lcQcG2VsIYl3OpGRPPK3zeu4B01ofcmwm8azjhgV7ubeqE%2Ba7xp5ZVKO2pMfFdQV7Ro5zHYIg2calCUFRbH%2FV02UYX7b3me6WGzHzYMM6YX6rd0pTYz%2BgYagOj97aI2EpV%2BgJVh7QPIaz5huBWvpR%2F%2FZNgawcG%2FKGjiC%2F3YEfQ9%2BL%2Bh5%2B7pNrIINhQMmZzfSLZZu5dQIPnc1JkkyYmvVhAi5e9t%2Byxzbg2l8aEaIG%2B3OOuhb6aUAQA6d%2Fy1vqWj3VFyuXICL58bBdgcmyQgfjr3zdsw%3D%3D",
#         low_memory = False, 
#         compression = "zip",
#         parse_dates = ["release_date"])
#     return df_train, df_test
# df_train, df_test = read_data()


In [0]:
def drop_cols(df):
    drop_if_exists = [
        "genres", "production_countries", "spoken_languages", "original_language",
        "homepage",
        "production_companies", "belongs_to_collection", "id", "imdb_id",
        "poster_path", "status", "original_title", "overview", "tagline",
        "title", "cast", "crew", "Keywords"]
    drop_cols = [x for x in drop_if_exists if x in df.columns]
    after = df.drop(drop_cols, axis = 1)
    return after

In [0]:
import datetime
def write_csv(rf, X_test):
    predictions = np.exp(rf.predict(X_test))
    pred_by_id = zip(range(3001, 3001+4398), predictions)
    csv = "id,revenue\n"
    for id, pred in pred_by_id:
        csv = csv + str(id)+","+str(pred)+"\n"
    with open('./out.csv', 'w') as file:
        file.write(csv)
    print(datetime.datetime.now())

In [0]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import svm
import math
def print_score(df, X_test = None):
    df = drop_cols(df)
    X = df.drop('revenue', axis = 1)
    y = np.log1p(df.revenue)
    sc = "neg_mean_squared_error"
    rf = ExtraTreesRegressor(n_estimators = 300, n_jobs = -1)
    scores = cross_val_score(rf, X, y, cv=5, scoring = sc)
    print("Accuracy: %0.2f (+/- %0.2f)" % (math.sqrt(-scores.mean()), scores.std() * 2))
    if X_test is not None:
        rf.fit(X, y)
        feature_importances = pd.DataFrame(
            rf.feature_importances_, index = X.columns,
            columns=['importance']).sort_values('importance',ascending=False)
        print(feature_importances)
        X_test = drop_cols(X_test)
        write_csv(rf, X_test)

In [0]:
import datetime
year = datetime.datetime.now().year
def get_release_season(date):
    m = date.month
    if m == 1 or m == 2:
        return 0
    elif m== 3 or m == 4:
        return 1
    elif m == 5 or m == 6 or m == 7 or m == 8:
        return 2
    elif m == 10 or m == 9:
        return 3
    else:
        return 4
def process_date(df):
    rel_year = df.release_date
    rel_year = rel_year.apply(lambda x: x.year)
    rel_year = rel_year.fillna(2019)
    rel_season = df.release_date.apply(get_release_season)
    rel_season = rel_season.fillna(5)
#     rel_month = df.release_date.apply(lambda x: x.month)
#     rel_month = rel_season.fillna(1)
#     rel_day = df.release_date.apply(lambda x: x.day)
#     rel_day = rel_season.fillna(1)
    after = df.drop("release_date", axis=1)
    after['release_year'] = rel_year
    after['release_season'] = rel_season
#     after['release_month'] = rel_month
#     after['release_day'] = rel_day
    return after

In [0]:
import ast
def bool_collection(x):
    if isinstance(x, float):
        return False
    else:
        return True
def proc_coll(df):
    bools = df.belongs_to_collection.apply(bool_collection)
    after = df.drop("belongs_to_collection", axis=1)
    after['belongs_to_collection_bool'] = bools
    return after

In [0]:
def proc_home(df):
    bools = df.homepage.apply(bool_collection)
    after = df.drop("homepage", axis=1)
    after['homepage_bool'] = bools
    return after

In [0]:
def get_director(x):
    if isinstance(x, float):
        return "NaN"
    crew = ast.literal_eval(x)
    for person in crew:
        if "job" in person.keys():
            if person['job'] == "Director":
                return person['name']
    return "NaN"
def get_first(x):
    if isinstance(x, float):
        return "NaN"
    crew = ast.literal_eval(x)
    for person in crew:
        if "order" in person.keys():
            if person['order'] == 0:
                return person['name']
    return "NaN"
def proc_dir(df):
    director = df.crew.apply(get_director)
    after = df.drop('crew', axis = 1)
    after["director"] = director
    return after
def proc_cast(df):
    first_billing = df.cast.apply(get_first)
    after = df.drop('cast', axis = 1)
    after["first_billing"] = first_billing
    return after
    

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import ExtraTreesRegressor
from pandas import DataFrame
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
def get_name_list(x):
    if isinstance(x, float):
        return set()
    if isinstance(x, list):
        return x
    d = ast.literal_eval(x)
    name_list = []
    for i in d:
        if 'order' in i:
            if i['order'] < 4:
                name_list.append(i['name'])
        if 'job' in i:
            if i['job'] == 'Director':
                name_list.append(i['name'])
        else:
            name_list.append(i['name'])
    return set(name_list)
def sub_df(all_labels):
    bins = mlb.fit_transform(all_labels)
    df_sub = DataFrame(data = bins, columns = mlb.classes_)
    return df_sub
def get_top_five(all_cols):
    rf = ExtraTreesRegressor(n_jobs = -1, n_estimators = 10)
    rf.fit(all_cols, df_train['revenue'])
    feature_importances = pd.DataFrame(
        rf.feature_importances_, index = all_cols.columns,
        columns=['importance']).sort_values('importance',ascending=False)
    return feature_importances[:5].index
def get_feature_importances(all_cols):
    rf = ExtraTreesRegressor(n_jobs = -1, n_estimators = 10)
    rf.fit(all_cols, df_train['revenue'])
    feature_importances = pd.DataFrame(
        rf.feature_importances_, index = all_cols.columns,
        columns=['importance']).sort_values('importance',ascending=False)
    return feature_importances
def add_top_five(train_df, test_df, header):
    train_labels = train_df[header].apply(get_name_list)
    train_sub = sub_df(train_labels)
    top_five = get_top_five(train_sub)
    print(top_five)
    train_after = pd.concat((train_df, train_sub[top_five]), axis = 1)
    
    test_labels = test_df[header].apply(get_name_list)
    test_sub = sub_df(test_labels)
    test_top_five = [x for x in top_five if x in test_sub.columns]
    test_after = pd.concat((test_df, test_sub[test_top_five]), axis = 1)
    return train_after, test_after

In [0]:
def proc_title(df):
    title_change = np.where(df.title == df.original_title, True, False)
    title_len = df.title.apply(lambda x: 0 if "nan" else len(x))
    after = df.drop(['original_title', 'title'], axis = 1)
    after['title_changed'] = title_change
    after['title_len'] = title_len
    return after

In [0]:
def proc_tagline(df):
    tag_len = df.tagline.apply(lambda x: 0 if "nan" else len(x))
    tag_words = df.tagline.apply(lambda x: 0 if "nan" else len(x.split(' ')))
    after = df.drop(['tagline'], axis = 1)
    after['tag_len'] = tag_len
    after['tag_words'] = tag_words
    return after

In [0]:
def proc_over(df):
    over_len = df.overview.apply(lambda x: 0 if "nan" else len(x))
    over_words = df.overview.apply(lambda x: 0 if "nan" else len(x.split(' ')))
    after = df.drop(['overview'], axis = 1)
    after['overview_len'] = over_len
    after['overview_words'] = over_words
    return after

In [0]:
def proc_time(df):
    df.runtime = df.runtime.fillna(0)
#     av = df.runtime.median()
#     print(df.runtime)
#     runs = df.runtime.apply(lambda x: 0 if "nan" else x)
#     diff = df.runtime.apply(lambda x: 0 if "nan" else x - av)
#     after = df#.drop(['runtime'], axis = 1)
#     after.runtime = runs
#     print(runs)
#     after['runtime_diff'] = diff
    return df

In [0]:
def proc_budget(df):
#     av_per_year = df.groupby(['release_year'])['budget'].aggregate('mean')
#     budget_diff = df.budget - av_per_year
    after = df.drop('budget', axis = 1)
#     after['budget_diff'] = budget_diff
    after['log_budget'] = np.log1p(df.budget)
    return after

In [0]:
# This cell creates the cast scoring system.
def get_scores_dict(header):
    name_lists = df_train[header].apply(get_name_list)
    sub = sub_df(name_lists)
    freq = sub.astype(bool).sum(axis = 0).sort_values(ascending = False)[:50]
    sub = sub[freq.index]
    scores = get_feature_importances(sub)
    return scores.to_dict()['importance']
def get_score(name_list, score_dict):
    score = 0
    for name in name_list:
        if name in score_dict:
            score = score + score_dict[name]
    return score
def scores_from_text(header):
    name_lists = df_train[header].apply(
        lambda x:
            set() if isinstance(x, float) else
            set(x.replace('.', '').replace(';', '').replace(':', '').split(' ')))
    sub = sub_df(name_lists)
    freq = sub.astype(bool).sum(axis = 0).sort_values(ascending = False)[:700]
    sub = sub[freq.index]
    scores = get_feature_importances(sub)
    return scores.to_dict()['importance']

In [0]:
def expand_cols(df, df_test, header):
    name_lists = df_train[header].apply(get_name_list)
    test_name_lists = df_test[header].apply(get_name_list)
    sub = sub_df(name_lists)
    sub_test = sub_df(test_name_lists)
    freq = sub.astype(bool).sum(axis = 0).sort_values(ascending = False)[:5]
    sub = sub[freq.index]
    sub_test = sub_test[freq.index]
    after = df.drop(header, axis = 1)
    after = pd.concat([after, sub], axis = 1)
    test_after = df_test.drop(header, axis = 1)
    test_after = pd.concat([test_after, sub_test], axis = 1)
    return after, test_after

In [0]:
%%time

X_train, X_test = process_date(df_train), process_date(df_test)
X_train, X_test = proc_budget(X_train), proc_budget(X_test)
X_train, X_test = proc_coll(X_train), proc_coll(X_test)
X_train, X_test = proc_time(X_train), proc_time(X_test)
expand_headers = [
    "genres", "cast", "crew", "production_companies", "production_countries"]
for header in expand_headers:
    X_train, X_test = expand_cols(X_train, X_test, header)
# X_train, X_test = expand_cols(X_train, X_test, 'genres')
# X_train = expand_cols(X_train, X_test, 'crew')
# X_train, X_test = expand_cols(X_train, X_test, 'production_companies')
# X_train, X_test = expand_cols(X_train, X_test, 'production_countries')
print_score(X_train, X_test = X_test)

In [0]:
%%time
# cast_dict = get_scores_dict('cast')
# print(cast_dict)
cast_dict = {'Johnny Depp': 0.12571618407236504, 'Robert Downey Jr.': 0.08814401931862209, 'Christian Bale': 0.0737746352851139, 'Ed Harris': 0.060835342214563715, 'Tom Cruise': 0.058052661449185805, 'Harrison Ford': 0.04865554876998225, 'Owen Wilson': 0.047884458800967956, 'Mark Wahlberg': 0.03009776831200684, 'Nicolas Cage': 0.029711723755908016, 'Michael Caine': 0.02755645178760372, 'Tom Hanks': 0.025310155108478727, 'Julianne Moore': 0.025020751861883645, 'Mel Gibson': 0.0229816945998509, 'Ewan McGregor': 0.022744617150188724, 'Halle Berry': 0.022418436998033378, 'George Clooney': 0.02224473783406203, 'Russell Crowe': 0.020701936428833456, 'Julia Roberts': 0.0182782949847725, 'Arnold Schwarzenegger': 0.0177678393999594, 'Matt Damon': 0.017472131144693648, 'Sylvester Stallone': 0.016371292879365652, 'Samuel L. Jackson': 0.014527434948675389, 'Bruce Willis': 0.012106738863966252, 'Kevin Bacon': 0.011794222188907008, 'Woody Harrelson': 0.011350618008047087, 'Robert Duvall': 0.009826587263508806, 'Gary Oldman': 0.008323786386627754, 'Robin Williams': 0.00817613807968587, 'Eddie Murphy': 0.008100194636730952, 'John Goodman': 0.00741689957510277, 'Keanu Reeves': 0.007142566492194598, 'Sigourney Weaver': 0.0068017635902161075, 'Ethan Hawke': 0.006376963571486897, 'Christopher Walken': 0.005784445236253831, 'Richard Gere': 0.005118410711555488, 'Meryl Streep': 0.005049595459992872, 'Liam Neeson': 0.004975156654226353, 'Bill Murray': 0.004865453823623963, 'Colin Farrell': 0.00422277697230816, 'John Cusack': 0.004161252836370486, 'Susan Sarandon': 0.003968705709160734, 'Morgan Freeman': 0.0038208340811114274, 'Denzel Washington': 0.0038005103756519774, 'Gene Hackman': 0.003403969596519653, 'Harvey Keitel': 0.0032584206548037947, 'Robert De Niro': 0.003144280080687166, 'Jason Statham': 0.003042184444944122, 'Kurt Russell': 0.003007786818521761, 'Nicole Kidman': 0.0024988648811251285, 'Dennis Quaid': 0.0021927559015521535}
# cast_dict = {'Robert Downey Jr.': 0.060900756015450495, 'Chris Pratt': 0.053077393900632194, 'Ray Romano': 0.05079819159537916, 'Shia LaBeouf': 0.05064334526498745, 'Ginnifer Goodwin': 0.043173118946873965, 'Elijah Wood': 0.0334272321325668, 'Emma Watson': 0.03140702680162994, 'Daniel Radcliffe': 0.028855939866338165, 'Vin Diesel': 0.025482235206029862, 'Kristen Stewart': 0.02347902830627426, 'Daniel Craig': 0.023102102686349456, 'Johnny Depp': 0.02264921704918419, 'Ian McKellen': 0.020398245205844358, 'Tom Cruise': 0.020365011593120735, 'Hugh Jackman': 0.01750359609500555, 'Sam Neill': 0.015937310584624755, 'Harrison Ford': 0.015100079272367803, 'Christian Bale': 0.013186038134785652, 'Mia Wasikowska': 0.01253817870666348, 'Tom Hanks': 0.011620730061445544, 'Ben Burtt': 0.010162871007428527, 'Jennifer Lawrence': 0.009712126267857397, 'Ryan Reynolds': 0.009535344024475282, 'Jim Carrey': 0.009368176850053112, 'Mark Wahlberg': 0.009322989399270322, 'Will Smith': 0.008462885784203868, 'Angelina Jolie': 0.008326729284432267, 'Pierce Brosnan': 0.008002114961355397, 'Owen Wilson': 0.007958787233226173, 'Henry Cavill': 0.0077504885452093384, 'Chris Evans': 0.007694103535682409, 'Julia Roberts': 0.007650017306253365, 'Craig T. Nelson': 0.007600102146691457, 'Leonardo DiCaprio': 0.007402255551883116, 'Adam Sandler': 0.007114038612437862, 'Matt Damon': 0.0070660524190704, 'Fran√ßois Cluzet': 0.006637119387031365, 'Sylvester Stallone': 0.006492059631180315, 'Chris Tucker': 0.006012192131160155, 'Jaden Smith': 0.005934426368384186, 'Brad Pitt': 0.005817540214352546, 'Ewan McGregor': 0.0057944328811551735, 'Garrett Hedlund': 0.005782464919771871, 'Arnold Schwarzenegger': 0.005379719299404259, 'Bai Baihe': 0.005336538351988019, 'Milla Jovovich': 0.005003542751307778, 'Helen Hunt': 0.005002524756541846, 'Sandra Bullock': 0.004892573236709921, 'Ben Stiller': 0.004545316519201123, 'Patrick Stewart': 0.004424293231764046, 'George Clooney': 0.0044155606842891095, 'Jason Biggs': 0.004353202737431309, 'Hank Azaria': 0.004260827526591203, 'Irene Bedard': 0.0042257897969147536, 'Bradley Cooper': 0.004081832208654785, 'Patrick Wilson': 0.004058703521555055, 'Jim Caviezel': 0.004052411185747566, 'Kevin Costner': 0.003953011956010832, 'Whitney Houston': 0.0038069606527487295, 'Dwayne Johnson': 0.0036323782075201676, 'Michael Douglas': 0.003590891056446262, 'Naomi Watts': 0.0035425729471247037, 'Jesse Eisenberg': 0.0034940441887112484, 'Natalie Portman': 0.0033727614892478915, 'Michael J. Fox': 0.003273529794552589, 'Eddie Murphy': 0.0031307185141811666, 'Robin Williams': 0.003116909811105673, 'Tony Goldwyn': 0.003116241078249214, 'Sarah Jessica Parker': 0.003073171522084987, 'Nicolas Cage': 0.003065860810294794, 'Linda Blair': 0.003025217852908241, 'Mel Gibson': 0.002987039362068008, 'Steve Carell': 0.002829651623638081, 'Alec Baldwin': 0.002786214265290958, 'Seth Rogen': 0.0027824935070753414, 'Katie Featherston': 0.002771036116187773, 'Ben Barnes': 0.00276584487363757, 'Russell Crowe': 0.002759561407948482, 'Roger Moore': 0.0026641149409650463, 'Noomi Rapace': 0.002522423499440354, 'Denzel Washington': 0.0025040187859032984, 'John C. Reilly': 0.0024930859787672343, 'Logan Lerman': 0.002437487324268005, 'Jeremy Renner': 0.002333119135370256, 'Rowan Atkinson': 0.002285816329385587, 'Jack Nicholson': 0.002036725816964231, 'Aamir Khan': 0.0020336769837744845, 'James McAvoy': 0.0020244222688260453, 'Anna Kendrick': 0.0019156903866422985, 'Kevin James': 0.00186828211201523, 'Ben Affleck': 0.0017556549816183068, 'Timothy Dalton': 0.0017264210220234364, 'Sean Connery': 0.0016789165920526859, 'Tim Allen': 0.0016196199865627058, 'Ioan Gruffudd': 0.0016163669719555854, 'Hayden Christensen': 0.0016022025786320937, 'Melissa McCarthy': 0.0016010535833249008, 'John Travolta': 0.0015846847147972156, 'Jamie Foxx': 0.0014726602411565394, 'Anthony Hopkins': 0.0013981785589949388, 'Patrick Swayze': 0.0013939702249483805, 'Britt Robertson': 0.0013933837822929472, 'Al Pacino': 0.001383398123384467, 'Sharlto Copley': 0.001379850232363723, 'Reese Witherspoon': 0.0013753774316272783, 'Cameron Diaz': 0.0013698446117743047, 'Nia Vardalos': 0.001341579036047174, 'Jason Clarke': 0.0013192995217529758, 'Brandon Routh': 0.0013175887870709563, 'Gerard Butler': 0.001210504755443571, 'Laura Linney': 0.001191050978411166, 'Kevin Spacey': 0.001132804756982846, 'Hugh Grant': 0.0011197692470583834, 'Sigourney Weaver': 0.0011147566988230617, 'Freddie Prinze Jr.': 0.0010817911816611995, 'Josh Hutcherson': 0.0009368558728949396, 'Channing Tatum': 0.0009193760834791253, 'Jamie Bell': 0.0009097770846403577, 'Sam Worthington': 0.000839580655832306, 'Jane Levy': 0.0008098279958528318, 'Rachel McAdams': 0.0008056342003347183, 'Meryl Streep': 0.0008049171400234416, 'Antonio Banderas': 0.000802217332658755, 'Roy Scheider': 0.0007985328511785923, 'NaN': 0.0007925746260900512, 'Michael Keaton': 0.0007833609332665162, 'Bruce Willis': 0.0007797157667136324, 'Holly Marie Combs': 0.0007696146222839886, 'Anne Hathaway': 0.0007672592104923032, 'Nicholas Hoult': 0.0007666181255962625, 'Val Kilmer': 0.0007529205389552418, 'Katherine Heigl': 0.0007523558674300001, 'Jonah Hill': 0.0007404034993235338, 'Jodie Foster': 0.0007258273126632558, 'Jason James Richter': 0.0006923800996652212, 'Bill Murray': 0.0006859696527708575, 'Tom Hardy': 0.0006814942940049866, 'Jodi Benson': 0.0006782838581537011, 'Briana Evigan': 0.0006647762824584724, 'Drew Barrymore': 0.0005919946192737278, 'Will Ferrell': 0.0005704001130551835, 'Richard Dreyfuss': 0.0005529446098322114, 'Luke Evans': 0.00054786090739301, 'Dennis Quaid': 0.0005398912055013964, 'Liam Neeson': 0.000538680889753926, 'Jake Gyllenhaal': 0.0005322873372815651, 'Ali MacGraw': 0.0005216416613499292, 'Paul Walker': 0.0004947942330720265, 'Sacha Baron Cohen': 0.00047987665132112205, 'Vince Vaughn': 0.0004789791276778514, 'Greg Kinnear': 0.00046387113676916596, 'Amy Schumer': 0.0004633311527763743, 'Dustin Hoffman': 0.000422078055926918, 'Jeff Bridges': 0.00042094156359407845, 'Barbra Streisand': 0.0004200088449954944, 'Zach Galligan': 0.0004199600807125767, 'Steve Martin': 0.00041579423487997326, 'William Shatner': 0.0004109655589079065, 'Marlon Brando': 0.0003722057263033687, 'Tony Leung Chiu-Wai': 0.00037043383005905495, 'Robert Redford': 0.0003693342051675168, 'Goldie Hawn': 0.0003525973411552847, 'Jackie Chan': 0.00035255492778570545, 'Uma Thurman': 0.00034863492548210773, 'Paul Rudd': 0.00034204825571149036, 'Kevin Hart': 0.000332323568974602, 'Veronica Taylor': 0.0003173088537940398, 'Helen Mirren': 0.0003158233337583862, 'David Arquette': 0.0003146869227261282, 'Harriet Owen': 0.0003055142220295516, 'Samuel L. Jackson': 0.00030272394252482665, 'Frank Grillo': 0.00029857114268624045, 'Miles Teller': 0.0002972373320557286, 'Charlton Heston': 0.000295859682850751, 'Tommy Lee Jones': 0.00029576531989073594, 'Robert De Niro': 0.00028575880924373696, 'Keanu Reeves': 0.00028520317734531393, 'Burt Reynolds': 0.00028471177383974325, "Jack O'Connell": 0.00027594054210858374, 'Ashton Kutcher': 0.0002539214186881647, 'Matthew McConaughey': 0.0002472344123854604, 'Mary Elizabeth Winstead': 0.0002377146009214078, 'Aaron Taylor-Johnson': 0.00023770714831604828, 'Jessica Chastain': 0.00023181576897017133, 'Ellen Page': 0.0002187559028807967, 'Martin Lawrence': 0.00021633440499376164, 'Zachary Gordon': 0.00020798481749855396, 'Richard Gere': 0.00020769103971431251, 'Jack Huston': 0.0002068463424321576, 'Alexa PenaVega': 0.00020253028540333763, 'Charles Grodin': 0.00019900201808294653, 'Paul Bettany': 0.0001951672146995849, 'Ajith Kumar': 0.00018966125305097643, 'Ali Larter': 0.00018535801108589606, 'Jennifer Love Hewitt': 0.00018474926388834678, 'Stephen Chow': 0.00018463787259904386, 'Clint Eastwood': 0.0001827854459210591, 'Aaron Eckhart': 0.00018159434300937867, 'Lindsay Lohan': 0.000181593186948279, 'Jude Law': 0.00017481743941020737, 'Morgan Freeman': 0.00017467655277364703, 'Ren√©e Zellweger': 0.00017417894657416425, 'Bobby Driscoll': 0.00017131119219343933, 'Gary Oldman': 0.00017004998409070278, 'Chris Hemsworth': 0.00016897789870253189, 'Tim Curry': 0.00015473650298028748, 'Edgar Ram√≠rez': 0.0001545411979582053, 'Meg Ryan': 0.00015415893645770717, 'Topher Grace': 0.0001495304156272389, 'Halle Berry': 0.0001455865913683839, 'Viggo Mortensen': 0.00014321923583859227, 'Thomas Jane': 0.00014172610387276673, 'Jason Bateman': 0.00014153236983323767, 'Steven Seagal': 0.0001357565354198846, 'John Wayne': 0.00013343536220358142, 'Billy Crystal': 0.00013171746744738578, 'Natasha Henstridge': 0.0001310358445248406, 'Cary Elwes': 0.00013072094295075873, 'Jet Li': 0.00012919144983442456, 'Samy Naceri': 0.00012779341142059756, 'Josh Brolin': 0.0001254961373324195, 'Whoopi Goldberg': 0.00012156131860594959, 'Hiroki Hasegawa': 0.00012132490327609362, 'Jason Statham': 0.00012129627966050028, 'Domhnall Gleeson': 0.0001208329737007357, 'J.J. Johnson': 0.00011883492457235248, 'Wesley Snipes': 0.00011730608482952108, 'John Goodman': 0.00011302302261196511, 'Tyler Perry': 0.00011251791993128954, 'Jamie Lee Curtis': 0.00011247552217668313, 'Ryan Gosling': 0.00011228107995635721, 'Paul Giamatti': 0.00011113403417988104, 'Josh Lucas': 0.00010952513377704381, 'Gene Hackman': 0.00010749404417273785, 'Jennifer Lopez': 0.00010740168131711798, 'Sean Penn': 0.00010523427292195151, 'Kurt Russell': 0.00010397397884351971, 'Barry Newman': 0.00010337166796582562, 'John Turturro': 0.00010123697425455534, 'Greta Garbo': 9.838576102614671e-05, 'Michael Shannon': 9.5457478926219e-05, 'Joaquin Phoenix': 9.17586022864211e-05, 'James Badge Dale': 9.088857215203716e-05, 'Heath Ledger': 9.068830563409367e-05, 'Walter Matthau': 9.037165391357702e-05, 'Frankie Muniz': 8.823375289798969e-05, 'Julie Andrews': 8.815611816599908e-05, 'Emily Blunt': 8.748762008024251e-05, 'Steve Guttenberg': 8.347145265739156e-05, 'Jos√© Luis Manzano': 8.173367485682546e-05, 'Eric Balfour': 8.146232768500391e-05, 'Holly Hunter': 8.06579807686519e-05, 'Jennifer Garner': 7.999862772113842e-05, 'Gwyneth Paltrow': 7.815731176289557e-05, 'Blake Lively': 7.755813536005126e-05, 'Emile Hirsch': 7.671846366271145e-05, 'Alan Rickman': 7.64104184906751e-05, 'Hume Cronyn': 7.549568041314753e-05, 'Josh Hartnett': 7.238865471639685e-05, 'Geena Davis': 6.984066166181823e-05, 'Benicio del Toro': 6.581804057122731e-05, 'Madonna': 6.408780701127287e-05, 'Britney Spears': 6.198845904792558e-05, 'Danielle Harris': 5.76090508577607e-05, 'Edward Norton': 5.755457605346483e-05, 'Diane Kruger': 5.7482693813600874e-05, 'Betsy Palmer': 5.7422863555780723e-05, 'Michiel Huisman': 5.738418362763303e-05, 'Drago»ô Bucur': 5.737656978812035e-05, 'Jeanne Moreau': 5.736583207493582e-05, 'Carla Gugino': 5.735937363827912e-05, 'Ed Harris': 5.7251841710330054e-05, 'Salman Khan': 5.7131494592744926e-05, 'Eric Stoltz': 5.711210056695861e-05, 'Christopher Lee': 5.624866467635772e-05, 'Mohammad Amir Naji': 5.61386910510756e-05, 'Kevin Kline': 5.596978223978368e-05, 'Renata Litvinova': 5.546942059876467e-05, 'Willem Dafoe': 5.544298581246266e-05, 'Lisa Kudrow': 5.500283677799849e-05, 'Terence Stamp': 5.493934474942563e-05, 'Mary Pickford': 5.476148493439158e-05, "Michael O'Keefe": 5.455886146552829e-05, 'Billy Connolly': 5.4548913485937426e-05, 'Dolph Lundgren': 5.421475587550531e-05, 'Gina Philips': 5.392668621864833e-05, 'Brooke Shields': 5.283797854175683e-05, 'Jane Fonda': 5.2675378206097984e-05, 'Paz Vega': 5.2611745166623395e-05, 'Guy Pearce': 5.2461655113961354e-05, 'Chuck Norris': 5.227872919650547e-05, 'Cary Grant': 5.200716309112089e-05, 'Anthony Edwards': 5.194468911678883e-05, 'Sam Elliott': 5.183764704683123e-05, 'Rachel Weisz': 5.086216209057309e-05, 'Riz Ahmed': 4.997418829751881e-05, 'Jeremy Irons': 4.896253236595465e-05, 'Fred Astaire': 4.807299510203753e-05, 'Charlie Chaplin': 4.681343577463773e-05, 'Akshay Kumar': 4.67086445759115e-05, 'Alan Arkin': 4.652033350386368e-05, 'Fyodor Bondarchuk': 4.619599732306239e-05, 'Scott Bakula': 4.6088451442445895e-05, 'Charlize Theron': 4.6002128990332654e-05, 'Annette Bening': 4.5930499471931014e-05, 'Jim Broadbent': 4.590426069972192e-05, 'Greta Gerwig': 4.5846507881075474e-05, 'Matthew Modine': 4.577670993687907e-05, 'Harvey Keitel': 4.57197460485726e-05, 'Ellen Burstyn': 4.534866675877429e-05, 'Dennis Hopper': 4.49721867936906e-05, 'Zach Braff': 4.4707051518236404e-05, 'Kevin Conroy': 4.469880384314812e-05, 'Keith Coogan': 4.467775924218822e-05, 'Charles Bronson': 4.457436746847979e-05, 'Priyanka Chopra': 4.4555331163906176e-05, 'Vikram': 4.449674167180186e-05, 'Veerle Baetens': 4.449093608501147e-05, 'Bruno Ganz': 4.339533756222811e-05, 'John Hawkes': 4.313964991367368e-05, 'Kate Capshaw': 4.2947389937217946e-05, 'Gregory Peck': 4.293709219884805e-05, 'Evan Rachel Wood': 4.2863189120238065e-05, 'Michelle Nicastro': 4.23822660253156e-05, 'Marlee Matlin': 4.230463447705494e-05, 'Roberto Benigni': 4.202210058722524e-05, 'Gene Wilder': 4.079425348035283e-05, 'Billy Bob Thornton': 3.967593071704275e-05, 'Humphrey Bogart': 3.9675199355593755e-05, 'Susan Sarandon': 3.93011032698298e-05, 'Don Johnson': 3.927340646496077e-05, 'Glen Campbell': 3.9170683371036793e-05, 'Cher': 3.912316532734327e-05, 'Ray Milland': 3.896295670005036e-05, 'James Woods': 3.853116360207038e-05, "Brian O'Halloran": 3.8523832638238566e-05, 'Gemma Arterton': 3.778736487344912e-05, 'Brendan Fraser': 3.726955772703488e-05, 'Jonathan Rhys Meyers': 3.674711507275784e-05, 'Jennifer Aniston': 3.67315522375072e-05, 'Richard E. Grant': 3.63391275814309e-05, 'Ashley Grace': 3.6262358824111204e-05, 'Kirsten Dunst': 3.6191272135817664e-05, 'Brahim Hadjadj': 3.608148170981404e-05, 'Forest Whitaker': 3.6004750996160905e-05, 'Bela Lugosi': 3.6003979917717405e-05, 'Adrien Brody': 3.6003538756830754e-05, 'Helena Bonham Carter': 3.599908881744093e-05, 'George Kennedy': 3.598450581085983e-05, 'James Frecheville': 3.59829678549564e-05, 'Freida Pinto': 3.594857748007452e-05, 'James Whitmore': 3.5902791420856234e-05, 'Carlos Padilla': 3.5884296390528634e-05, 'Billy Crudup': 3.586174924133728e-05, 'Hwang Jung-min': 3.586060996942139e-05, 'Hugo Weaving': 3.585718298454626e-05, 'Billy Zane': 3.5851646866094556e-05, 'Ally Sheedy': 3.5842836520107686e-05, 'Glynis Johns': 3.583931269604639e-05, 'Koji Yakusho': 3.583220154589743e-05, 'James Nesbitt': 3.582089933691277e-05, 'Antonio Cantafora': 3.5820214498161284e-05, 'Roddy McDowall': 3.5819620596942065e-05, 'Jared Leto': 3.581333425527342e-05, 'Ray Liotta': 3.581062813127249e-05, 'Franchot Tone': 3.5809458182189985e-05, 'Demi Moore': 3.5802880133657924e-05, 'Francesco Pannofino': 3.578875797489022e-05, 'Barry Otto': 3.578256122292997e-05, 'Ajay Devgn': 3.5773442391754954e-05, 'Antonio Flores': 3.576708982726378e-05, 'James Franco': 3.575238279495472e-05, 'Nanako Matsushima': 3.5752181179180016e-05, 'Kate Beckinsale': 3.5743096453710145e-05, 'Evgeny Mironov': 3.574028797629683e-05, 'Isabelle Adjani': 3.5734309218621954e-05, 'Eva Mendes': 3.573043748539063e-05, 'Diane Keaton': 3.572554195670546e-05, 'Jakob Cedergren': 3.5718676693189004e-05, 'Buck Brannaman': 3.5693871239453966e-05, 'Bob Belleville': 3.5689256704626174e-05, 'Bernadette Lafont': 3.568447467467947e-05, 'Asia Argento': 3.567840886130341e-05, 'James Farentino': 3.5675113652938365e-05, 'Jon Voight': 3.567114902885723e-05, 'Toni Collette': 3.566987396142451e-05, 'Bruce Balden': 3.566858822319602e-05, 'Gabe Nevins': 3.566051554696207e-05, 'Erland Josephson': 3.565725988128567e-05, 'Justin Long': 3.565580285023489e-05, 'Ice-T': 3.5638975635899e-05, 'Glenda Jackson': 3.562905057153442e-05, 'Bruce Campbell': 3.560776409314574e-05, 'Bill Barretta': 3.559433428523677e-05, 'James Marshall': 3.557569705772834e-05, 'Jacob Wysocki': 3.55738244771582e-05, 'James Marsden': 3.552722300149141e-05, "Beverly D'Angelo": 3.551837277388471e-05, 'Kate Bosworth': 3.550913766793146e-05, 'Freddie Highmore': 3.548697732374043e-05, 'Ayesha Dharker': 3.548651252317359e-05, 'Jacques Dutronc': 3.5442259110190515e-05, 'Brianna Brown': 3.542667959827951e-05, 'Kevin Bacon': 3.5407468066098746e-05, 'Barbara Sarafian': 3.535012093523099e-05, 'Geert Hunaerts': 3.5319015509256455e-05, 'Han Hyo-joo': 3.530048083571601e-05, 'Bill Paxton': 3.52914822057529e-05, 'Ir√°n Castillo': 3.5270844949094056e-05, 'Garik Kharlamov': 3.527071295829891e-05, 'Hugo Armstrong': 3.526563006154929e-05, 'Ethel Waters': 3.5232652472143837e-05, 'Evan Glodell': 3.522036520233825e-05, 'William Holden': 3.520066013183949e-05, 'Brie Larson': 3.519749407060916e-05, 'Marion Cotillard': 3.51933481257089e-05, 'Calvin Lee Reeder': 3.517393714901403e-05, 'Phoebe Cates': 3.5131677260405993e-05, 'Jason Alexander': 3.512568745355385e-05, 'Jah Shams': 3.509793438078684e-05, 'Richard Harris': 3.503243220686174e-05, 'Felicity Jones': 3.5022630317901454e-05, 'Ben Weber': 3.498243489909622e-05, 'Mekhi Phifer': 3.497935178294375e-05, 'Bill Sage': 3.494049049542283e-05, 'Dan Aykroyd': 3.489797013834883e-05, 'Alicia Vikander': 3.488830995363245e-05, 'Eric Bana': 3.486302143276578e-05, 'Greg Cusick': 3.484626076766662e-05, 'Bette Davis': 3.4822538261755466e-05, 'Gottfrid Svartholm': 3.480051476912332e-05, 'Armin Mueller-Stahl': 3.475762766626985e-05, 'Harry Nilsson': 3.4721742376619544e-05, 'Devon Sawa': 3.469957852512429e-05, 'J. J. Feild': 3.468035787321143e-05, 'Ben Schnetzer': 3.464002636679793e-05, 'Isabelle Huppert': 3.463336207023734e-05, 'Jacqueline Bisset': 3.460596756626627e-05, 'Jamie McShane': 3.456297650969581e-05, 'Gorka Otxoa': 3.4556848954477144e-05, 'Ir√®ne Jacob': 3.453839860885061e-05, 'Michael Caine': 3.44965903880503e-05, 'James Abourezk': 3.443604307358573e-05, 'Erin Fisher': 3.4389391744753314e-05, 'Gaston Modot': 3.436161515631659e-05, 'Eric Jacobus': 3.431687712293959e-05, 'Brad Johnson': 3.428092329430856e-05, 'Jaime Pressly': 3.423837340285044e-05, 'Florence Pugh': 3.419316533795692e-05, 'Bobby Hackney': 3.4148850619976365e-05, 'Kathleen Turner': 3.4109628558729745e-05, "George O'Brien": 3.410261989907492e-05, 'Billy Gardell': 3.4056626929366955e-05, 'Helen Hayes': 3.4010661651491654e-05, 'Atsuko Tanaka': 3.39838871694456e-05, 'Eric Kot': 3.3964731554562725e-05, 'Ja Rule': 3.391886677980568e-05, 'Fares Fares': 3.3873102402637645e-05, 'James Dean': 3.382742808721519e-05, 'Helen Shaver': 3.3751666045761766e-05, 'Ethan Hawke': 3.37015734255529e-05, 'Bradley Gregg': 3.365155871455114e-05, 'Camilla Belle': 3.360121050966356e-05, 'Iko Uwais': 3.338282904172294e-05, 'Bob Dermer': 3.338085101579974e-05, 'Errol Flynn': 3.3210939312537096e-05, 'Hideaki Ito': 3.3187914664318995e-05, 'Jane Adams': 3.3153240692156e-05, 'Winona Ryder': 3.3091759401471345e-05, 'Edward Furlong': 3.302208743942677e-05, 'James Gandolfini': 3.295947358716245e-05, 'James Caan': 3.2913367545882545e-05, 'Bill Pullman': 3.28919730140476e-05, 'Emma Bell': 3.285072216831996e-05, 'Arnold Friedman': 3.277125383805582e-05, 'Sam Rockwell': 3.2208249979539944e-05, 'Alia Bhatt': 3.218325040762238e-05, 'Bill Nighy': 3.217769111494213e-05, 'Blake Jenner': 3.2173434243343817e-05, 'Aubrey Plaza': 3.194920128110741e-05, 'Jason London': 3.153877175102233e-05, 'Shahid Kapoor': 3.1520979434944616e-05, 'Aqib Khan': 3.146971261947023e-05, 'Rebecca De Mornay': 3.1445362132754605e-05, 'Jason Flemyng': 3.14437579575927e-05, 'Artyom Tkachenko': 3.143823622822747e-05, 'James Cagney': 3.1360007728059354e-05, 'Robert Carradine': 3.130763649335437e-05, 'Olivia Newton-John': 3.0840491366014245e-05, 'Irrfan Khan': 3.084034905153653e-05, 'Dhanush': 3.017625298747635e-05, 'Jamie Dornan': 3.0148590476585175e-05, 'James Spader': 3.0112444992597494e-05, 'Hyun Bin': 2.9739666534990383e-05, 'Allan Edwall': 2.9429281876982617e-05, '≈ûahan G√∂kbakar': 2.8637962184218172e-05, 'Fernanda Montenegro': 2.8067375908817726e-05, 'Fabrizio Bentivoglio': 2.7625068338621787e-05, 'Frederick Stafford': 2.760821884006314e-05, 'Fernando Fern√°n G√≥mez': 2.7599524707185834e-05, 'C. Thomas Howell': 2.7587191024336498e-05, 'Mickey Rourke': 2.758256782125812e-05, 'Heike Makatsch': 2.7562293147028534e-05, 'Burt Lancaster': 2.7503745676143097e-05, 'Michael York': 2.7352399655664572e-05, 'Manuela Velasco': 2.732172192968055e-05, 'Bob Hoskins': 2.71813741009655e-05, 'Vanessa Redgrave': 2.7154499911481488e-05, 'Ben Winchell': 2.7126716144686498e-05, 'Jackie Gleason': 2.712295054746936e-05, 'Christian Slater': 2.6039068702347965e-05, 'Tatsuya Fujiwara': 2.600482705584374e-05, 'Suriya ': 2.5965414102645288e-05, 'Matt Dillon': 2.5937999790026324e-05, 'Michael Fassbender': 2.5918065157049127e-05, 'Romain Duris': 2.5882813153615823e-05, 'G√©rard Depardieu': 2.5680191170808775e-05, 'Geraldine Page': 2.5662387439205527e-05, 'James Le Gros': 2.5562301541807515e-05, 'Hiroyuki Sanada': 2.5480664286227594e-05, 'Tom Berenger': 2.4149788673775757e-05, 'Jada Pinkett Smith': 2.4106593693898835e-05, 'Cuba Gooding Jr.': 2.407653286230678e-05, 'Donald Pleasence': 2.4074164295621186e-05, 'Jack Palance': 2.3934087766009373e-05, 'Ayrton Senna': 2.3901246039605065e-05, 'Jay Chou': 2.3470359379062276e-05, 'Gael Garc√≠a Bernal': 2.3322025655867657e-05, 'Benedict Cumberbatch': 2.327365104218492e-05, 'Tim Robbins': 2.3005334239269712e-05, 'Tom Selleck': 2.288661954157477e-05, 'Ben Browder': 2.280078110434751e-05, 'Marcello Mastroianni': 2.2785612597485827e-05, 'Antonio Albanese': 2.2781210724316866e-05, 'Brian Hooks': 2.2722208089922403e-05, 'Idris Elba': 2.2599493370649715e-05, 'Bubba Smith': 2.258582360385108e-05, 'Greer Garson': 2.257786335298107e-05, 'Mark Strong': 2.255707270744277e-05, 'James Mason': 2.2425619164828542e-05, 'Frances Conroy': 2.199392324559839e-05, "James D'Arcy": 2.098583077060576e-05, 'Derek Luke': 2.084257365577913e-05, 'Benjamin Bratt': 2.0800339951051607e-05, 'Chris Farley': 2.0783640787584917e-05, 'David Niven': 2.0756916427402466e-05, 'Catherine Deneuve': 2.0436557905857653e-05, 'Griffin Dunne': 2.006537556992874e-05, 'Kristen Bell': 2.005686915050368e-05, 'Marilyn Monroe': 1.9977338088552094e-05, 'F. Murray Abraham': 1.989823452385656e-05, 'Dudley Moore': 1.9436584511447658e-05, 'Howard Keel': 1.9408452180972436e-05, 'Imran Khan': 1.9296950868501507e-05, 'Ari Folman': 1.928040071628986e-05, 'Javier Bardem': 1.8724673114024062e-05, 'Ryan Phillippe': 1.8317359446847965e-05, 'Bernie Mac': 1.8084769936627686e-05, 'Fabrice Luchini': 1.795060907077325e-05, 'Barry Pepper': 1.775978058719358e-05, 'Bo Derek': 1.631618248622667e-05, 'Fabio De Luigi': 1.61612665784068e-05, 'Hrithik Roshan': 1.6077221141174978e-05, 'Konstantin Khabenskiy': 1.5909960751276637e-05, 'Zac Efron': 1.5387233885813278e-05, 'Luke Wilson': 1.536077301235068e-05, 'Farrah Fawcett': 1.534015078473475e-05, 'Bing Crosby': 1.5331261623696702e-05, 'Jack Lemmon': 1.5076101408809619e-05, 'Brendan Mackey': 1.4767554284213059e-05, 'Jon Hamm': 1.4646186538559645e-05, 'James Garner': 1.4417008065569555e-05, 'Michael Jai White': 1.437644009518036e-05, 'Choi Min-sik': 1.4137849425920998e-05, 'Gary Busey': 1.4068345446948148e-05, 'James Stewart': 1.4048676721329769e-05, 'Fred Ward': 1.4043405755793305e-05, 'Shah Rukh Khan': 1.3945027618541215e-05, 'John Cena': 1.37192605929154e-05, 'Franco Nero': 1.3712748319609962e-05, 'Spencer Tracy': 1.3666623260559486e-05, 'Patrick Dempsey': 1.3657872787281085e-05, 'Artur ≈ªmijewski': 1.362641701300327e-05, 'John Lithgow': 1.3594587358741551e-05, 'Candy Clark': 1.3393015258709203e-05, 'Charlie Sheen': 1.3378985244191942e-05, 'Ashley Bell': 1.3210354773365647e-05, 'Ariana Richards': 1.2936089283719696e-05, 'Jamie Campbell Bower': 1.2919617728108794e-05, 'Woody Allen': 1.2304747080040825e-05, 'Richard Burton': 1.2303926500365313e-05, 'Hulk Hogan': 1.196656835917801e-05, 'Emma Roberts': 1.1592629433055775e-05, 'Robert Duvall': 1.137608256350153e-05, 'Gabriel Byrne': 1.134319072375143e-05, 'Gugu Mbatha-Raw': 1.132018545010048e-05, 'Bruce Ramsay': 1.113052645760997e-05, 'Alexis Bledel': 1.089450428086715e-05, 'Henry Winkler': 1.0822391130059793e-05, 'Joseph Gordon-Levitt': 1.0726492218403449e-05, 'Julianne Moore': 1.0520764153012344e-05, 'Steve Zahn': 1.0507263693320564e-05, 'Chris Rock': 1.0377909944562546e-05, 'Bruce Dern': 9.957459348437186e-06, 'Donnie Yen': 9.73421031653025e-06, 'Juliette Binoche': 9.603047969474173e-06, 'Carey Mulligan': 9.184451850394317e-06, 'Daniel Day-Lewis': 9.120291667581079e-06, 'Ralph Fiennes': 8.678284330288627e-06, 'Bob Balaban': 8.525589533406271e-06, 'Albert Finney': 8.505525656730492e-06, 'Hailee Steinfeld': 8.474352614546559e-06, 'James Franciscus': 8.24490530775846e-06, 'Jim Belushi': 8.238263291024579e-06, 'Erik Stolhanske': 8.068918593668837e-06, 'Audrey Hepburn': 7.980138998337945e-06, 'Philip Seymour Hoffman': 7.755487858474023e-06, 'Ritesh Deshmukh': 7.294177884958894e-06, 'Griffin Gluck': 7.147386804596003e-06, 'Graham Chapman': 7.091911175549352e-06, 'Barret Oliver': 6.974983013673678e-06, 'Amanda Bynes': 6.9638257978208635e-06, 'John Cusack': 6.936158757906816e-06, 'Carl Weathers': 6.8954919311290835e-06, 'Lou Diamond Phillips': 6.685945585854178e-06, 'Hilary Duff': 6.46936471057975e-06, 'Jason Griffith': 6.32721629555228e-06, 'William H. Macy': 6.211659638143763e-06, 'George Hamilton': 6.161526020263762e-06, 'Chevy Chase': 6.149692111510494e-06, 'Rajinikanth': 6.144097465997761e-06, 'Hal Holbrook': 6.052546530565993e-06, 'Ben Kingsley': 5.8793415192904604e-06, 'Anthony Perkins': 5.686376381370993e-06, 'Vincent Cassel': 5.207651272622622e-06, 'Brittany Murphy': 5.053578806300985e-06, 'Lea Thompson': 5.012654094511463e-06, 'Chris Pine': 4.934230865864709e-06, 'Simon Pegg': 4.557930313339686e-06, 'Fiona Glascott': 4.179415240625814e-06, 'Nick Nolte': 3.8946393932481265e-06, 'Orlando Bloom': 3.860248857685546e-06, 'Hayley Orrantia': 3.745663204780171e-06, 'Christopher Lambert': 3.6983589486048355e-06, 'Christopher Plummer': 3.63083992830218e-06, 'Alex Pettyfer': 3.4214668098891097e-06, 'Keira Knightley': 3.1534495456685633e-06, 'Jason Ian Drucker': 2.752332185419453e-06, 'Jean-Claude Van Damme': 2.7514610300368444e-06, 'Anton Yelchin': 2.655881539638832e-06, 'Farhan Akhtar': 2.606743127945568e-06, 'Andy Lau': 2.539711210890031e-06, 'Mira Sorvino': 2.458140830077174e-06, 'Geoffrey Rush': 2.23972030414279e-06, 'Danny DeVito': 2.2372167221928e-06, 'Hayden Panettiere': 2.195738599516283e-06, 'Hilary Swank': 2.0794691909396547e-06, 'Kip Pardue': 1.9371800501588993e-06, 'Kamal Haasan': 1.8990522935865203e-06, 'Brenda Blethyn': 1.759928339024179e-06, 'Paul Newman': 1.7123927881695163e-06, 'Steve McQueen': 1.6436688392242403e-06, 'Alicia Witt': 1.4161654992794842e-06, 'Bette Midler': 1.3146454234106998e-06, 'Jake T. Austin': 1.284208337944484e-06, 'Sally Field': 1.2613685589078232e-06, 'Richard Anconina': 1.2455741336889108e-06, 'Olivia Wilde': 1.240346418254662e-06, 'Matthew Broderick': 1.233080580327036e-06, 'Amy Adams': 1.0432985318587138e-06, 'Colin Farrell': 9.674883912003682e-07, 'Tina Fey': 9.344755755752697e-07, 'Nicole Kidman': 9.08414029903685e-07, 'John Candy': 8.010147692059726e-07, 'Peter Sellers': 6.882584733055192e-07, 'George C. Scott': 5.675938766420915e-07, 'Alex Vincent': 5.39077413168394e-07, 'Clive Owen': 5.003005563695519e-07, 'Christopher Walken': 4.791709511742204e-07, 'Gad Elmaleh': 4.3378767510274097e-07, 'Bill Farmer': 4.1528410903155994e-07, 'Megumi Ogata': 3.013980292805488e-07, 'Tom Everett Scott': 3.010653225062033e-07, 'Kirk Douglas': 2.7996195023868214e-07, 'Jeff Daniels': 2.389114312981587e-07, 'Cate Blanchett': 2.312822097799778e-07, 'Kate Hudson': 2.1997200979979004e-07, 'Brian Krause': 1.6390966340405797e-07, 'Erica Linz': 1.4460876412216297e-07, 'Rick Moranis': 8.064732096215901e-08, 'Scarlett Johansson': 7.044243722775288e-08, 'Kristen Connolly': 7.011267897921576e-08, 'Colin Firth': 5.437027123410142e-08, 'Gabrielle Union': 2.8573152851364097e-08, 'Mae Whitman': 2.1317008296361526e-08, 'Brian Bedford': 5.277084038618702e-09, 'Garret Dillahunt': 5.082728899361471e-09, 'Diane Lane': 2.6690073099040023e-09, 'Ice Cube': 6.866626408194515e-11, 'Ashley Judd': 1.5511061497201174e-16}
# cast_dict = {'Stan Lee': 0.07043499158245359, 'Judi Dench': 0.025696435428329657, 'Ian McKellen': 0.02407396405673045, 'Jess Harnell': 0.02351134950787031, 'John Ratzenberger': 0.02041991629544609, 'Hugo Weaving': 0.020035556352191352, 'Christopher Lee': 0.01890044326202294, 'Alan Rickman': 0.015357003942242126, 'Jeremy Renner': 0.01448609002549348, 'Tyrese Gibson': 0.014356527382280732, 'Orlando Bloom': 0.013474559773588902, 'Cate Blanchett': 0.011964698578571518, 'Wayne Knight': 0.010410958663678374, 'Scarlett Johansson': 0.010387123807934351, 'Shia LaBeouf': 0.010193528543854049, 'Alan Tudyk': 0.00984487246879159, 'Stanley Tucci': 0.00961548587328254, 'Johnny Depp': 0.009190073341375359, 'Samuel L. Jackson': 0.008749773689496955, 'Vin Diesel': 0.008464559956698278, 'Kurt Russell': 0.008440655691891548, 'Andy Serkis': 0.008267470880002573, 'Dwayne Johnson': 0.00761076079897451, 'Daniel Craig': 0.007583459438934388, 'Robbie Coltrane': 0.007317318040445528, 'Billy Burke': 0.006549614508115862, 'Anna Kendrick': 0.006412096634052166, 'Josh Gad': 0.006391944983806648, 'Tom Lister Jr.': 0.0061984842862014685, 'Natalie Portman': 0.006160130171010379, 'Stellan Skarsg√•rd': 0.005990492488269587, 'Jim Broadbent': 0.0058867990944453516, 'Simon Pegg': 0.00587913772719178, 'Jason Bateman': 0.005733856392338536, 'Tom Cruise': 0.005658812970578, 'Miranda Richardson': 0.005589126336293068, 'Stephen Root': 0.0053293941646343, 'Kristen Wiig': 0.0051260839291948634, 'Tilda Swinton': 0.0050420124108594795, 'Marton Csokas': 0.005037908312797045, 'Ewan McGregor': 0.004971308270765971, 'Mark Ruffalo': 0.004926502077287621, 'Jason Statham': 0.004868010273972, 'Djimon Hounsou': 0.004784506726032687, 'Sandra Bullock': 0.004617764678241468, 'Cheech Marin': 0.004584907391794013, 'Kevin Costner': 0.0045642574453717746, 'Ed Harris': 0.004485015190246649, 'Tom Hanks': 0.004342033275859925, 'Philip Ettington': 0.004225248914984738, 'Dash Mihok': 0.004139876731416, 'Jeff Goldblum': 0.0041252841964641174, 'Nicky Katt': 0.00407804011643831, 'Aaron Eckhart': 0.00398724630482296, 'Elijah Wood': 0.0038893356800706593, 'Matt Damon': 0.0035541359252871385, 'Geoffrey Rush': 0.003508590044783958, 'Michael Sheen': 0.00348276713233765, 'Patrick Stewart': 0.0034826768609242387, 'Emma Thompson': 0.0034570867084566526, 'Harrison Ford': 0.0034139098735403986, 'Liam Neeson': 0.003367579599483551, 'Diane Keaton': 0.0033670800353928373, 'Arnold Schwarzenegger': 0.0033517134873935206, 'Jim Cummings': 0.0032517129535274644, 'Glenn Morshower': 0.003221329299033772, 'Angelina Jolie': 0.0031130374068870205, 'Gary Oldman': 0.0031045907703525005, 'John Leguizamo': 0.003078517252126294, 'Chris Evans': 0.003055314099959157, 'Jamie Foxx': 0.003041863088344595, 'Joey King': 0.003001433857064785, 'Helena Bonham Carter': 0.0029378738570215073, 'Sean Connery': 0.0029324126106418523, 'Stephen Graham': 0.0029066709226310073, 'Matt Walsh': 0.002869210150319979, 'John Turturro': 0.0028342524076644363, 'David Morse': 0.00281041736348288, 'Christian Bale': 0.0028085159957865766, 'Kristen Stewart': 0.002787352808023331, 'Robert Downey Jr.': 0.0027532412519280314, 'Paul Bettany': 0.002738758696791815, 'Bradley Cooper': 0.0027203409592633982, 'Morgan Freeman': 0.002717332323390774, 'James Earl Jones': 0.0027136324288389766, 'Desmond Llewelyn': 0.002684795851694894, 'Karen Allen': 0.0026783281036463517, 'Paul Walker': 0.002621869288142735, 'Matthew Broderick': 0.0026199463402357018, 'Ben Whishaw': 0.0025942570622164633, 'Michael Madsen': 0.0025774866590476163, 'Kevin McNally': 0.0025746842233939017, 'Charlize Theron': 0.0025744964615022742, 'Sigourney Weaver': 0.0025196946408770162, 'Logan Lerman': 0.0024862239514864158, 'Steve Buscemi': 0.002479429205879131, 'Michelle Monaghan': 0.0024504298064885245, 'Miriam Margolyes': 0.0023855324879259376, 'Seann William Scott': 0.002371685171873102, 'J.K. Simmons': 0.0023540586773989616, 'George Clooney': 0.002347486303109832, 'Christopher Fairbank': 0.002291379743496353, 'Stephen Fry': 0.002288655136395858, 'Richard Schiff': 0.002278898677003339, 'Hank Azaria': 0.0022396815424159068, 'Hugh Jackman': 0.002230218679767686, 'Sylvester Stallone': 0.002226770933454497, 'William Fichtner': 0.002216043293748822, 'Brad Pitt': 0.0022003571844913697, 'David Thewlis': 0.002171603623144708, 'Maury Chaykin': 0.0020697811944994813, 'Jim Carrey': 0.002050221460085068, 'Jack Black': 0.002028778982539951, 'Adam Sandler': 0.0020181713768486756, 'Octavia Spencer': 0.002003224854886251, 'Julianne Moore': 0.0019384088847902857, 'Tom Hardy': 0.001930679255685383, 'John Rhys-Davies': 0.0019260093109549434, 'Lukas Haas': 0.0019231656992244669, 'Michael Caine': 0.0018960321206784396, 'Kevin Kline': 0.0018903304434680578, 'Sean Bean': 0.0018900682317304566, 'Christopher Mintz-Plasse': 0.0018436635792332553, 'Julia Roberts': 0.0018434217632343926, 'Keira Knightley': 0.0018417829665538295, 'Mark Wahlberg': 0.0018390803085325449, 'Idris Elba': 0.0018171006565363193, 'Jackie Chan': 0.0018063139150770637, 'Toby Kebbell': 0.001779748911128818, 'Will Yun Lee': 0.0017633298217136287, 'Cameron Diaz': 0.0017567994493937465, 'Jonathan Pryce': 0.0017447627744406808, 'Patrick Dempsey': 0.0017157152810656946, 'Gerard Butler': 0.0016906782456953796, 'John Cleese': 0.00167725762093698, 'Laurence Fishburne': 0.0016580149310090597, 'Cillian Murphy': 0.0016515525190096778, 'Rita Wilson': 0.0016008266869998317, 'Amy Adams': 0.0015881133666525327, 'John Malkovich': 0.0015708610609073037, 'Russell Crowe': 0.0015662162903076423, 'Jessica Alba': 0.0015578971672913292, 'Mel Gibson': 0.0015553393866447674, 'Pierce Brosnan': 0.0015505754450890572, 'Rosamund Pike': 0.0015481678894967161, 'Nicolas Cage': 0.0015301776754062908, 'Kevin Spacey': 0.0015296033193477744, 'Robert De Niro': 0.0015202406768032339, 'Joe Mantegna': 0.0015175580711530895, 'Jeffrey Wright': 0.001513506067510047, 'Andy Garc√≠a': 0.0014879989314489736, 'Kenny Baker': 0.0014793810372963697, 'Dennis Quaid': 0.0014716891351228205, 'Willie Garson': 0.001469398516751673, 'Hugh Grant': 0.0014690554983677936, 'Bill Nighy': 0.0014638942087162878, 'Owen Wilson': 0.0014623289270891064, 'Joaquin Phoenix': 0.0014596297073854554, 'Ken Jeong': 0.001458780831791668, 'Frank Welker': 0.0014533461262266948, 'Michael Gambon': 0.0014461407255930072, 'Roy Scheider': 0.0014455655850129686, 'Leonardo DiCaprio': 0.0014419043813227448, 'Harvey Keitel': 0.0014362997313972967, 'Whoopi Goldberg': 0.0014195413340918036, 'Dan Aykroyd': 0.0014148042049842678, 'Tom Wilkinson': 0.0014023086346091184, 'Leslie Mann': 0.0013981336222837221, 'John Cho': 0.0013941090905191373, 'Jamie Bell': 0.001351832242915658, 'Toby Jones': 0.001350625381468154, 'Candice Bergen': 0.0013466129092581385, 'John Ortiz': 0.0013406780865542508, 'Michelle Rodriguez': 0.0013315361786681375, 'Paul Giamatti': 0.0013278506938245408, 'Michael J. Fox': 0.0013277367304186322, 'Eddie Izzard': 0.0013248780558472687, 'Peter Stormare': 0.0013248140566669638, 'Philip Seymour Hoffman': 0.0013223178066334627, 'Bruce Willis': 0.0013154296182808416, 'Jeffrey Tambor': 0.0013017726969055431, 'Anna Faris': 0.0012973812769304942, 'Vera Farmiga': 0.001292920710416487, 'Jessica Chastain': 0.0012857394213630804, 'Josh Hutcherson': 0.001285311037168791, 'Abigail Breslin': 0.0012749398425908158, 'Jon Lovitz': 0.001269945971496911, 'Holly Hunter': 0.0012688092031245318, 'Max von Sydow': 0.001265404969727594, 'Josh Duhamel': 0.0012646764318494427, 'Alec Baldwin': 0.0012573864204441926, 'Richard Dreyfuss': 0.0012508391355463759, 'Bill Hader': 0.0012425945271468545, 'Heather Graham': 0.0012389816099267522, 'Tony Goldwyn': 0.0012378364328248562, 'Noah Emmerich': 0.0012244572735742132, 'Ryan Reynolds': 0.001220669296429265, 'Nick Offerman': 0.001204638804710739, 'Eddie Murphy': 0.0012030905250355187, 'John Goodman': 0.0011912428837601258, 'Willem Dafoe': 0.0011891441358081705, 'Elizabeth Banks': 0.0011707953011523347, 'Richard Harris': 0.0011607378164544566, 'Philip Baker Hall': 0.0011487098010625853, 'Jeff Bridges': 0.001143307469812248, 'Giovanni Ribisi': 0.0011154164753967018, 'Jane Lynch': 0.0011049040032533194, 'Brendan Gleeson': 0.0010978823262538283, 'Ian McShane': 0.00109567217361228, 'Chris Cooper': 0.0010786460115474981, 'Famke Janssen': 0.0010653256597136446, 'Zoe Saldana': 0.0010570903536127531, 'Fiona Shaw': 0.0010480671911695621, 'Jim Caviezel': 0.0010313177346680993, 'Eugene Levy': 0.0010304884450209836, 'Milla Jovovich': 0.0010226072652659369, 'Frank Langella': 0.0010197368160403148, 'Molly Shannon': 0.001019484198556468, 'Kirsten Dunst': 0.0010109063318207623, 'Missi Pyle': 0.0010065544323307667, 'Danny Mann': 0.001001465618875719, 'John Travolta': 0.0009931872387114568, 'Jane Krakowski': 0.0009819921150957156, "Denis O'Hare": 0.0009699206216307715, 'Jon Voight': 0.0009493240932518474, 'Carla Gugino': 0.0009451017905514833, 'Dominic West': 0.0009427257017846751, 'Pen√©lope Cruz': 0.0009347949207106871, 'Kurtwood Smith': 0.0009166336255418767, 'Donald Sutherland': 0.0009158949817367068, 'Jay Leno': 0.0009157188295265539, 'Michael McKean': 0.0009137557842782941, 'Tobey Maguire': 0.000912767748301406, 'Marlon Brando': 0.0009035703385255102, 'Rob Riggle': 0.0008794919430205447, 'Zeljko Ivanek': 0.0008687888016508931, 'Lili Taylor': 0.0008566459448344817, 'Al Pacino': 0.0008423498646480633, 'Josh Peck': 0.000842302170481045, 'Shane Rimmer': 0.0008409569391433905, 'Tony Shalhoub': 0.0008403395425103319, 'Parker Posey': 0.0008393389943730176, 'Ellen Page': 0.0008260831974097433, 'Matt Dillon': 0.0008260813985185861, 'Michael Douglas': 0.0008121072557522432, 'Xander Berkeley': 0.0008078461054734497, 'Michael Pe√±a': 0.0008026490746679188, 'Cary Elwes': 0.0007873725399396632, 'Channing Tatum': 0.0007801416735031357, 'Queen Latifah': 0.0007674949214477265, 'Judy Greer': 0.0007649223269610517, 'Mark Margolis': 0.0007633338817088968, 'Kevin Bacon': 0.0007630591653472973, 'Steve Carell': 0.0007610032879818488, 'Michael Shannon': 0.0007609918992941783, 'Luis Guzm√°n': 0.0007545097030876214, 'Kevin Dunn': 0.0007483948813525535, 'Lance Henriksen': 0.0007386838142672646, 'Michael Fassbender': 0.0007328534838359285, 'Robin Williams': 0.0007320907072441276, 'Ralph Fiennes': 0.0007289087310442817, "Sean O'Bryan": 0.0007192829524922128, 'Adrien Brody': 0.0007136112847757356, 'John Michael Higgins': 0.0007119722156923598, 'Christopher Walken': 0.0007036832956807428, 'Christopher Plummer': 0.0006996783810390746, 'Meg Ryan': 0.0006959231099553512, 'Bill Cobbs': 0.0006937498728308696, 'Steve Zahn': 0.0006925065810521396, 'Kurt Fuller': 0.0006908061770687375, 'Jennifer Coolidge': 0.0006860597321264062, 'Amanda Peet': 0.0006781785062996516, 'Catherine Zeta-Jones': 0.0006644726634352341, 'Fisher Stevens': 0.0006615129272269933, 'Catherine Keener': 0.0006492199785458332, 'Lea Thompson': 0.0006471357282490765, 'Brian Cox': 0.0006454386795985776, 'Patrick Swayze': 0.0006397579603801707, 'James Marsden': 0.0006370101731194523, 'Patrick Malahide': 0.000623552878400367, 'Melissa McCarthy': 0.0006156596354432315, 'Andrea Martin': 0.0006121573525172332, 'Denzel Washington': 0.0006058247522258437, 'Laura Linney': 0.0006008425798983758, 'Richard Gere': 0.0005986287272351225, 'Christian Slater': 0.0005980238707493635, 'Delroy Lindo': 0.0005879279388058071, 'Jeremy Piven': 0.0005866730616093609, 'Richard E. Grant': 0.0005793864421024658, 'Susan Sarandon': 0.0005792969050354626, 'Seth Green': 0.0005763140784484907, 'Jennifer Connelly': 0.0005728455301906266, 'Don McManus': 0.0005722404999494693, 'Richard Jenkins': 0.0005718114631395423, 'Cliff Curtis': 0.0005711573578925398, 'Tracey Walter': 0.0005703849978044845, 'Danny Huston': 0.0005671424213159195, 'Colin Farrell': 0.0005647710934691964, 'Olivia Wilde': 0.0005627001413706915, 'Harris Yulin': 0.0005579408859162506, 'Sam Elliott': 0.0005564701804827241, 'Michael Rooker': 0.0005543362029858461, 'Casey Affleck': 0.0005537510334923173, 'Marisa Tomei': 0.0005434438519972053, 'Lois Smith': 0.0005427941468140667, 'Rachel McAdams': 0.0005363910053559486, 'Jason Lee': 0.0005267220548351387, 'Dan Hedaya': 0.0005263526214485419, 'Bob Gunton': 0.0005259822791070431, 'Michael Biehn': 0.0005248673366947749, 'Richard Crenna': 0.0005207415408849907, 'Frank Grillo': 0.0005150101842866713, 'James Franco': 0.0005083425083026512, 'Frances McDormand': 0.0005071840400046483, 'Thomas Kretschmann': 0.0005059371219342188, 'Jeremy Sisto': 0.0005055486071677534, 'Anthony Hopkins': 0.0005044371444383142, 'Scoot McNairy': 0.0004962874494031178, 'Antonio Banderas': 0.0004955136484165775, 'Mike Starr': 0.0004921257347358078, 'Bruce McGill': 0.0004909671571101704, 'James Woods': 0.0004897458722944237, 'Danny DeVito': 0.000489556431655458, 'Ray Liotta': 0.0004889092043617967, 'Annette Bening': 0.0004854283681344924, 'Oliver Platt': 0.000483199238525165, 'Vincent Cassel': 0.0004786603737983356, 'Helen Mirren': 0.0004758079126669184, 'Michael Kenneth Williams': 0.0004638774089388227, 'Naomi Watts': 0.00045698329966829994, 'Isla Fisher': 0.0004540473963526879, 'Mark Strong': 0.00044845478238676815, 'Ciar√°n Hinds': 0.00044816176209717923, 'Reese Witherspoon': 0.00044342646291142503, 'Tim Robbins': 0.0004410280269371406, 'Pat Hingle': 0.0004392434907179851, 'Winona Ryder': 0.00043882539824595054, 'Jonah Hill': 0.00043462259772038644, 'John C. McGinley': 0.0004318238057976938, 'Allison Janney': 0.0004306866563662243, 'Rachael Harris': 0.00042918010438562914, 'Barry Pepper': 0.00042720544229679636, 'Margo Martindale': 0.0004271931181089119, 'Ewen Bremner': 0.0004256527280822732, 'Troy Evans': 0.0004234936196470198, 'Bill Paxton': 0.00042345905080885436, 'Jack McGee': 0.00042256698584758654, 'John Hurt': 0.0004208299224501744, 'Sam Worthington': 0.0004204998228201475, 'Liev Schreiber': 0.000420198736942898, 'Woody Harrelson': 0.0004174082370985645, 'Ben Stiller': 0.00041718492182493406, 'Billy Slaughter': 0.0004171350195640303, 'Joel Edgerton': 0.00041694968372430353, 'Loretta Devine': 0.0004157358762895384, 'Rutger Hauer': 0.0004139530907414296, 'Bruce Greenwood': 0.0004112432380240755, 'Gene Hackman': 0.00040982441930635845, 'Scott Glenn': 0.0004060556792780851, 'Kris Kristofferson': 0.00040512465887171997, 'John C. Reilly': 0.000402247659024, 'E.G. Daily': 0.0003996424800511253, 'Colin Salmon': 0.0003979769811281791, 'Ethan Hawke': 0.000397804449032544, 'Ashley Judd': 0.00039724310103134917, 'Guy Pearce': 0.00039652557610428163, 'Ben Affleck': 0.0003894691431167032, 'Patton Oswalt': 0.000384049710479221, 'Adam Scott': 0.0003805806742673839, 'Gabriel Byrne': 0.00038005089605068346, 'Kathy Bates': 0.00037825715648139366, 'Clancy Brown': 0.00037752544347163336, 'Joe Pantoliano': 0.00036923857606280675, 'James Cromwell': 0.00036655611553278086, 'Halle Berry': 0.00036248600377293907, 'Barbara Hershey': 0.0003609589516561162, 'Angela Bassett': 0.00035841373389280154, 'Tom Waits': 0.00035809004321378626, 'Bette Midler': 0.00035480352573220316, 'Michael Ironside': 0.0003534801290198787, 'Robert Prosky': 0.0003526879115718914, 'James Remar': 0.0003518915648249936, 'Chlo√´ Grace Moretz': 0.00034958307166470216, 'Jennifer Aniston': 0.00034834583567756595, 'Kathryn Hahn': 0.00034546193114023347, 'Tim Blake Nelson': 0.00034209686721891225, 'Charles Dance': 0.0003405090363747155, 'Jake Gyllenhaal': 0.0003387887014494805, 'Terence Stamp': 0.000338191829272901, 'Michael Keaton': 0.00033561091118411684, 'Robert Redford': 0.0003354228056628266, 'Terry Crews': 0.00033480149496329195, 'Olivia Williams': 0.00033267202091763904, 'Bradley Whitford': 0.0003317131347901613, 'Dave Franco': 0.00033142585500391136, 'Jon Hamm': 0.00032954773505947403, 'Lee Garlington': 0.0003288357826150527, 'Charles S. Dutton': 0.00032313492175786707, 'Carrie Fisher': 0.0003216914284541684, 'Anthony Edwards': 0.00032134973669797917, 'Ed Lauter': 0.0003209816304563836, 'Frankie Faison': 0.000316963942807658, 'Keanu Reeves': 0.0003144315196215124, 'Meryl Streep': 0.00030909476316716346, 'Zach Galifianakis': 0.0003083523458612794, 'Eric Stoltz': 0.0003045035961048234, 'Ron Perlman': 0.0003026001366299281, 'Vinnie Jones': 0.0002991053920961267, 'Ryan Phillippe': 0.0002974400790050018, 'Sharon Stone': 0.0002923516161042489, 'Alan Cumming': 0.0002921739485566072, 'Danny Trejo': 0.00028776685373902534, 'Kristin Scott Thomas': 0.00028574602703814703, 'Bobby Cannavale': 0.0002851156055718782, 'Robert Davi': 0.0002837546932847179, 'Bill Murray': 0.0002828746621172797, 'Dianne Wiest': 0.00028173521395238475, 'Will Ferrell': 0.0002751536913113981, 'Alfred Molina': 0.0002740701311514396, 'Cuba Gooding Jr.': 0.00027383724423708626, 'John Carroll Lynch': 0.0002676895332159296, 'James Caan': 0.0002661816809508889, 'Greg Kinnear': 0.0002661455181455276, 'Nick Nolte': 0.0002658195237287334, 'Nicole Kidman': 0.0002636190211138356, 'Jeremy Strong': 0.000261836923148297, 'Gary Cole': 0.0002611605357246061, 'Benicio del Toro': 0.0002595519660302611, 'Robert Duvall': 0.0002594091721887253, 'Diane Lane': 0.00025925356342395417, 'Edward Norton': 0.00025576062847816166, 'Regina King': 0.0002554569984334127, 'Peter Boyle': 0.00025525844063029317, 'Wallace Shawn': 0.0002537830001460432, 'Stacy Keach': 0.0002534032728683402, 'Rebecca De Mornay': 0.00025329523614832, 'Josh Lucas': 0.00025190931515538603, 'Stephen Tobolowsky': 0.00025111898483396914, 'Sydney Pollack': 0.0002502875059131699, 'Jared Harris': 0.00024832284203209003, 'Clint Howard': 0.00024745480123645433, 'Felicity Jones': 0.0002470872128098068, 'Terrence Howard': 0.0002464299082472971, 'Dick Miller': 0.00024557843670645457, 'Seth Rogen': 0.0002435024751224689, 'Anna Paquin': 0.0002413714856703482, 'Edgar Ram√≠rez': 0.00023988474183754918, 'Colleen Camp': 0.00023833283856532, 'Victor Argo': 0.00023754791638520622, 'Caroline Aaron': 0.00023668329719295193, 'John Hawkes': 0.0002345890761198943, 'Michelle Pfeiffer': 0.00023401760715574047, 'Kiefer Sutherland': 0.0002331885827381475, 'Keith David': 0.0002328551082883557, 'Danny Glover': 0.00023093374238614988, 'Jason Isaacs': 0.00022963093486835307, 'J√ºrgen Prochnow': 0.00022772399599897464, 'Robert Picardo': 0.00022655828491994004, 'Matt Craven': 0.00022608970192708683, 'Ryan Gosling': 0.00022564028246822317, 'Rachel Weisz': 0.0002254576823766486, 'Jason Clarke': 0.00022487527604296982, 'Joe Lo Truglio': 0.00022477250568641015, 'Jennifer Lopez': 0.00022458635110094273, 'Mary Ellen Trainor': 0.00022338541610001503, 'James Coburn': 0.00022101192817018423, 'Jim Belushi': 0.00021861166360114654, 'Benjamin Bratt': 0.00021839308719193585, 'Shea Whigham': 0.00021560356060388165, 'Joe Spinell': 0.00021320645327019627, 'Brian Doyle-Murray': 0.00021283663435923565, 'Kim Basinger': 0.00021222602745928076, 'Rob Schneider': 0.00021169258147266985, 'Amy Ryan': 0.00021037979069494424, 'Tim Curry': 0.00021026878260226245, 'Giancarlo Esposito': 0.0002092462446366816, 'Tom Arnold': 0.00020839233472688085, 'Uma Thurman': 0.00020797455183538895, 'Ralph Brown': 0.00020712295234213263, 'Victor Garber': 0.0002061766029974685, 'Cary-Hiroyuki Tagawa': 0.00020606316524823077, 'Jon Favreau': 0.00020605357193561086, 'Ben Foster': 0.00020527903531601844, 'Brion James': 0.00020519233727143814, 'Lindsay Lohan': 0.0002028779975697114, 'David Paymer': 0.00020233029507162119, 'Drew Barrymore': 0.00020205010409883863, 'Paul Dooley': 0.00020176777410454088, 'Allen Covert': 0.00020155362182627018, 'J.D. Evermore': 0.00020133016351558727, 'Dennis Hopper': 0.0001984836152486949, 'Gary Busey': 0.00019797701452586093, 'Robin Wright': 0.00019771993170250606, 'J.T. Walsh': 0.0001976855959907758, 'Jet Li': 0.00019429656414118412, 'Kevin Nealon': 0.00019277713333515164, 'Katherine Heigl': 0.00019222598705947606, 'Bill McKinney': 0.00019048562206778262, 'Katie Holmes': 0.00018908921154333567, 'Peter Falk': 0.00018758128715126086, 'William Hurt': 0.0001874762909031133, 'Steve Martin': 0.00018740067348805417, 'Holt McCallany': 0.00018710637709717383, 'Roger Allam': 0.0001868183078886182, 'Vanessa Redgrave': 0.0001844521405051219, 'Stacey Travis': 0.00018231019799636175, 'January Jones': 0.00017946900480735718, 'Juliette Lewis': 0.00017889320848781136, 'Cloris Leachman': 0.00017644449523105318, 'Kyra Sedgwick': 0.00017514140456045374, 'Kevin Pollak': 0.0001734394157057744, 'John Mahoney': 0.00017228524600887012, 'Sam Shepard': 0.00017185423964013682, 'Dylan Baker': 0.00017182321498527138, 'Anjelica Huston': 0.0001712113792173486, 'Danielle Harris': 0.00017097939979306267, 'Tommy Lee Jones': 0.00017026474200846413, 'Charlton Heston': 0.00016947474147128874, 'Joe Chrest': 0.00016870894827764022, 'Josh Brolin': 0.00016802634074341007, 'Sean Penn': 0.00016720978501636412, 'Melanie Lynskey': 0.00016542592531811501, 'Mickey Rourke': 0.00016517361032023328, 'David Krumholtz': 0.00016439140777422902, 'James Gandolfini': 0.0001631599328978525, 'Peter Jason': 0.00016304639381416712, 'John Lithgow': 0.00016243619224740532, 'David Jensen': 0.00015891461215685067, 'Dolph Lundgren': 0.00015836955885267025, 'Joe Don Baker': 0.00015719685873836993, 'Josh Charles': 0.000156746962956034, 'Brent Briscoe': 0.00015575258419387003, 'Viggo Mortensen': 0.000155555986080414, 'Kathy Baker': 0.000154763648549766, 'Marshall Bell': 0.00015461271886983764, 'Tom Hollander': 0.00015405497047439492, 'William H. Macy': 0.0001531993279294585, 'Maria Bello': 0.00015312904956690248, 'Emily Watson': 0.00015238352575233817, 'Josh Pais': 0.00015083958612381387, 'Melissa Leo': 0.00015027587475045814, 'Don Cheadle': 0.00014973185282440848, 'Dustin Hoffman': 0.0001479855468295834, 'Harry Dean Stanton': 0.0001471834133831234, 'Justin Long': 0.0001460335369421858, 'Burt Reynolds': 0.00014597092495053694, "Catherine O'Hara": 0.0001448298747192279, 'Matthew McConaughey': 0.00014372352408018773, 'Ren√©e Zellweger': 0.0001427675283642122, 'Wesley Snipes': 0.00014009684457551888, 'Rance Howard': 0.00013953947640719236, 'Viola Davis': 0.00013899968316444928, 'Jude Law': 0.00013788126435005488, 'Clint Eastwood': 0.00013649989521732268, 'Chris Rock': 0.00013635479127841615, 'Brian Dennehy': 0.00013578029622831605, 'Forest Whitaker': 0.0001357310551036367, 'Frank McRae': 0.0001354098721964003, 'Tom Kenny': 0.00013385595242161602, 'David Koechner': 0.00013172543877162257, 'Will Patton': 0.0001315895550702545, 'Domenick Lombardozzi': 0.00012999908631816513, 'Burt Lancaster': 0.00012917424637885015, 'Bill Pullman': 0.00012842427042591807, 'Val Kilmer': 0.00012770931177567306, 'John Krasinski': 0.00012745531666648113, 'Cheryl Hines': 0.00012732734268085184, 'Caroline Goodall': 0.00012730338728078017, 'Patricia Clarkson': 0.00012691053169156032, 'Frances Conroy': 0.00012682728910103024, 'Timothy Olyphant': 0.000126537884544008, 'Eddie Marsan': 0.0001255079100897128, 'Jason Flemyng': 0.00012407343722844463, 'Ian Holm': 0.00012398248436309474, 'Paul Dano': 0.0001225712657182162, 'Hal Holbrook': 0.00012217584549718117, 'Dean Norris': 0.00012192407678484786, 'Dom DeLuise': 0.00012178244921422455, 'F. Murray Abraham': 0.0001182650396781957, 'Tate Donovan': 0.00011815497084963385, 'Corey Johnson': 0.00011800293789352208, 'John Gielgud': 0.00011757534145278334, 'Rosario Dawson': 0.00011507132526121546, 'Elias Koteas': 0.00011478615994836297, 'Kenneth Welsh': 0.00011449031208997907, 'James Hong': 0.00011359328921875949, 'Robert Patrick': 0.00011357339948866883, 'Jeff Daniels': 0.00011349818093623755, 'Dennis Haysbert': 0.00011330409514101743, 'Ethan Suplee': 0.00011255394344236884, 'Martin Short': 0.00011152475529439807, 'Jennifer Tilly': 0.00011047855522766304, 'Richard Kind': 0.00010968847795926083, 'Woody Strode': 0.00010962171003193718, 'Dean Stockwell': 0.00010869419013230592, 'David Niven': 0.00010835924444244874, 'Steve Coogan': 0.00010802783889193185, 'Beth Grant': 0.0001074001909309253, 'Woody Allen': 0.00010646301391486347, 'Vince Vaughn': 0.0001061320467483152, 'Tch√©ky Karyo': 0.00010568960870454194, 'Bridgette Wilson': 0.00010484615425877147, 'Charles Durning': 0.00010448766671932187, 'Ben Kingsley': 0.00010427399219747552, 'John Heard': 0.00010265258679916401, 'Dermot Mulroney': 0.00010197131847444422, 'Donald Faison': 0.00010083234362704702, 'Brendan Fraser': 9.956850774350213e-05, 'Ann Dowd': 9.917334130469808e-05, 'Rosanna Arquette': 9.893824211059457e-05, "Vincent D'Onofrio": 9.767941950486711e-05, 'Charlie Sheen': 9.760848917991946e-05, 'Michael Papajohn': 9.685162725647411e-05, 'Romany Malco': 9.616269819750885e-05, 'Bruce Campbell': 9.614796313010545e-05, 'Cedric the Entertainer': 9.579907661099872e-05, 'Marco Perella': 9.551028548879874e-05, 'Kristy Swanson': 9.539475974467077e-05, 'Jeffrey Dean Morgan': 9.46969316090989e-05, 'John Cusack': 9.40126898927552e-05, 'Gwyneth Paltrow': 9.313879921245773e-05, 'Thomas Haden Church': 9.081377285870717e-05, 'Anthony Mackie': 9.078240713963835e-05, 'Toni Collette': 8.993508136861208e-05, 'Stephen Lang': 8.932088562648441e-05, 'Lindsay Crouse': 8.889455254033817e-05, 'Paul Rudd': 8.79938803302397e-05, 'Aasif Mandvi': 8.344843808088059e-05, 'Ned Beatty': 8.094492151248411e-05, 'Thomas Lennon': 8.070597194737624e-05, 'Art LaFleur': 7.94761268989432e-05, 'Salma Hayek': 7.930858725963241e-05, 'Juliette Binoche': 7.909562133371769e-05, 'Wayne P√©re': 7.684568831889862e-05, 'Michael Rispoli': 7.665093459062133e-05, 'Robert Carradine': 7.553023254163808e-05, 'Burt Kwouk': 7.332146314985012e-05, 'Alan Arkin': 7.159004860753568e-05, 'Rosemarie DeWitt': 7.121674385469395e-05, 'Jena Malone': 7.10097770196589e-05, 'Kate Winslet': 7.080498654702756e-05, 'Rebel Wilson': 6.980543889820806e-05, 'Brad Dourif': 6.970532691163948e-05, 'Joseph Gordon-Levitt': 6.820264241296211e-05, 'Radha Mitchell': 6.713810946685186e-05, 'Emile Hirsch': 6.617292390174642e-05, 'Fred Ward': 6.5655873251173e-05, 'Rick Overton': 6.497791487747819e-05, 'Marcia Gay Harden': 6.287559392228673e-05, 'Larry Miller': 5.929926510097885e-05, 'John Sessions': 5.739667724996877e-05, 'Jenifer Lewis': 5.606200885418123e-05, 'Jean-Claude Van Damme': 5.38268471701004e-05, 'Clea DuVall': 5.19635045829034e-05, 'Albert Finney': 5.1745095054599696e-05, 'Luke Wilson': 4.994187893983098e-05, 'Beau Bridges': 4.93028061089168e-05, 'Jane Fonda': 4.908540135376316e-05, 'Chevy Chase': 4.877034427026708e-05, 'George Kennedy': 4.711172341378008e-05, 'Ice Cube': 4.6477113757413736e-05, 'M.C. Gainey': 4.581306228082407e-05, 'Eva Mendes': 4.4901004957850864e-05, 'Bruce Dern': 4.481807050943339e-05, 'Derek Luke': 4.460572276055782e-05, 'Om Puri': 4.362279719173747e-05, 'Robert Wagner': 4.31929693318841e-05, 'Paul Newman': 4.279838441563292e-05, 'H√©ctor Elizondo': 4.27031770688362e-05, 'Edward Herrmann': 4.2138676233650337e-05, "Terry O'Quinn": 4.129299717149491e-05, 'Richard Bradford': 4.0514039377407414e-05, 'Julie Delpy': 3.8183924672064825e-05, 'Kelly Preston': 3.7391547066650994e-05, 'Tony Curtis': 3.665001073351491e-05, 'Jack Lemmon': 3.5295035977255416e-05, 'Tom Sizemore': 3.4395390537684215e-05, 'M. Emmet Walsh': 3.2125363308847485e-05, "Michael O'Keefe": 3.0314003032481562e-05, 'Nora Dunn': 3.014168369689136e-05, 'Isabella Rossellini': 2.946525205590188e-05, 'Evan Rachel Wood': 2.9460536812116242e-05, 'Jennifer Jason Leigh': 2.781174592066267e-05, 'John P. Ryan': 2.7595943237468982e-05, 'Jeremy Northam': 2.673880101766531e-05, 'Chuck Norris': 2.5849775278308358e-05, 'Edward Burns': 2.49553894676117e-05, 'Martin Landau': 2.484706050357326e-05, 'Emma Stone': 2.229863626985379e-05, 'Pip Torrens': 2.1432253811783305e-05, 'John Wayne': 2.0810462120630756e-05, 'Billy Bob Thornton': 2.074303563392916e-05, 'Geoffrey Lewis': 2.0685751551429183e-05, 'Zach Grenier': 1.788128509441518e-05, 'Richard Roundtree': 1.6596259423065517e-05, 'Maggie Gyllenhaal': 1.5408470970738505e-05, 'Breckin Meyer': 1.5000132309671298e-05, 'Richard Masur': 1.4758620448827783e-05, 'Bess Flowers': 1.1207962361556232e-05, 'Shah Rukh Khan': 5.6578360316781e-07}
# com_dict = get_scores_dict('production_companies')
# print(com_dict)
com_dict = {'Walt Disney Pictures': 0.21085509264708321, 'Legendary Pictures': 0.09920367051166508, 'Warner Bros.': 0.06971302318288154, 'Amblin Entertainment': 0.06139436292850993, 'TSG Entertainment': 0.05098297049409419, 'Universal Pictures': 0.04862677266031785, 'Dune Entertainment': 0.04452258598998357, 'Paramount Pictures': 0.04435507302487634, 'Metro-Goldwyn-Mayer (MGM)': 0.04334378015377291, 'New Line Cinema': 0.04238804348346703, 'Original Film': 0.03776391619762112, 'Columbia Pictures': 0.03765777568137933, 'Relativity Media': 0.03727533067596223, 'Twentieth Century Fox Film Corporation': 0.025041548704410128, 'Village Roadshow Pictures': 0.01825288630991343, 'DreamWorks SKG': 0.013073497423186572, 'Fox 2000 Pictures': 0.010429639851734968, 'Summit Entertainment': 0.009947684837735858, 'Regency Enterprises': 0.009852731939866542, 'Touchstone Pictures': 0.008060245033641323, 'Working Title Films': 0.0077701153763430385, 'Spyglass Entertainment': 0.006911540083438895, 'Canal+': 0.005979590325935646, 'TriStar Pictures': 0.0055443035395846265, 'Columbia Pictures Corporation': 0.005360987992247041, 'New Regency Pictures': 0.004303038424419431, 'United Artists': 0.004204584815791845, 'Lionsgate': 0.0031832616933699467, 'StudioCanal': 0.002879911284392863, 'Imagine Entertainment': 0.0028688970611830014, 'Miramax Films': 0.0027891470464148013, 'Blumhouse Productions': 0.0027442874801823962, 'Millennium Films': 0.00259653339386092, 'Screen Gems': 0.0024416302440445894, 'Lakeshore Entertainment': 0.002194061922601432, 'Dimension Films': 0.002173981989918337, 'Epsilon Motion Pictures': 0.0020665670930686358, 'BBC Films': 0.0020358210454207563, 'Castle Rock Entertainment': 0.0018745548634686228, 'Participant Media': 0.0014716116752551771, 'Fox Searchlight Pictures': 0.0013669095631639004, 'Silver Pictures': 0.0010496639499699824, 'Hollywood Pictures': 0.0008462318681984852, 'Scott Rudin Productions': 0.0007796512302341163, 'France 2 Cinéma': 0.00037214617902142655, 'Focus Features': 0.0003655371163684348, 'Morgan Creek Productions': 0.00036288192906197667, 'Film4': 0.0003417980433321104, 'Orion Pictures': 0.0002545687952345243, 'The Weinstein Company': 0.00012555224236985492}
# country_dict = get_scores_dict('production_countries')
country_dict = {'United States of America': 0.2989976606231105, 'New Zealand': 0.1938067083154506, 'United Kingdom': 0.08778724678965183, 'Canada': 0.08583954504276206, 'Czech Republic': 0.07276927505749703, 'Germany': 0.022490776357778235, 'France': 0.021414872987119377, 'United Arab Emirates': 0.019609678556912695, 'Italy': 0.016774116888116166, 'Australia': 0.015533697368071938, 'Morocco': 0.014583340979145348, 'Malta': 0.013648927103906042, 'China': 0.012591527617347456, 'Japan': 0.011709149843104477, 'Ireland': 0.011035107430057398, 'Taiwan': 0.010278920721408137, 'Hong Kong': 0.010088964294074024, 'Spain': 0.008494275021441575, 'Netherlands': 0.007170425944387357, 'South Korea': 0.00703355119918134, 'Russia': 0.0059081809747370696, 'South Africa': 0.004160097697346236, 'India': 0.0035673908632360674, 'Belgium': 0.0032301615277180043, 'Qatar': 0.002549462231472539, 'Brazil': 0.0025049700392635716, 'Mongolia': 0.0024880676460193505, 'Bahamas': 0.002427040301952415, 'Chile': 0.0020615245054321435, 'Luxembourg': 0.0020349872186934794, 'Romania': 0.0018755708939631088, 'Sweden': 0.0018467514099028688, 'Iceland': 0.0018174787584289832, 'Greece': 0.001791427250167654, 'Hungary': 0.001745263273079482, 'Israel': 0.0015137681696520632, 'Switzerland': 0.0014957956455159621, 'Denmark': 0.0014359739879974884, 'Turkey': 0.001342270986721542, 'Indonesia': 0.0009711095917970013, 'Mexico': 0.0009634986134672065, 'Ghana': 0.0009426161011451776, 'Peru': 0.0008326274408967285, 'Philippines': 0.0007585423638004329, 'Finland': 0.0007174370558812039, 'Namibia': 0.0006800161125878028, 'Austria': 0.0006794499245611535, 'Norway': 0.0006454965651261116, 'Bulgaria': 0.0006129863930874343, 'Poland': 0.0005553848321374108, 'Costa Rica': 0.0005190587645151675, 'Slovenia': 0.00048110092062111156, 'Burkina Faso': 0.0004039783290622154, 'Bosnia and Herzegovina': 0.00037444691278059694, 'Paraguay': 0.00036102910245488785, 'Serbia and Montenegro': 0.0003556852029777104, 'Serbia': 0.0003049204840395361, 'Puerto Rico': 0.0003022026450293975, 'Croatia': 0.00029460264024065556, 'Iran': 0.000138136567298852, 'Cambodia': 0.00011373642139039214, 'Mauritania': 9.379838265323938e-05, 'Uruguay': 9.025727561372606e-05, 'Portugal': 8.355737481796943e-05, 'Argentina': 8.201790025643096e-05, 'Algeria': 5.17798989770829e-05, 'Cyprus': 4.530718485265907e-05, 'Singapore': 4.454473263424258e-05, 'Pakistan': 1.9160957129395332e-05, 'Tunisia': 1.5361337936184845e-05, 'Saudi Arabia': 6.03613672222033e-06, 'Ukraine': 2.4287391279813553e-06, 'Jordan': 2.012045574069991e-06, 'Ethiopia': 1.7235269830867042e-06}
# genre_dict = get_scores_dict('genres')
genre_dict = {'Adventure': 0.25210773489340677, 'Drama': 0.1273215647376299, 'Animation': 0.11046561715870509, 'Thriller': 0.06322942079405194, 'Action': 0.06148278323793743, 'Fantasy': 0.059837041496161356, 'Romance': 0.05401860615800932, 'Science Fiction': 0.04623336770216128, 'Family': 0.04607131713502517, 'Comedy': 0.04169238987792928, 'Horror': 0.03697467704457309, 'Mystery': 0.020415716191705374, 'Music': 0.020409588498730643, 'Crime': 0.019422482255111208, 'War': 0.016661040410904576, 'Documentary': 0.007475486266885209, 'History': 0.006602718230398562, 'Western': 0.005949602752104889, 'TV Movie': 0.0020951067320263816, 'Foreign': 0.0015337384265425417}
crew_dict = get_scores_dict('crew')
print(crew_dict)
crew_dict = {'Peter Jackson': 0.3607585833952878, 'Michael Bay': 0.22788508075832561, 'Steven Spielberg': 0.19870629923784947, 'Jon Turteltaub': 0.0325603224159614, 'Ridley Scott': 0.029789421316623193, 'Todd Phillips': 0.026656691262478594, 'Tim Burton': 0.025348163208238045, 'Ron Howard': 0.020528508879087113, 'Paul W.S. Anderson': 0.012870197832332264, 'Wolfgang Petersen': 0.012063284091476239, 'Tony Scott': 0.007507925252101475, 'Ivan Reitman': 0.006701125863214139, 'John Glen': 0.005124794246015245, 'Billy Wilder': 0.003161555869235673, 'Alfred Hitchcock': 0.003054986654732341, 'Garry Marshall': 0.0027003887450840944, 'Blake Edwards': 0.0026102018071695377, 'Michael Mann': 0.0022647740408133915, 'Ralph Bakshi': 0.001978492390377222, 'Richard Linklater': 0.0019709967327820855, 'Luis Mandoki': 0.0017744945539599631, 'John Frankenheimer': 0.0017513420262908967, 'Sidney Lumet': 0.0015331505757758797, 'John Huston': 0.0014378253057944943, 'Lasse Hallstr√∂m': 0.0013841503688306243, 'Wes Anderson': 0.001043541574545877, 'Rob Reiner': 0.00092482735002796, 'Joel Schumacher': 0.0007281160983497745, 'Steven Soderbergh': 0.0006607362209407297, 'Peter Hyams': 0.0005656935012359114, 'Brian De Palma': 0.0005246693899198042, 'Wes Craven': 0.0004573253327711163, 'George P. Cosmatos': 0.00045215562393440684, 'Ted Kotcheff': 0.00044907800720037257, 'Francis Ford Coppola': 0.00044095157040038304, 'Paul Weitz': 0.00036336062352983775, 'Gary Goldman': 0.0003473368230160132, 'Harold Ramis': 0.00023535688669234405, 'Martin Scorsese': 0.0001539734959086227, 'Robert Rodriguez': 9.142532354653883e-05, 'David R. Ellis': 8.773798655048142e-05, 'Clint Eastwood': 7.409114409372124e-05, 'Woody Allen': 5.694214669088703e-05, 'Luc Besson': 4.7103741926278544e-05, 'Renny Harlin': 4.6575787760929886e-05, 'Oliver Stone': 4.632281521659503e-05, 'Walter Hill': 4.4648892630675684e-05, 'John Singleton': 2.6101721913362375e-05, 'Roger Donaldson': 9.170480750471188e-06, 'Barry Levinson': 6.306102198553238e-10}
# crew_dict = {'Christopher Boyes': 0.08774463790439324, 'Luca Marco Paracels': 0.05551667920355111, 'John Lasseter': 0.045048533805877464, 'Andrew Lesnie': 0.03563747664881076, "Chris O'Connell": 0.032844942597281886, 'Hans Zimmer': 0.025980565508243635, 'John Williams': 0.024944977905317817, 'Thomas Tull': 0.023238367336256567, 'Tim Gomillion': 0.01752372842135341, 'Larry White': 0.015382628313984733, 'Howard Shore': 0.014509787535613317, 'Roger Barton': 0.014348987541060065, 'Howard R. Campbell': 0.01416521492652425, 'James Wan': 0.014161154728772631, 'Dariusz Wolski': 0.014158954179841421, 'Lucy Bevan': 0.014050502651593893, 'John Powell': 0.013959523605107983, 'George Lucas': 0.013520458762351823, 'Stephen P. Robinson': 0.012286623168076645, 'David Hoberman': 0.01213813045814291, 'Alex Rouse': 0.01210079709890375, 'Michael G. Wilson': 0.011835342984466521, 'James Newton Howard': 0.009187076714167786, 'Brian N. Bentley': 0.009044353862339607, 'Tom Johnson': 0.0089066193516674, 'Steven Spielberg': 0.008588950763389221, 'Bub Asman': 0.008174907701321711, 'Chris Lebenzon': 0.007575818131497358, 'John Roesch': 0.0072357713099262585, 'Denise Chamian': 0.006550597340897022, 'Jeffrey Wilhoit': 0.006265571162706454, 'Brian Tyler': 0.00620273183735895, 'Lorenzo di Bonaventura': 0.006099027395762116, 'Per Hallberg': 0.006042252044158935, 'Janet Hirshenson': 0.005664155737643702, 'William B. Kaplan': 0.00554336626210369, 'Michael Giacchino': 0.005467053214387659, 'William Hoy': 0.004921508503728029, 'Daryn Okada': 0.004571363256154636, 'Amir Mokri': 0.004406644399327508, 'Gary Summers': 0.004035298507621837, 'Kimberly French': 0.003965881090883847, 'James Horner': 0.003914179626523095, 'Kym Barrett': 0.003756421947900734, 'Randy Thom': 0.003688569110959064, 'Stephen McEveety': 0.003672237413360979, 'Mark Helfrich': 0.0034661239483014155, 'Albert R. Broccoli': 0.0034441166916746543, 'Priscilla John': 0.0033333197192843985, 'Craig Wood': 0.003310037685885282, 'Roger Mussenden': 0.0032254507241282135, 'Dottie Starling': 0.0032092784206537743, 'Dean Semler': 0.0031609079976966776, 'John T. Cucci': 0.003157993792813539, 'Gary A. Hecker': 0.0031493565887746725, 'Steve Boeddeker': 0.0030867351376453996, 'Mo Henry': 0.0030603382323889615, 'Frederick H. Stahly': 0.0030523889836853737, 'Brian Avery': 0.0030444456016217414, 'Ve Neill': 0.0030107360205235815, 'Gwendolyn Yates Whittle': 0.0029053625659861424, 'Lisa Lovaas': 0.002848660255337431, 'Robert Mark Kamen': 0.0028199231696380005, 'Sylvester Stallone': 0.0026428245590340927, 'Bruce Berman': 0.0026175339514379536, 'David Minkowski': 0.002397393101314237, 'Paul W.S. Anderson': 0.0023890290206688464, 'Doug Hemphill': 0.0023709963164152393, 'Michael Queen': 0.0023499114581268573, 'Roger Birnbaum': 0.002335003869012939, 'Avy Kaufman': 0.00232513620938951, 'Paul Massey': 0.0023107938803112885, "Louis D'Esposito": 0.002307210059754082, 'Peter Burgis': 0.0022963904238099226, 'Mark A. Mangini': 0.0022816443445584445, 'William Ladd Skinner': 0.0022547806005676118, 'John Davis': 0.002184823057003378, 'Mary Vernieu': 0.0021626918235092326, 'Mike Fenton': 0.002050864916194963, 'Phil Tippett': 0.0020239481295482825, 'Sheila Jaffe': 0.0020077476696799147, 'Maurice Jarre': 0.0019699386002255564, 'Graeme Revell': 0.0018961976172192513, 'Guillermo del Toro': 0.0018715975219588971, 'Anna Pinnock': 0.0018695095525912819, 'Steve Jablonsky': 0.0018653950338132414, 'Arthur Schmidt': 0.0018289756489026118, 'Ronna Kress': 0.001828049964978954, 'Bobby Cohen': 0.0017885810631876789, 'Jasin Boland': 0.0017500598477027337, 'Robert Troy': 0.001726848560127035, 'Luc Besson': 0.0017213755434149637, 'Peter Howitt': 0.001718420029090438, 'Neal H. Moritz': 0.0016772053641356146, 'Nerses Gezalyan': 0.0016593193178280035, 'Bob Badami': 0.001653095632032478, 'Judianna Makovsky': 0.0016244008709403238, 'Bill Corso': 0.0016216832206476111, 'Terri Taylor': 0.0016212352305868631, 'Graham King': 0.00161434104846938, 'Trevor Jones': 0.00160260509965293, 'Lauren Shuler Donner': 0.0015853869539653523, 'Charles Martin Inouye': 0.0015584708737758379, 'John Finklea': 0.0015501164080461004, 'Francine Maisler': 0.001541303395210585, 'Judd Apatow': 0.0014921667648041647, 'Wyck Godfrey': 0.0014882358655675812, 'Roy Lee': 0.0014539396203958595, 'Amanda Mackey': 0.0014510120122355336, 'Mel Metcalfe': 0.0014342440261515435, 'Javier Aguirresarobe': 0.0014281055295795471, 'Juliet Taylor': 0.001422969839351929, 'Frank Marshall': 0.0014082340971403003, 'Allen Hall': 0.0014079851212034909, 'Mary Selway': 0.0013867392963338287, 'Barry Bernardi': 0.0013738171695732103, 'Ivan Reitman': 0.0013736993449613543, 'Kevin Kaska': 0.0013719572654116176, 'John Debney': 0.0013637579062469186, 'Simon Kinberg': 0.0013564439729330125, 'Andy Nelson': 0.0013446794672101882, 'Aaron Zigman': 0.0013443156099650177, 'Avi Arad': 0.0013197810683420535, 'John Papsidera': 0.0013171976123454458, 'Leslie Shatz': 0.0013110887882030386, 'Richard L. Anderson': 0.0013091393166117077, 'David Appleby': 0.0013040832147615726, 'Sean Button': 0.001303717000609924, 'Deborah Lynn Scott': 0.001295308471448662, 'Gary Burritt': 0.0012496139762440985, 'Chris Columbus': 0.0012366634213425624, 'Edouard F. Henriques': 0.001225444625526369, 'Joseph Middleton': 0.0012199431404334737, 'Sarah Finn': 0.0012190699559087935, 'Nina Gold': 0.0012138757930918382, 'Harry Gregson-Williams': 0.0012088330688873787, 'Lori McCoy-Bell': 0.0012053679237066706, 'Will Files': 0.0011962358264554726, 'Mark Edward Wright': 0.001175437967791696, 'Eric Fellner': 0.0011743005607657746, 'Scott J. Ateah': 0.001149175013362131, 'Jerry Goldsmith': 0.0011415595072892847, 'Dan Webster': 0.0011325490547897646, 'Ray McIntyre Jr.': 0.001121868351721941, 'Marshall Winn': 0.001118424716772096, 'Jason Blum': 0.001114556449434203, 'James Baker': 0.0011044563603902074, 'Kerry Barden': 0.001093517741353212, 'Odin Benitez': 0.0010929850683830348, 'Arnon Milchan': 0.001085001046756688, 'Brian Grazer': 0.0010812845393158642, 'Thomas Newman': 0.0010585916337939854, 'Adam Greenberg': 0.0010514463209366218, 'Donald Mowat': 0.001031610808011891, 'Howard Feuer': 0.0010274123963552775, 'William Goldenberg': 0.0009952899879684513, 'Terry Porter': 0.0009891908208377776, 'Dean Cundey': 0.0009839532381492018, 'Elliot Goldenthal': 0.0009779862086371468, 'Dana Goldberg': 0.0009779543403350614, 'Lalo Schifrin': 0.0009656640803964136, 'Paul Hirsch': 0.0009607224250360836, 'Dianne Dreyer': 0.0009587396093427061, 'Mindy Marin': 0.0009564310280169101, 'Robert Gould': 0.0009507067925744831, 'Nancy Foy': 0.0009411023879609352, 'Glen MacPherson': 0.0009299804278147418, 'Deborah Aquila': 0.0009112626909509319, 'Jonathan Glickman': 0.0009075938857361198, 'Michael Ballhaus': 0.0009050664689074835, 'Stan Lee': 0.000904268043159757, 'Kevin Feige': 0.0009014016983991794, 'Robert Richardson': 0.0008919259618010633, 'Mary Zophres': 0.000881865781620793, 'Scott Rudin': 0.0008706862376504778, 'Anne McCarthy': 0.0008593404663214735, 'Lora Hirschberg': 0.0008446779561672333, 'Camille Friend': 0.0008304677792861359, 'Christopher Greenbury': 0.0008282397861969345, 'Steve Baine': 0.0008173203080022314, 'Joel Silver': 0.0008142695147107103, 'Sean Daniel': 0.0008019946833830186, 'Dede Gardner': 0.000797399441600375, 'Guy Williams': 0.0007942620069087789, 'Tom Hanks': 0.0007855319927469776, 'John Morris': 0.0007624541002497171, 'Richard King': 0.0007567310334549177, 'Henry Jackman': 0.0007371663362186418, 'Tim Bevan': 0.0007362440949265256, 'Phil Meheux': 0.000724412565967403, 'David B. Nowell': 0.0007167999475905905, 'John Barry': 0.0006929994273218302, 'Karen Golden': 0.0006902631540890408, 'Bill Butler': 0.0006886819747579549, 'Randy Edelman': 0.0006850446892346034, 'David Siegel': 0.0006793403660341325, 'Tricia Wood': 0.0006790560358670378, 'Mark Mothersbaugh': 0.0006788394456211591, 'Tak Fujimoto': 0.0006771426518891416, 'Eric Alan Edwards': 0.0006738496549066533, 'Emmanuel Lubezki': 0.0006643387275888201, 'Robert W. Cort': 0.0006614373135318324, 'Richard Vane': 0.000644269968517119, 'Bruce Green': 0.0006416244168473185, 'John Seale': 0.000635197926242988, 'Heike Brandstatter': 0.0006345590882307715, 'Colin Anderson': 0.0006319526072306814, 'Gloria Gresham': 0.0006176496375336028, 'John Marzano': 0.0006108432112631408, 'Matthew Libatique': 0.0006086874050297124, 'Marco Beltrami': 0.0006043402365678004, 'Stefan Sonnenfeld': 0.0006043398298501677, 'Marcia Ross': 0.0005985922015342224, 'Seamus McGarvey': 0.0005921125073928594, 'Steven Reuther': 0.0005876694195409758, 'Theodore Shapiro': 0.0005865423809324479, 'Mary Claire Hannan': 0.0005783451014536721, 'Mychael Danna': 0.0005765056921144228, 'Dave Kupczyk': 0.0005751382171141521, 'Joanna Colbert': 0.0005749377726299699, 'Greg P. Russell': 0.0005702645635803805, 'Randall Poster': 0.0005518040532650489, 'Richard Pearson': 0.0005502204312922996, 'Donald E. Thorin': 0.0005469719494253373, 'Clint Mansell': 0.0005433253554732256, 'William Steinkamp': 0.0005426985946188334, 'Sanja Milkovic Hays': 0.0005318203993404843, 'Marc Shaiman': 0.0005305234579549222, 'Kathleen Kennedy': 0.0005299832658396975, 'Douglas Aibel': 0.0005298534978795653, 'Trevor Rabin': 0.0005280713877732176, 'Scott Stuber': 0.0005278267161912935, 'O. Nicholas Brown': 0.0005184151964743907, 'John McAlary\t': 0.0005154168620341542, 'Julio Macat': 0.0005151667872029022, 'Jann Engel': 0.0005145399846667036, 'David Rubin': 0.0005126737217204346, 'Chris David': 0.0005097378147051362, 'Dennis Liddiard': 0.0005032876809356841, 'Jim Henrikson': 0.0005009241361358062, 'Rich Mento': 0.0004982099642139145, 'Michael Kahn': 0.0004955696681453227, 'Michael Magill': 0.0004909837628916706, 'Don Carmody': 0.0004900963980925046, 'Dennis Gassner': 0.0004898650431248494, 'John Houlihan': 0.0004810427191341383, 'Gary Barber': 0.00047915271432433704, 'Suzanne Smith': 0.0004728674992651873, 'Peter Jackson': 0.0004699035784667519, 'Mark Johnson': 0.00046828203091379874, 'Karen Baker Landers': 0.00046758798109261665, 'Matthew F. Leonetti': 0.0004675552522918477, 'Carol Kiefer': 0.00045983956970703667, 'Maher Ahmad': 0.0004594770959350734, 'James Muro': 0.00045775052906024395, 'Ben Waisbren': 0.00045748639803080407, 'Louise Frogley': 0.00045709185483415937, 'Jon Title': 0.0004570574442819009, 'George Fenton': 0.0004503301217707832, 'Robert Elswit': 0.000445729073235063, 'Jina Jay': 0.0004436051624031836, 'Debra Zane': 0.0004435385314097838, 'Lon Bender': 0.00044287869085513326, 'Rolfe Kent': 0.00044264353582034913, 'J.J. Makaro': 0.0004422119329808227, 'Brad Pitt': 0.0004390732507357609, 'Glenn Freemantle': 0.00043626586423294543, 'Michael Bay': 0.00043152354067157994, 'Peter Teschner': 0.00042661279738626085, 'Frank Masi': 0.00042178387580033146, 'Alexandre Desplat': 0.00041400236971871854, 'Gale Anne Hurd': 0.0004114285040938763, 'Waldo Sanchez': 0.0004105782339807442, 'Victoria Thomas': 0.0004096445885188012, 'Steve Maslow': 0.00040946471457058484, 'Mary Hidalgo': 0.0004075563267565047, 'Billy Weber': 0.0004035704301042434, 'Pietro Scalia': 0.00039788867902620684, 'Lee Dichter': 0.0003975092016037641, 'Jeff Carson': 0.0003952737976931878, 'Steve Golin': 0.0003952517264419199, 'Lindsay Graham': 0.0003927372199188381, 'Anne V. Coates': 0.00039226325809035704, 'Michael Kamen': 0.0003902414268857138, 'Sheldon Kahn': 0.00038846705202786306, 'William R. Dean': 0.0003868088362283445, 'Erica Edell Phillips': 0.0003861066983792741, 'Jeffrey Kurland': 0.0003803853501947478, 'James Ashwill': 0.0003784832690612831, 'Mark Gordon': 0.0003758011408536868, 'Stuart Baird': 0.0003722390509651805, 'Newton Thomas Sigel': 0.0003693882187996131, 'Chris Bender': 0.0003672189883493299, 'Jeff Atmajian': 0.00036454237120650415, 'David Newman': 0.0003621396093795329, 'Steven J. Winslow': 0.0003611222908285292, 'Barbara Harris': 0.0003585816856553904, 'David Arnold': 0.0003441824806704733, 'Fred Roos': 0.00034353638104153196, 'Caleb Deschanel': 0.0003428840163274399, 'Mitchell Amundsen': 0.0003420549874362129, 'Gabriel Beristain': 0.00034159121822083217, 'Pete Anthony': 0.0003407403246559383, 'Nathan Crowley': 0.0003379483690115285, 'Gregg Rudloff': 0.00033673312479929067, 'Marlene Stewart': 0.0003345551032124314, 'Richard Brener': 0.00033443287068374316, 'Ellen Chenoweth': 0.00033017147658303815, 'Mark Goldblatt': 0.00032966682843789256, 'Julie Weiss': 0.0003292834582297506, 'Ellen Lewis': 0.00032926625956682814, 'Tom Rosenberg': 0.0003277111668922742, 'Charlene Amateau': 0.0003264145354294753, 'Francis Ford Coppola': 0.00032084508422749547, 'James Bamford': 0.0003176249723093616, 'Coreen Mayrs': 0.0003173101146391427, 'Laurie MacDonald': 0.0003094082884968803, 'Lucinda Syson': 0.0003091021675827803, 'Derek R. Hill': 0.00030674963012720577, 'Michael Shamberg': 0.0003055281322779125, 'Mark Canton': 0.00030531277306298577, 'Vanessa Baker': 0.0003035941675298058, 'Robert Leighton': 0.0003024120572531455, 'Sydney Pollack': 0.00030114011867163944, 'Bob Weinstein': 0.00030012839297820687, 'Mike Medavoy': 0.0003000454271992406, 'Clint Eastwood': 0.00030000643408690803, 'Alex Thomson': 0.0002946659543497586, 'Danny Elfman': 0.00029417974900439517, 'Jeffrey L. Kimball': 0.00029044654970095997, 'Sam Raimi': 0.0002874752765661293, 'Robert De Niro': 0.0002856318718192092, 'Ed Shearmur': 0.0002849372981530324, 'Garrett Lewis': 0.00028423438517607334, 'Roger Deakins': 0.00028410626287565924, 'Nick Moore': 0.00027933750643402026, 'Lawrence Gordon': 0.00027879809614213285, 'Stephen Mirrione': 0.000277306396612163, 'Michael Semanick': 0.00027262806957032885, 'Jeffrey J. Haboush': 0.00027141665525515804, 'Jim Passon': 0.00027041764680031004, 'Mario Kassar': 0.0002693627726784411, 'Jennifer L. Smith': 0.0002665249400963818, 'David Brown': 0.00026639792210471456, 'Richard Foreman Jr.': 0.00026616641121260333, 'Jerry Bruckheimer': 0.00025936673555190966, 'Perry Andelin Blake': 0.0002574668175390458, 'Jon Hutman': 0.00025738660527986466, 'Peter McCaffrey': 0.00025588596522725497, 'Nancy Nayor': 0.00025441422131562236, 'Christopher Young': 0.00025090975573937305, 'Lawrence Sher': 0.0002502467306809017, 'Bonnie Timmermann': 0.00025015210774676557, 'Mark Friedberg': 0.0002500779475530828, 'Christopher Assells': 0.00024986736884076465, 'Randi Hiller': 0.00024976373849527336, 'Peter Deming': 0.00024921164838018683, 'Thomas L. Fisher': 0.00024665876624823096, 'Helen Jarvis': 0.00024563746636302285, 'Phedon Papamichael': 0.0002448485460612046, 'Hans Bjerno': 0.00024353345582616348, 'Adam Jenkins': 0.000243052588129613, 'Jane Feinberg': 0.00024293391527955245, 'Dennis McNeill': 0.00024212113147165895, 'Dan Hegeman': 0.00024010271698808326, 'Michael Taylor': 0.00023994550523499564, 'Mali Finn': 0.0002390153494502743, 'Jeffrey Harlacker': 0.00023841381402781373, 'Roque Ba√±os': 0.00023790317193549655, 'Philippe Rousselot': 0.0002348334903504085, 'Debbie McWilliams': 0.00023446184355389497, 'Andrew Max Cahn': 0.00023420837239456087, 'Ryan Kavanaugh': 0.0002327227290546706, 'Dan Wallin': 0.00023168238155456137, "Dan O'Connell": 0.00022939506979933788, 'Alan Silvestri': 0.0002281064788425602, 'Hector C. Gika': 0.0002276530742361622, 'Maggie Martin': 0.00022732149698325208, 'Billy Hopkins': 0.00022614784176853888, 'Dean A. Zupancic': 0.00022581974970590336, 'Alicia Stevenson': 0.00022259791885389626, 'Craig Armstrong': 0.0002225420703968833, 'Lynn Stalmaster': 0.0002206395648147789, 'Anthony Katagas': 0.00021997876325810555, 'Rachel Portman': 0.00021495517731459055, 'John Caglione Jr.': 0.00021423982832853707, 'Andy Malcolm': 0.00021263502171437162, 'Zak Penn': 0.00020953448789139923, 'Robin D. Cook': 0.0002092807285793209, 'Rick Avery': 0.00020211153025526175, 'R.J. Kizer': 0.00019976607482134374, 'Stephen Goldblatt': 0.00019738322509366257, 'Mike Hill': 0.00019722672671820306, 'Steve Mann': 0.00019039463502918986, 'Gary Rizzo': 0.00018924727144518266, 'Cosmas Paul Bolger Jr.': 0.00018840204788652068, 'Colleen Atwood': 0.0001876011819114332, 'Penny Perry': 0.00018732354263213718, 'Harold Ramis': 0.00018557027178313089, 'Kerry Lyn McKissick': 0.00018403322842816327, 'Andrew G. Vajna': 0.00018312907926948255, 'Lisa Beach': 0.00018229873195635385, 'David Tattersall': 0.00018179189801112238, 'Perry Robertson': 0.00017685504949172855, 'George Simpson': 0.00017483507768289687, 'Walter Murch': 0.0001735262610556619, 'Doug Coleman': 0.00016971674915711434, 'Sarah Katzman': 0.00016913848631468974, 'David J. Bomba': 0.00016865959516829693, 'Michael Minkler': 0.00016769384211094755, 'Mark Bridges': 0.00016736732766778404, 'Wally Westmore': 0.0001655814263421748, 'Daniel P. Hanley': 0.00016509252942057876, 'David E. Campbell': 0.00016292900279571567, 'Oliver Wood': 0.00016291773161282555, 'Adam Sandler': 0.00015896272049641002, 'Bryan Bowen': 0.00015616194676372244, 'Stacey Sher': 0.0001554736017748421, 'Brian De Palma': 0.00015269287646654182, 'Juel Bestrop': 0.000151862712165065, 'Gary Jones': 0.00015139709944545838, 'Basil Poledouris': 0.00014976388104084068, 'Alex Gibson': 0.00014969203382379827, 'Jon Peters': 0.00014967412944945303, 'Jeannine Oppewall': 0.000148535293402277, 'Edward R. Pressman': 0.00014680562058893768, 'Craig McKay': 0.00014654958947365955, 'William Goldman': 0.00014524954604418441, 'Menahem Golan': 0.0001444700344753683, 'Joe Medjuck': 0.00014346786759164906, 'Gerald B. Greenberg': 0.0001419792012181286, 'Tony Scott': 0.00014197212777897566, 'Stuart Dryburgh': 0.00014000755691360276, 'Leesa Evans': 0.00013853917588104985, 'Scott Martin Gershin': 0.00013851760348997233, 'Denise Di Novi': 0.00013830851017288963, 'Julie Feiner': 0.00013798769273328746, 'Mark Irwin': 0.00013778973427486688, 'John Hughes': 0.00013761664929982158, 'Michael Peyser': 0.0001359602927861816, 'Clayton Townsend': 0.00013585646152062487, 'Cathy Sandrich': 0.0001336982193203792, 'Yoram Globus': 0.0001336081872482865, 'David Orr': 0.00013097551031151833, 'Christophe Beck': 0.00013097067577535796, 'Daniel R. Chavez': 0.0001306367463861216, 'Toby Emmerich': 0.00013025797566033214, 'Michael Uslan': 0.00012937978576450874, 'Stephen Hunter Flick': 0.00012931043077736156, 'Robert Folk': 0.00012897903692976907, 'Jenno Topping': 0.00012846677123919044, 'Jack Giarraputo': 0.00012734495954730424, 'Don Burgess': 0.00012721921852237405, 'Michael Tronick': 0.00012698378459890955, 'Jim Brookshire': 0.000126667164429133, 'Victoria Burrows': 0.0001263508254402034, 'Sandy Powell': 0.00012573215737054065, 'Mace Neufeld': 0.00012558755736703615, 'Art Linson': 0.0001255820944554554, 'Barbara Broccoli': 0.00012492310875917109, 'John Frizzell': 0.00012482808134989132, 'Richard Roberts': 0.00012423712815427918, 'Chris Jenkins': 0.0001233601213648538, 'Dante Spinotti': 0.00012223257656559524, 'Frank Mancuso Jr.': 0.0001221264211121972, 'Gregory Lundsgaard': 0.00012196610931621819, 'Christian P. Minkler': 0.00012170217167555826, 'Mark Isham': 0.00012062786957539473, 'Zoltan Elek': 0.00011899028422677671, 'Jane Rosenthal': 0.00011763882565309045, 'Juan Peralta': 0.00011591374077036096, 'Richard Price': 0.0001154501688019211, 'Jack Fisk': 0.00011543104109443299, 'Bill Conti': 0.00011537334204639366, 'John Ottman': 0.00011438099514741334, 'David V. Butler': 0.00011421107986151719, 'Carter Burwell': 0.0001121164010273922, 'Rick Kline': 0.0001111933874305739, 'Philip Harrison': 0.00011079987176292264, 'Michael Silvers': 0.00011070844691125794, 'Jeff Gourson': 0.0001086946751850061, 'Kevin Stitt': 0.00010856791859073189, 'Gregg Barbanell': 0.00010820797928437396, 'Harvey Weinstein': 0.00010776227324424117, 'David Nichols': 0.0001070505945088902, 'David Rosenbloom': 0.0001048476992747414, 'Zachary Woodlee': 0.00010450220911313311, 'John Bailey': 0.00010447277565502418, 'Marc Fishman': 0.00010416576800622913, 'Dino De Laurentiis': 0.00010092205749852948, 'Steven Rosenblum': 9.964069526348853e-05, 'Robert Zemeckis': 9.892758449258272e-05, 'Peter Hyams': 9.866128435299542e-05, 'Wylie Stateman': 9.80727476093434e-05, 'Robert Jackson': 9.702023868202276e-05, 'Ron Bartlett': 9.655380090955527e-05, 'Bruce Tanis': 9.46505574286936e-05, 'Pete Romano': 9.411928557825343e-05, 'George S. Clinton': 9.405638547758052e-05, 'Duane Manwiller': 9.272826228810168e-05, 'Michael Mann': 9.197549111610929e-05, 'Becky Sullivan': 9.113989646131662e-05, 'Stephen King': 9.106853045902412e-05, 'Marko A. Costanzo': 9.098683657517698e-05, 'Ron Howard': 9.067969478996087e-05, 'Judy Becker': 8.991957771406381e-05, 'Robert Rodriguez': 8.972120928720132e-05, 'Tom Stern': 8.651514292344658e-05, 'Ellen Mirojnick': 8.594357354074287e-05, 'Ashok Amritraj': 8.542492801891815e-05, 'Walter M. Scott': 8.436171118578438e-05, 'George Drakoulias': 8.205186538137563e-05, 'Laurence Mark': 8.117117353152313e-05, 'Jan de Bont': 8.101978976251457e-05, 'Woody Allen': 8.064905956967353e-05, 'Michael Hertlein': 8.059773695931005e-05, 'Joel Schumacher': 8.01787861893477e-05, 'John Murphy': 7.924480469082655e-05, 'Peter Lamont': 7.901784505006896e-05, 'Susie Figgis': 7.898099364865127e-05, 'J. Dennis Washington': 7.858019470340957e-05, 'Patrick Doyle': 7.729706667434374e-05, 'Merrick Morton': 7.324668881733979e-05, 'Steve Pederson': 7.251474820771126e-05, 'Anna Rane': 7.17759055556122e-05, 'Cedric Gibbons': 6.997362167871876e-05, 'Walter Hill': 6.987360695404518e-05, 'Sarah Green': 6.934655032558366e-05, 'E. Bennett Walsh': 6.873246150640941e-05, 'Liza Chasin': 6.865070486607652e-05, 'Don Zimmerman': 6.864065774662832e-05, 'Craig Berkey': 6.859889399922953e-05, 'Paula Weinstein': 6.859574679428535e-05, 'Kent Beyda': 6.805990234094329e-05, 'Rosemary Brandenburg': 6.750375344989679e-05, 'Philip Glass': 6.719411846731786e-05, 'John Casey': 6.634334743947691e-05, 'Julie Hewett': 6.584743361712926e-05, 'Martin Scorsese': 6.411122389774102e-05, 'Vilmos Zsigmond': 6.325789704234941e-05, 'Paul Schnee': 6.302909570365419e-05, 'Akiva Goldsman': 6.298897500638944e-05, 'Joel Kramer': 6.264594501473825e-05, 'Dylan Goss': 6.201092996006006e-05, 'Shay Cunliffe': 6.150557485953313e-05, 'Dave Jordan': 6.136033521590014e-05, 'John T. Reitz': 6.11875951231183e-05, 'Wendy M. Craig': 6.11378387893045e-05, 'Michael De Luca': 6.107410653903286e-05, 'Peter Staubli': 6.104186231188885e-05, 'Sharen Davis': 6.042466126233452e-05, 'Steve Mirkovich': 6.036376464375124e-05, 'Elmer Bernstein': 5.936978375596816e-05, 'Christopher Tellefsen': 5.8430063516923195e-05, 'Brett Ratner': 5.800595929661325e-05, 'Mitch Dubin': 5.786491761580345e-05, 'Jeanne McCarthy': 5.689149989963058e-05, 'Andrzej Bartkowiak': 5.678990842064607e-05, 'Andrew Dunn': 5.676128359380052e-05, 'Marion Dougherty': 5.667526036708572e-05, 'John Sloss': 5.661557192364931e-05, 'Aggie Guerard Rodgers': 5.636925477161746e-05, 'Mike Prestwood Smith': 5.613219807853201e-05, 'Gary Lucchesi': 5.557567015052402e-05, 'Marcelo Zarvos': 5.514625917928656e-05, 'Geoff Hubbard': 5.46249776542113e-05, 'Terry Stacey': 5.4379385604154526e-05, 'Jeremy Braben': 5.3325455794863775e-05, 'David Esparza': 5.202310968146431e-05, 'James Whitaker': 5.10583207724623e-05, 'Ruth E. Carter': 4.947513676811006e-05, 'Tim Burton': 4.886261619066095e-05, 'Randall Emmett': 4.882159962519379e-05, 'Jack Stew': 4.788491391894297e-05, 'Ann Roth': 4.7823356942061966e-05, 'Erwin Stoff': 4.684596413445574e-05, 'Dario Marianelli': 4.670651984328109e-05, 'David C. Robinson': 4.5029081869989824e-05, 'William S. Beasley': 4.466560283654175e-05, 'Conrad Buff IV': 4.417037525292467e-05, 'Johanna Ray': 4.416897109197476e-05, "Alan D'Angerio": 4.370155473167282e-05, 'Gail Stevens': 4.303894677091979e-05, 'Stephen E. Rivkin': 4.213359814665841e-05, 'Catherine Harper': 4.208683651410901e-05, 'Scott Sanders': 4.2080851508382925e-05, 'Shawn Murphy': 4.101991895637374e-05, 'Thierry Arbogast': 4.094304726779489e-05, 'Rob Farris': 4.0367243578596286e-05, 'Lise Richardson': 4.0067054548949594e-05, 'Edward Lachman': 3.916034500341932e-05, 'Gary Rydstrom': 3.9104048787333437e-05, 'Paul Aulicino': 3.8953929920129254e-05, 'Chris Hogan': 3.879979132340338e-05, 'Patrick Malone': 3.841633834542113e-05, 'James Moriana': 3.837111691591168e-05, 'Dug Rotstein': 3.798753869433448e-05, 'Nick Wechsler': 3.795922043126759e-05, 'Nelson Coates': 3.768779104541812e-05, 'James G. Robinson': 3.7318751846262695e-05, 'George Furla': 3.6182779127528404e-05, 'Steve Nelson': 3.5836499051272495e-05, 'Klemens Becker': 3.496667076780815e-05, 'John Hubbard': 3.487900532276073e-05, 'George Clooney': 3.485862865677473e-05, 'Ennio Morricone': 3.4696687699521055e-05, 'Steven Soderbergh': 3.41532474364656e-05, 'Andrew DeCristofaro': 3.414842690481991e-05, 'Ray Fisher': 3.296834654224002e-05, 'Tessa Ross': 3.241929484681572e-05, "Kevin O'Connell": 3.2396354177992516e-05, 'Susan Downey': 3.167953155585465e-05, 'Lora Kennedy': 3.161429796679949e-05, 'Jacques Jouffret': 3.1609095753198334e-05, 'Margery Simkin': 3.104814970699032e-05, 'Rita Ryack': 3.0310477523803275e-05, 'Jay Wilkinson': 3.0272412843499462e-05, 'Mary Cybulski': 2.926254497594639e-05, 'Leo Shuken': 2.90100928797343e-05, 'Gregg Landaker': 2.8968582388281048e-05, 'Gregory Nicotero': 2.8022269428270496e-05, 'Hugh Waddell': 2.7501423353003305e-05, 'Freddie Hice': 2.7441220191077947e-05, 'Pam Dixon': 2.713365780016208e-05, 'Tim Suhrstedt': 2.622722098836304e-05, 'Don Davis': 2.6067358766962986e-05, 'Rob Cohen': 2.5311612266264905e-05, 'Teresa Eckton': 2.506781685773912e-05, 'David S. Goyer': 2.5028603761494594e-05, 'Howard Berger': 2.4942686337220577e-05, 'Kathy Driscoll': 2.4598292226324035e-05, 'Danny Dimbort': 2.3898599587723083e-05, 'Cindy Carr': 2.2840824679596796e-05, 'Phyllis Huffman': 2.2704678780867937e-05, 'Avi Lerner': 2.255999135730887e-05, 'Merie Weismiller Wallace': 2.2184093794330206e-05, 'Charles Roven': 2.1144795662962318e-05, 'Piero Mura': 2.057015768984974e-05, 'Bruce Broughton': 2.0516309108804946e-05, 'Blake Edwards': 2.0492162206234785e-05, 'Kate Dowd': 2.0283300597231164e-05, 'Dino Dimuro': 2.0149680725534687e-05, 'Joel Cox': 1.9925571465519406e-05, 'Janice Hampton': 1.985414236976685e-05, 'Kenneth Karman': 1.9840719870590133e-05, 'Cate Hardman': 1.9237027655641394e-05, 'Laura Rosenthal': 1.9207715673343834e-05, 'Robert D. Yeoman': 1.9108683247229045e-05, 'Harry Cohen': 1.881226488039168e-05, 'Gene Serdena': 1.8724753066726842e-05, 'Missy Stewart': 1.82611799927475e-05, 'Henry Mancini': 1.7960118379984588e-05, 'Lyle R. Wheeler': 1.7537753418511947e-05, 'Jacqueline West': 1.7317602139593575e-05, 'Moshe Diamant': 1.7170826037341318e-05, 'Wes Craven': 1.5712775792569974e-05, 'J.C. Spink': 1.5686655939559754e-05, 'Karin Silvestri': 1.5544651038727745e-05, 'Ridley Scott': 1.4648959616439162e-05, 'Felicity Bowring': 1.3737067387728052e-05, 'Sheila Waldron': 1.3632143548198086e-05, 'Edith Head': 1.3532182573041534e-05, 'Charles Maynes': 1.2767473998313362e-05, 'Peter Afterman': 1.2566714672180025e-05, 'Billy Wilder': 1.2317593408992298e-05, 'Dale E. Grahn': 1.2103482098638326e-05, 'Tom Duffield': 1.196897623948881e-05, 'Patrick Lussier': 1.1659293289153605e-05, 'Bruce Davey': 1.1318126189770622e-05, 'Jo Edna Boldin': 1.1262511445532623e-05, 'Jon Taylor': 1.07063734380946e-05, 'Laura Harris Atkinson': 1.0468185300769848e-05, 'Greg Hedgepath': 1.0060839141116143e-05, 'Mark Damon': 9.868291865116557e-06, 'Jon Jashni': 9.757278593731111e-06, 'A. R. Rahman': 9.709053925507177e-06, 'Ian Fleming': 9.0433025487902e-06, 'Sidney Kimmel': 8.846089409308617e-06, 'Peter Tothpal': 8.740870597493962e-06, 'Kimaree Long': 8.682048542915594e-06, 'Jon Gregory': 8.474472002025698e-06, 'Robert Greenhut': 7.817046358163349e-06, 'Albert Gasser': 7.564996045302638e-06, 'Daniel Hubbard': 7.425554315050193e-06, 'Liz Mullane': 7.40346291085585e-06, 'Jon Kilik': 7.193465144370577e-06, 'John C. Stuver': 7.120424451040737e-06, 'Walter F. Parkes': 6.838207430317319e-06, 'Bob Kensinger': 6.78566096403114e-06, 'Jane Jenkins': 5.730046730453866e-06, 'Michael Keller': 5.713152183721661e-06, 'Ida Random': 5.656499319622818e-06, 'Susan Bierbaum': 5.599993791974918e-06, 'Frank A. Monta√±o': 4.907388072272936e-06, 'Lawrence Bender': 2.4907633653590885e-06, 'Drew Jiritano': 2.17739901701149e-06, 'Michael I. Rachmil': 1.267840337825194e-06, 'Ronnie Screwvala': 8.786634805727044e-07, 'Richard Dwan Jr.': 6.467758229417303e-07, 'Lisa Zeno Churgin': 3.5746137678431555e-07}
# keys_dict = get_scores_dict('Keywords')
keys_dict = {'superhero': 0.08179226494398037, '3d': 0.0617379050572196, 'anthropomorphism': 0.0401213268017019, 'elves': 0.038787729865452605, 'duringcreditsstinger': 0.03795890088297619, 'imax': 0.03466317069496598, 'aftercreditsstinger': 0.018226820036467417, 'british secret service': 0.01788945461735819, 'exotic island': 0.01748325576716272, 'saving the world': 0.01748323810821752, 'revenge': 0.01697266326062145, 'suspense': 0.015854321656979208, 'werewolf': 0.015753163380704552, 'dinosaur': 0.01568329621975273, 'moon': 0.013207420202701933, 'car': 0.011092758451012253, 'witch': 0.010511277001626637, 'space opera': 0.010254935375950295, 'island': 0.009243155477685165, 'fang vamp': 0.00915862174402654, 'amnesia': 0.008852581913642295, 'robot': 0.00867705137547892, 'sequel': 0.008193292179812055, 'space': 0.008160111931063984, 'spaceship': 0.008143774806405374, 'mission': 0.007546778296177328, 'new york': 0.007546554471315206, 'fantasy': 0.006986357069359561, 'animation': 0.006797427817528413, 'alien invasion': 0.006749593516724281, 'secret agent': 0.006440668614739889, 'redemption': 0.006051146923810669, 'traitor': 0.005920068849989374, 'uncle': 0.0057371955246755, 'friendship': 0.005559153878209347, 'army': 0.00541751554199686, 'super powers': 0.005039324203883302, 'showdown': 0.005036215286252717, 'riddle': 0.005012417322256888, 'jungle': 0.004830842874592516, 'dystopia': 0.004523241298622415, 'mutiny': 0.004492634336867887, 'corruption': 0.004454103400975199, 'independent film': 0.004440460064453891, 'astronaut': 0.004314987699588662, 'queen': 0.0043143448091222504, 'based on novel': 0.0042774642379056075, 'villain': 0.0037988557617685673, 'revolution': 0.0037836689187205606, 'apocalypse': 0.0035789614788845707, 'torture': 0.0035288743593346215, 'woman director': 0.0033516940213194494, 'mermaid': 0.003224744700937438, 'mutant': 0.003205790803372488, 'holiday': 0.003194140684825969, 'las vegas': 0.00316823858644836, 'toy comes to life': 0.003129748604886123, 'insanity': 0.0031025980575905892, 'brother brother relationship': 0.003034738915343993, 'explosive': 0.0030100113970724296, 'hotel': 0.002884399632587788, 'bullying': 0.002734912312516842, 'virus': 0.002719798890571203, 'treasure': 0.0025570905597968145, 'soldier': 0.0025466780961336836, 'on the run': 0.002502819167943869, 'love': 0.002475314008218311, 'father son relationship': 0.0024727574736733075, 'brother sister relationship': 0.0024471927412570166, 'teenage crush': 0.0024177042748452296, 'fbi agent': 0.0024147663845306296, 'wedding': 0.0023731271011150456, 'shipwreck': 0.0023530648358397804, 'dying and death': 0.0023496378914490508, 'secret service': 0.0023333331728629756, 'family relationships': 0.002311474040873681, 'marvel comic': 0.002281482101852064, 'male friendship': 0.0022674803611915035, 'violence': 0.002235177358948816, 'marriage crisis': 0.0022307587617192495, 'fire': 0.0022246057821427813, 'rescue': 0.002159104430003368, 'skeleton': 0.0021429479721035488, 'psychopath': 0.0021385190162557252, 'roman empire': 0.0020643767412985023, "love of one's life": 0.00199926050471112, 'oscar award': 0.0019952653316747085, 'religion and supernatural': 0.001983322626108578, 'crime fighter': 0.001972240442987613, 'battle': 0.0019334596600024962, 'talking animal': 0.001920156704128628, 'alien': 0.0018963652923988604, 'female nudity': 0.0018500818285027382, 'fairy tale': 0.0018445190415260506, 'magic': 0.0018335479017759439, 'government': 0.0017408023912452186, 'u.s. navy': 0.0017155461177673886, 'winter': 0.001697958476294612, 'italy': 0.0016904466835461495, 'fight': 0.0016790309826638775, 'sex': 0.0016661044709660987, 'investigation': 0.0016593084506000013, 'scientist': 0.0016567005905342082, 'santa claus': 0.0016531411117578082, 'treasure hunt': 0.0016402321863259524, 'native american': 0.0016171095450024917, 'bank': 0.0015990041758793567, 'mental illness': 0.0015981616810945465, 'secret identity': 0.0015927193921207571, 'washington d.c.': 0.0015643478262532147, 'falsely accused': 0.001547049069632506, 'firemen': 0.0015269508414919686, 'archaeologist': 0.0014805912038606994, 'spy': 0.00144201135922778, 'loss': 0.001437694303822013, 'zombie': 0.0014232761486381105, 'baby': 0.0014168221457126462, 'inventor': 0.0014134051563625752, 'los angeles': 0.001398460663114782, 'faith': 0.001387083399937567, 'chaos': 0.001384006735837148, 'supernatural powers': 0.0013821396272835598, 'knight': 0.0013777283284624234, 'exorcism': 0.0013711135262475973, 'future': 0.0013343748807834687, 'orphan': 0.001331576719490519, 'based on young adult novel': 0.001328771079669178, 'christmas': 0.0013122813745103704, 'castle': 0.0013058011231127016, 'based on comic': 0.0012872507915083194, 'mother son relationship': 0.0012861716762224255, 'romantic comedy': 0.0012816757925812137, 'death penalty': 0.0012754036824514582, 'shark': 0.0012545750837796715, 'biography': 0.001248906519181816, 'monster': 0.0012459040879450353, 'san francisco': 0.0012210643675234628, 'demonic possession': 0.0012131353185902023, 'wolf': 0.0012112828922696192, 'bible': 0.0012043824754036153, 'remake': 0.001191007990379135, 'conspiracy': 0.001175480297831316, 'based on true story': 0.0011674284488225593, 'parent child relationship': 0.0011364357796282095, 'small town': 0.0011045495754531919, 'california': 0.0011038566661188424, 'intelligence': 0.001100194838303542, 'android': 0.0010924184056316578, 'alcoholism': 0.0010905567533887838, 'evidence': 0.0010711433784615309, 'cia': 0.0010698278328541823, 'singing': 0.0010687471518766875, 'afterlife': 0.001055786416448257, 'lovers': 0.00103164686569864, 'london england': 0.0010258589602842407, 'fireworks': 0.0010245604873023219, 'hospital': 0.0010141260335046099, 'single mother': 0.0010131424581779755, 'murder': 0.0010044590996197455, 'sister sister relationship': 0.001000413008336446, 'boxer': 0.0010002800902168254, 'human animal relationship': 0.0009958378927863948, '1960s': 0.0009787849305050165, 'disappearance': 0.0009763090076844369, 'assassin': 0.0009586839669659602, 'psychiatrist': 0.0009540428922175787, 'motorcycle': 0.00092025491825472, 'student': 0.0009184382647875057, 'reporter': 0.0009133316309433776, 'mobster': 0.0009044533961155754, 'bride': 0.0008917712839087381, 'snow': 0.0008910515841793854, 'nerd': 0.0008888018980427717, 'egypt': 0.0008861217666394578, 'relationship': 0.000864800254509801, 'boy': 0.0008562382816157601, 'teenager': 0.000851329294653266, 'dual identity': 0.0008468773145789577, 'disaster film': 0.0008420567129876183, 'animal': 0.0008411926987840011, 'first time': 0.0008358178792367375, 'brothel': 0.0008346744733077179, 'undercover agent': 0.0008303822149369963, 'lawyer': 0.0008299824108837318, 'new york city': 0.0008247007564481043, 'road trip': 0.0008106214589573477, 'movie star': 0.0008103540485949213, 'new love': 0.0008092871160853453, 'explosion': 0.0007936356159470219, 'basketball': 0.0007936086414212958, 'wife husband relationship': 0.000789139611753245, 'cancer': 0.000777103816846017, 'based on tv series': 0.0007668766000449638, 'blood': 0.0007571136069706621, 'nature': 0.000738642300046052, 'money': 0.0007291026340653449, 'farm': 0.0007076098355159801, 'lake': 0.0007018940033716181, 'musical': 0.000691449993647865, 'culture clash': 0.0006876314191404879, 'stranded': 0.0006864713227061751, 'river': 0.0006744080148206502, 'thief': 0.0006719765857459862, 'found footage': 0.0006603762483838913, 'technology': 0.0006481862524524375, 'prison': 0.0006469326381283575, 'rape': 0.0006446743496914499, 'training': 0.0006277495709544389, 'sexual obsession': 0.0006234667519510612, 'dracula': 0.0006218072731930006, 'double life': 0.0006169913957852979, 'midlife crisis': 0.0006121289815918128, 'scotland': 0.0006115241760049966, 'book': 0.0006106265149239679, 'florida': 0.0005924686762551138, 'car chase': 0.0005882680367583011, 'paris': 0.0005827237398789905, 'china': 0.0005744490094834546, 'robbery': 0.0005665645175772949, 'india': 0.0005658825917157495, 'professor': 0.0005607258402164632, 'based on video game': 0.0005586511841478135, 'kiss': 0.0005538037822993912, 'gambling': 0.0005515052248346527, 'cat': 0.0005506770688059399, 'suicide': 0.0005491233043776323, 'children': 0.0005473053695289037, 'detective': 0.0005460688640866541, 'france': 0.0005367378977212233, 'hostage': 0.0005305548402552514, 'pregnancy and birth': 0.0005226258770478124, 'romance': 0.0005212180343055564, 'betrayal': 0.0005202624033524954, 'love triangle': 0.0005195247019094192, 'competition': 0.0005176763860340907, 'party': 0.0005174438312251227, 'sea': 0.0005157461456234094, 'time travel': 0.0005061654073739383, 'desert': 0.0005029641547812344, 'experiment': 0.000502639107507597, 'adoption': 0.000498768695144749, 'prince': 0.0004893589360760621, 'underworld': 0.0004884124975778053, 'friends': 0.00048810359254852114, 'cabin': 0.0004830471716178691, 'alcohol': 0.00047840184691610375, 'casino': 0.00047678953442508945, 'mountain': 0.0004747412550266461, 'war': 0.00047343229253951017, 'daughter': 0.0004731952450894884, 'fbi': 0.00047156019051013235, 'blackmail': 0.00047030221567452297, 'drug': 0.00046815201374554897, 'celebration': 0.00046472969356240953, 'england': 0.0004626827804400034, 'marriage proposal': 0.000462015929221231, 'weapon': 0.0004614184726006585, 'wheelchair': 0.00045877424137394515, 'hero': 0.00045142320329696043, 'spoof': 0.00045140665281096713, 'based on true events': 0.0004505041981436897, 'repayment': 0.00044603328719295115, 'photographer': 0.0004443906539545898, 'prisoner': 0.000444218762492201, 'amusement park': 0.0004428819694061313, 'dysfunctional family': 0.00044128586180468526, 'best friend': 0.00043896952208136305, 'airplane': 0.00043806761250723676, 'ship': 0.000429159188855806, 'family': 0.00042856855887062637, 'deception': 0.00042839245588193153, 'boat': 0.00042587020913622213, 'nazis': 0.00042309908618380057, 'extraterrestrial technology': 0.0004212337309063686, 'terrorism': 0.0004184093964725243, 'police': 0.0004160165406812421, 'youth': 0.0004157617750273745, 'dancing': 0.00041204798760174753, 'fame': 0.0004108249729459085, 'boxing match': 0.000410072607261118, 'child': 0.0004054280830671114, 'vatican': 0.0004053548930942921, 'chase': 0.00040253902321332876, 'car crash': 0.0003963956991569303, 'family vacation': 0.00039184156096468804, 'convict': 0.0003888303812728744, 'escape': 0.00038757305758440556, 'undercover': 0.00038676998872309555, 'restaurant': 0.0003851459892561857, 'sniper': 0.0003850140817352527, 'jealousy': 0.0003800200437910611, 'forbidden love': 0.00037917630863216583, 'loss of child': 0.00037342577828320485, 'doctor': 0.00037066068631482114, 'good vs evil': 0.00036916948293242216, 'helicopter': 0.0003674163717688083, 'comedy': 0.00036680468616050226, 'graduation': 0.000364721135318447, 'horse': 0.0003633577932438513, 'painting': 0.00036051940331291394, 'millionaire': 0.0003603501322063895, 'coma': 0.00035881208517029597, 'infidelity': 0.0003563596004765325, 'survival': 0.00035400889671909416, 'kidnapping': 0.0003507301900445983, 'storm': 0.00034841605051699504, 'drug dealer': 0.0003442006793471088, 'trainer': 0.00034226350373207275, 'terrorist': 0.00034203256092201785, 'venice': 0.000341251764296763, 'post-apocalyptic': 0.00032911227048549896, 'past': 0.0003257696092033496, 'taxi': 0.000325697321705424, 'falling in love': 0.00032400339476591094, 'narration': 0.0003236640754268772, 'dark comedy': 0.0003225218046468899, 'ocean': 0.0003194815084722574, 'extramarital affair': 0.00031915947438582333, 'individual': 0.00031908853795453376, 'age difference': 0.000318879355174173, 'xenophobia': 0.0003178509730462117, 'kgb': 0.00031625021761995273, 'transporter': 0.0003095322667003494, 'mayor': 0.0003091790275427194, 'hitman': 0.00030667231812538305, 'pregnancy': 0.0003036121690713863, 'racism': 0.0003011606780050482, 'fashion': 0.00029969876650568275, 'village': 0.0002987124055205624, 'business man': 0.00029789676552149393, 'sport': 0.0002967507919457565, 'prisoners of war': 0.0002916559366440846, '1980s': 0.00029087747363588197, 'governor': 0.0002902078137295467, 'loss of father': 0.0002879052533181265, 'road movie': 0.00028434340579758623, 'thanksgiving': 0.0002833410928443928, 'widow': 0.00028239771074086724, 'high school student': 0.0002813887847765128, 'hawaii': 0.0002803293143401124, 'fear': 0.0002802586466861169, 'secret intelligence service': 0.0002780883431635327, 'usa president': 0.0002780735803505468, 'computer': 0.00027714799494500295, 'desperation': 0.00027425723591302514, 'highway': 0.00027355865309345044, 'police operation': 0.00027353849024351173, 'addiction': 0.00027318406998953006, 'thailand': 0.0002722403391406302, 'teenage girl': 0.0002721674953569076, 'death': 0.0002715470497047372, 'boston': 0.00027141051916270765, 'world war ii': 0.00027121253432858176, 'pirate': 0.0002704468216300957, 'heart attack': 0.00026743003623607266, 'marriage': 0.0002667932449630727, 'pistol': 0.0002643067858790097, 'business': 0.0002632404515633882, 'angel': 0.0002609842534910646, 'japan': 0.0002592420206070454, 'satire': 0.00025257664572236166, 'submarine': 0.0002478712209115676, 'upper class': 0.0002456483841799184, 'escape from prison': 0.0002455476824909322, 'politician': 0.0002446810654275904, 'rural setting': 0.0002442980223853371, 'forest': 0.00024377471462653198, 'male nudity': 0.00024303206412540942, 'baseball': 0.00024170084504049203, 'corpse': 0.00024168537521543533, 'vietnam veteran': 0.00023923149655866575, 'high school': 0.0002392130467715119, 'spirit': 0.00023833822270529336, 'unemployment': 0.00023806225823426265, 'period drama': 0.00023435768550763726, 'unsociability': 0.00023299407021176773, 'false identity': 0.00023232375425080043, 'new orleans': 0.00023108199235635034, 'pop star': 0.00023000838115684545, 'drama': 0.0002268924069978847, 'martial arts': 0.00022504752046897556, 'flashback': 0.00022344961431136124, 'marijuana': 0.00022269912321910886, 'single': 0.0002208833450372387, 'soviet union': 0.0002201403909197278, 'heroin': 0.00021817003666884153, 'prostitute': 0.00021812670231069757, 'skiing': 0.00021804996966425183, 'transformation': 0.00021684191387128063, 'princess': 0.0002162010608191676, 'dog': 0.000214974041831329, 'demon': 0.00021244847989890483, 'vigilante': 0.00021038197731619823, 'coming of age': 0.00021001214514799346, 'neighbor': 0.000209249198989637, 'killer': 0.00020920334250913594, 'virtual reality': 0.00020759998360290284, 'assassination': 0.00020621868613117652, 'gangster': 0.00020547735837680615, 'teen comedy': 0.00020545151069727253, 'summer': 0.00020481496777339413, 'loss of family': 0.00020310109959004614, 'undercover cop': 0.00020221833929699962, 'dc comics': 0.000201804581177081, 'hollywood': 0.00019635927212535579, 'teleportation': 0.00019399197037860925, 'nudity': 0.00019277635107817627, 'mother daughter relationship': 0.00019192960304162935, 'adultery': 0.00018931969600536884, 'newspaper': 0.00018722736672255162, 'suspicion of murder': 0.00018668063216405726, 'auto': 0.0001865930308154438, 'serial killer': 0.00018174014224044908, 'smuggling': 0.00018086776205241016, 'paranoia': 0.0001799268655428724, 'child abuse': 0.00017982379836827605, 'secret': 0.00017970348754936137, 'mask': 0.00017835981555444164, 'music': 0.00017697865257195258, 'american football': 0.0001744145173095946, 'divorce': 0.0001736110116354617, 'surrealism': 0.0001718179216185053, 'ghost': 0.00017145557641308663, 'planned murder': 0.00017109954569191976, 'pilot': 0.00017036844050318423, 'officer': 0.00016871665335869643, 'space marine': 0.00016856331720440304, 'coming out': 0.0001678374006290249, 'male female relationship': 0.0001664018910785443, 'resistance': 0.0001658060971562939, 'eroticism': 0.00016453594739700806, 'vampire': 0.00016292952492924165, 'usa': 0.00016243108735679063, 'gun': 0.00016208680963300556, 'sadism': 0.00016185117977174425, 'hip-hop': 0.00015860168290428542, 'bite': 0.00015859708599085632, 'success': 0.00015851840641237106, 'black people': 0.00015633545126416646, 'radio': 0.00015629621527730547, 'mother': 0.00015609183325396045, 'shotgun': 0.00015538917837812024, 'becoming an adult': 0.00015401326908569285, 'ex-cop': 0.00015393249265153905, 'afghanistan': 0.00015048848742454636, 'nasa': 0.00014985997829133235, 'royalty': 0.00014891536594760735, 'young adult': 0.0001485630594316747, 'airport': 0.00014823424073650274, 'one-night stand': 0.00014741690211709545, 'female friendship': 0.00014635544441705373, 'bomb': 0.00014540816829215362, 'whale': 0.00014516286180480764, 'gunfight': 0.0001447201325445302, 'human experimentation': 0.00014392628650040554, 'crime': 0.0001437541637814055, 'african american': 0.00014370362095833409, 'rivalry': 0.0001426197971886347, 'funeral': 0.0001420233397203946, 'artificial intelligence': 0.000139158641394287, 'occult': 0.00013802574289934277, 'sea monster': 0.0001377604546762353, 'circus': 0.00013755867450079184, 'ensemble cast': 0.000137256591560337, 'car accident': 0.00013724195464786688, 'based on play or musical': 0.0001368376095194241, 'teacher': 0.00013622686324337748, 'tattoo': 0.00013494901146161978, 'criminal': 0.00013413642830428365, 'immortality': 0.00013317528003012157, 'film making': 0.00013161691770517252, 'anger': 0.00013123795580545456, 'vietnam': 0.0001308096830265043, 'capitalism': 0.00012907241314210184, 'anime': 0.0001285185786740701, '19th century': 0.0001246532600187363, 'iraq': 0.00012428672908723352, 'mistaken identity': 0.00012392082760315904, 'religion': 0.0001228780094026437, 'ancient rome': 0.00012216488454778887, 'military': 0.00012022315743036514, 'berlin': 0.0001201309578168791, 'agent': 0.00011841535141412723, 'chicago': 0.00011806326496892035, 'world war i': 0.00011771873133062173, 'misfit': 0.00011719201241469648, 'cover-up': 0.00011687290353980519, 'mafia': 0.00011653671791292557, 'vietnam war': 0.00011573496655293915, 'sheriff': 0.00011495446200653265, 'killing': 0.00011470897088949474, 'death of a friend': 0.00011428217215432644, 'car journey': 0.00011398609390660514, 'hypnosis': 0.00011394400643195328, 'slavery': 0.00011335874507987095, 'virgin': 0.00011182356437045102, 'hallucination': 0.00011103008460599484, 'court case': 0.00010775908911753647, 'loss of mother': 0.00010762142897697606, 'lesbian': 0.00010466171987314463, 'suspicion': 0.00010454059072911037, 'slasher': 0.0001043624806689626, 'theft': 0.00010375234726886752, 'comedian': 0.00010356841349952414, 'cult film': 0.0001027160283867448, 'swimming pool': 0.00010220411004539334, 'cold war': 0.00010176875818360844, 'supernatural': 0.0001004805714782863, 'sexuality': 0.00010009790016495587, 'photography': 9.951176806948078e-05, 'school': 9.887671489513805e-05, 'silent film': 9.801445054710357e-05, 'college': 9.694221291794838e-05, 'interracial relationship': 9.690346764853527e-05, 'hoodlum': 9.55088119387198e-05, 'olympic games': 9.5111008282597e-05, 'mexico': 9.501864368548557e-05, 'underdog': 9.264300919900656e-05, 'twins': 9.218140548455926e-05, 'street gang': 9.195009467326449e-05, 'manipulation': 9.166473601936161e-05, 'science': 9.125359645877185e-05, 'dirty cop': 8.973779876220319e-05, 'police officer': 8.81726214736295e-05, 'historical figure': 8.77821586199271e-05, 'apartment': 8.737109646617908e-05, 'identity': 8.723308087794302e-05, 'incest': 8.659048427243365e-05, 'lgbt': 8.623209362495643e-05, 'southern usa': 8.606034547781236e-05, 'mass murder': 8.550007405221284e-05, 'rebel': 8.469441236097837e-05, 'massacre': 8.389301206497682e-05, 'roommate': 8.193755859864415e-05, 'gay': 8.143890060110567e-05, 'sister': 8.143154378147899e-05, 'author': 8.138458670469903e-05, 'dream': 8.123630676844227e-05, 'stop motion': 8.075413173360503e-05, 'american': 7.96796653712162e-05, 'british': 7.96377217504587e-05, 'curse': 7.921296414878606e-05, 'alaska': 7.921219945326676e-05, 'alcoholic': 7.751156622085278e-05, 'nightmare': 7.7060389419361e-05, 'africa': 7.53664688774826e-05, 'space travel': 7.527373585468661e-05, 'survivor': 7.505694731240096e-05, 'surveillance': 7.466860411255547e-05, 'loneliness': 7.398691893687982e-05, 'interview': 7.281768601433164e-05, 'disguise': 7.113880992632107e-05, 'fugitive': 7.091353037407196e-05, 'female protagonist': 7.014025611499898e-05, 'gold': 6.956089078124895e-05, 'loss of lover': 6.851206950050146e-05, 'mercenary': 6.81725066250854e-05, 'hacker': 6.739582299437252e-05, 'church': 6.67449164896812e-05, 'disaster': 6.644417876667144e-05, 'mockumentary': 6.587959083353025e-05, 'outer space': 6.53378642918979e-05, 'outlaw': 6.472491659117752e-05, 'teen movie': 6.457549640987912e-05, 'ballet': 6.256064951189292e-05, 'secret love': 6.230935100086065e-05, 'bully': 6.217141297917408e-05, 'memory': 6.170428363050166e-05, 'terror': 6.164509344134034e-05, 'gangster boss': 6.164116125637916e-05, 'texas': 6.131023917498259e-05, 'handcuffs': 6.102986407690836e-05, 'existentialism': 6.03526666551645e-05, 'disabled': 5.910278418301598e-05, 'drug abuse': 5.856347299946856e-05, 'epic': 5.78601073527978e-05, 'sword fight': 5.754225397404651e-05, 'nightclub': 5.6476243474149e-05, 'suicide attempt': 5.619101065908165e-05, 'shootout': 5.547448940539832e-05, 'retirement': 5.487522489326746e-05, '1970s': 5.458635949162851e-05, 'priest': 5.4579883778940676e-05, 'prayer': 5.412015162449313e-05, 'journalism': 5.244320448342276e-05, 'right and justice': 5.233445554428099e-05, 'ransom': 5.198078867336659e-05, 'extreme violence': 5.1885021129231444e-05, 'stand-up comedy': 5.1864510715153914e-05, 'history': 5.162135383748652e-05, 'journalist': 5.123137760228227e-05, 'nun': 5.0517885391732455e-05, 'sword': 5.007860022582351e-05, 'dance': 4.80201981379335e-05, 'organized crime': 4.781375371828783e-05, 'beach': 4.734082384084627e-05, 'lesbian relationship': 4.6420137951071674e-05, 'father daughter relationship': 4.465442864860233e-05, 'lie': 4.4563650772468804e-05, 'film noir': 4.2320999032298926e-05, 'american civil war': 4.2250810477533445e-05, 'policeman': 4.1658685974816596e-05, 'cyberpunk': 4.1387266990780384e-05, 'decapitation': 4.136841147971622e-05, 'father': 4.1250411425904054e-05, 'puppet': 3.942981143091629e-05, 'train': 3.932258913448876e-05, 'blood splatter': 3.93045701754576e-05, 'kung fu': 3.915882923587892e-05, 'witness': 3.908591562360068e-05, 'neo-noir': 3.841564910201865e-05, 'evil': 3.836150147592509e-05, 'russia': 3.811115568745006e-05, 'animal attack': 3.783455357632287e-05, 'police corruption': 3.7663167418910144e-05, 'seduction': 3.723768981740246e-05, 'gang': 3.719033081748266e-05, 'motel': 3.67084735828007e-05, 'court': 3.6693350817382207e-05, 'car race': 3.6303761452836244e-05, '1950s': 3.45729794679025e-05, 'wilderness': 3.454521189715002e-05, 'power': 3.4065749917600166e-05, 'bar': 3.3976665828715325e-05, 'fistfight': 3.395258179075466e-05, 'miracle': 3.301257353130975e-05, 'obsession': 3.2519556453344155e-05, 'vacation': 3.1922240748702146e-05, 'cheating': 3.0441054610755163e-05, 'australia': 3.015810886333231e-05, 'stripper': 2.9956157033409235e-05, 'aids': 2.8914510482560938e-05, 'lovesickness': 2.855103877220349e-05, 'lone wolf': 2.662885186571204e-05, 'biker': 2.5579861991397108e-05, 'arbitrary law': 2.5193954277319323e-05, 'hell': 2.5171367638343183e-05, 'high school sports': 2.4739272256759985e-05, 'transvestism': 2.4606530879145563e-05, 'parallel world': 2.4013345429096407e-05, 'career': 2.4001086801154947e-05, 'isolation': 2.367606713935494e-05, 'diner': 2.305414375279757e-05, 'camping': 2.2811521834493374e-05, 'guilt': 2.2589165675788986e-05, 'cop': 2.1209008944918997e-05, 'girlfriend': 2.119875754622512e-05, 'hotel room': 2.089217206302807e-05, 'brutality': 2.0322223607434956e-05, 'politics': 1.8953460101592997e-05, 'subway': 1.859150773071128e-05, 'cyborg': 1.8327505407036378e-05, 'writer': 1.800445654654913e-05, 'cemetery': 1.7310254648610943e-05, 'cigarette smoking': 1.7146357531504506e-05, 'dancer': 1.668259176427207e-05, 'stalker': 1.651922820065141e-05, 'voyeur': 1.6433548364106032e-05, 'end of the world': 1.635578246053831e-05, 'waitress': 1.6106463187266975e-05, 'mutation': 1.5973396819212307e-05, 'killer robot': 1.545078691080249e-05, 'heist': 1.544911560875149e-05, 'family holiday': 1.5444705546423795e-05, 'gore': 1.48573721718819e-05, 'travel': 1.4848400615162748e-05, 'slapstick': 1.473115865521147e-05, 'internet': 1.4632590714889042e-05, 'musician': 1.3652687369733273e-05, 'strip club': 1.342068953056029e-05, 'mumblegore': 1.234843494714283e-05, 'patriotism': 1.094827138371244e-05, 'germany': 1.0512906445202305e-05, 'prom': 1.0476786586049881e-05, 'bus': 1.0219420660080074e-05, 'cocaine': 9.94892943536506e-06, 'drug addiction': 9.743836386641633e-06, 'nurse': 9.202932457607148e-06, 'police brutality': 8.884479769237237e-06, 'smoking': 8.820944158090273e-06, 'homosexuality': 8.62387310734474e-06, 'brazilian': 8.415799022300033e-06, 'loss of virginity': 7.88806806857381e-06, 'bank robbery': 7.560006684429486e-06, 'birthday': 7.482012618566678e-06, 'singer': 7.3642880090789765e-06, 'white house': 6.971656767125286e-06, 'burglar': 6.613134555720722e-06, 'celebrity': 6.384494476731872e-06, 'factory': 5.238099935121458e-06, 'liar': 5.107587807456909e-06, 'juvenile delinquent': 4.97078245358291e-06, 'brother': 4.6541447829535245e-06, 'supermarket': 4.541645688162867e-06, 'tragedy': 2.5317725892813395e-06, 'jazz': 1.7185782257818856e-06, 'crush': 1.5719888625775818e-06, 'parody': 8.544019156243455e-07, 'depression': 3.7367176717654876e-07, 'van': 1.5703310591056257e-07}
# over_dict = scores_from_text('overview')
over_dict = {'threatens': 0.022368482236060437, '‚Äì': 0.01908350164844564, 'realizes': 0.017871576303104937, 'epic': 0.015597127740954139, 'get': 0.015265341525926897, 'home': 0.014805904145627356, 'war': 0.013895182243976147, 'against': 0.012786870662163477, 'mysterious': 0.011598379527889589, 'Earth': 0.011384347279964933, 'seems': 0.010876148537013222, 'brother': 0.010792039088741514, 'where': 0.010654917698105075, 'break': 0.01026677772690925, 'classic': 0.010099929427199592, 'unexpected': 0.01008022553637202, 'man': 0.00901959272865307, 'nothing': 0.008557519782364315, 'put': 0.008554018418410433, 'protect': 0.008376839462839774, 'at': 0.008247161449249765, 'world': 0.008050919303601132, 'love': 0.007864804390508783, 'in': 0.007713811111690954, 'meet': 0.007335474324824913, 'children': 0.007303952246468977, 'been': 0.007039391597142923, 'truth': 0.006855284820710879, 'even': 0.006578197442920643, 'chance': 0.0065293947184997285, 'dead': 0.006463050325274654, 'sends': 0.006451091009749335, 'forces': 0.006430536646703006, 'struggle': 0.006318501815653452, 'than': 0.006289882047970615, 'destroy': 0.006189139425839748, 'sent': 0.006098032333107879, 'their': 0.006052326305302328, 'members': 0.006043657474636395, 'and': 0.006017112406193973, 'rival': 0.005863487367718391, 'John': 0.005739157894045456, 'dangerous': 0.005626985256826595, 'human': 0.005533097473382662, 'him': 0.005515161352418985, 'Sam': 0.005485684340802622, 'series': 0.005458165213977033, 'back': 0.005377586606582015, 'knows': 0.005332254235007876, 'to': 0.0052533666290129006, 'true': 0.005198566645697511, 'a': 0.004991489363636566, 'are': 0.004858192027610698, 'And': 0.004730505251044683, "she's": 0.004666655363634861, 'past': 0.004628015611027973, 'his': 0.004531611001424876, 'late': 0.004507986768318357, 'dark': 0.004403180404405201, 'ship': 0.0042608214072128465, 'journey': 0.004242381375412621, 'friends': 0.004134239714273314, 'or': 0.004038067852951085, 'day,': 0.0039983394260118595, 'City': 0.0039556413468574015, 'learn': 0.003876993064550073, 'being': 0.0038338192882910425, 'returns': 0.0037688087301535, 'female': 0.0036700925585229994, 'takes': 0.003647072291330425, 'forever': 0.003632561714309271, 'film': 0.0036126362953157985, 'global': 0.0036046569924029067, '-': 0.0035893518428105086, 'family,': 0.0034965104691727705, 'family': 0.003493235379809956, 'death': 0.0034837897391754224, 'other': 0.0034601869144564347, 'behind': 0.0034495550123922482, 'While': 0.003434446984299551, 'space': 0.003407045231129357, 'The': 0.0033830006215123274, 'mission': 0.0033382043478962344, 'around': 0.0032997270771775015, 'go': 0.0032686234283048113, 'means': 0.003260383583136236, 'CIA': 0.0032235554224411176, 'enemy': 0.003216664694773036, 'plot': 0.0032054224206717703, 'prove': 0.0032012498238861613, 'falling': 0.00319667696908259, 'arrives': 0.003159329101598428, 'upon': 0.0031333260774330123, 'tries': 0.0031266822798232514, 'revenge': 0.0030626781574945973, 'be': 0.003039135307945007, 'soon': 0.003016373736900389, 'well': 0.0030079455801650706, 'streets': 0.003006486435901152, 'seeks': 0.0029932464294747277, 'from': 0.002966341825503978, 'make': 0.002942947258794684, "father's": 0.002930362235807252, 'race': 0.0029101715250535003, 'that': 0.0028999984919584752, 'she': 0.002884217078304079, 'An': 0.002834726317369757, 'much': 0.0028150243382405377, 'case': 0.0028066448506815405, 'adventure': 0.002800154302714966, 'Russian': 0.0027830872917837925, 'As': 0.0027575842733510002, 'In': 0.00271574072417081, 'with': 0.0026986065490037557, 'named': 0.0026860052804841927, 'an': 0.0026780466676715577, 'novel': 0.002669821611869672, 'One': 0.0026402011445848684, 'wealthy': 0.0026141610868344505, 'under': 0.0026043517033226453, 'any': 0.0025672862130589336, 'city': 0.0025357737350710776, 'test': 0.0025245516165446065, 'as': 0.0025201635810230202, 'things': 0.0025002512417705413, 'heart': 0.0024900525139930547, 'evil': 0.002461056820332162, 'crime': 0.002450793056446957, 'change': 0.002448715512276306, 'three': 0.002436351898177097, 'our': 0.002410813216689363, 'can': 0.0023233615439010595, 'fight': 0.002317552210063878, 'hope': 0.002314792004073444, 'leads': 0.002292561890775674, 'themselves': 0.0022901211712303415, 'But': 0.002281786497751562, 'leader': 0.002256551235195056, 'led': 0.002226776134560854, 'then': 0.0022070207404767173, "who's": 0.0021954122481381, 'he': 0.0021414194936693826, 'investigate': 0.0021354325164584156, 'by': 0.0020971289054894085, 'rescue': 0.0020722562025994847, 'father': 0.0020706059845488566, 'face': 0.0020527664152608597, 'Now': 0.0020526126600060994, 'have': 0.0020227569970044652, 'land': 0.0020009703720987896, 'turned': 0.001989832046940765, 'must': 0.0019472658463201176, 'What': 0.0019297425684986693, 'together': 0.001919581403184015, 'years': 0.0018704613018512722, 'attempt': 0.0018701876597390886, 'Jack': 0.0018565314579319398, 'who': 0.0018492899985194002, 'team': 0.0018462339127423386, 'new': 0.0018290308164694413, 'discover': 0.0018270968020652795, 'travel': 0.001800954903880401, 'away': 0.0017938888804856982, 'agrees': 0.0017838936743594454, 'director': 0.001761560093101223, 'known': 0.0017370230091189407, 'during': 0.0017332919912897396, 'day': 0.0016953807250008, 'police': 0.0016880475955712115, 'ancient': 0.0016722664477898494, 'do': 0.0016347665038985864, 'later,': 0.0016230911287633324, 'far': 0.0016206874228000493, 'different': 0.0016155756245730482, 'sets': 0.0016006790051540458, 'lead': 0.0015792465375205219, 'place': 0.0015724244266349225, 'track': 0.0015700880201609234, 'had': 0.0015351705153845885, 'for': 0.0015294355668468266, 'was': 0.0015236982716812005, 'married': 0.0015180603958147687, 'anything': 0.001516805191754097, 'Police': 0.001515783879617233, 'terrorist': 0.0014999556837898948, 'one': 0.0014598989670103998, 'into': 0.0014300520504229057, 'father,': 0.0014283732293883383, 'military': 0.0013774580795047817, 'goes': 0.0013731780130669439, 'story': 0.001371712674337637, 'leading': 0.0013666080557827834, 'year': 0.0013404690090691703, 'book': 0.001335738546897576, 'woman': 0.001332431910538855, 'give': 0.0013307851813510146, 'child': 0.0013123289766066063, 'living': 0.0013107754916174487, 'son': 0.0013069501357374253, 'head': 0.0013056582743164128, 'but': 0.0012948608163277906, 'becomes': 0.001294477112003759, 'discovers': 0.0012907318490704186, 'finds': 0.0012844352901219287, "he's": 0.0012649614856669108, 'tale': 0.001261255976416174, 'something': 0.0012395193528633784, 'struggles': 0.0012392111665356334, 'FBI': 0.001236430507221197, 'the': 0.001223609599080851, 'too': 0.0012188026720127737, 'experience': 0.0012187706038802094, 'escape': 0.0012105521012129915, 'out': 0.001190845776656273, 'girlfriend': 0.0011907771543059635, 'after': 0.0011834464359878785, 'on': 0.0011718690590863587, 'desperate': 0.00115650546629792, 'With': 0.001145711593361001, 'learns': 0.0011432379214650805, 'few': 0.0011355436153330277, 'given': 0.0011323140418452592, 'attempts': 0.0011291945181420167, 'of': 0.00110725703152091, 'near': 0.001105496987839029, 'identity': 0.0011000123272703996, 'if': 0.0010994293438492753, 'To': 0.0010950490149209172, 'thing': 0.0010862962160202438, 'left': 0.0010845891912672008, 'After': 0.0010809244283819107, 'see': 0.0010599098258597408, 'A': 0.0010480242900556239, 'turn': 0.0010472877657536086, 'has': 0.001045818263259344, 'entire': 0.001031837706179798, 'become': 0.0010170179359522222, 'own': 0.0010118715272842175, 'last': 0.0010090831853819134, 'so': 0.0010018663754768197, 'them': 0.0009893678725812944, 'young': 0.0009739068372608236, 'World': 0.0009662448948444379, 'while': 0.0009579225158202322, 'two': 0.0009577194786640425, 'win': 0.0009480783797044869, 'is': 0.0009425524369366063, 'always': 0.000934179754860913, 'gang': 0.000927538456547513, 'accident': 0.0009275055421995556, 'know': 0.0009207149476076744, 'it': 0.0009190958272499213, 'will': 0.0008872689231126166, 'plans': 0.0008861372791065643, 'time': 0.0008717973957924688, 'friend': 0.0008714360858590751, 'When': 0.0008590111492661593, 'join': 0.0008525769015380778, 'days': 0.0008498672616275135, 'decides': 0.0008355810362049784, 'caught': 0.000825098281499639, 'murdered': 0.0008169727173997495, 'husband,': 0.0008066713739388916, 'end': 0.0008039454370580467, 'also': 0.0007946414740277082, 'save': 0.0007827616126318791, 'seek': 0.0007759548257517149, 'beautiful': 0.0007734468120033966, 'live': 0.0007692735921809954, 'about': 0.0007671628760157632, 'come': 0.0007658813232202916, 'down': 0.0007645053331749525, 'meets': 0.000762255932282002, 'kids': 0.0007613129850435691, 'this': 0.0007552917444876684, 'her': 0.0007452888787856588, 'romantic': 0.0007405944287251776, 'what': 0.0007399743873087447, 'begins': 0.0007389282171841869, 'when': 0.0007386424378050569, 'estranged': 0.0007289958779270703, 'powerful': 0.0007162461169221234, 'Now,': 0.0007045632345543771, 'comes': 0.000703881033350784, 'name': 0.000703056261160783, 'undercover': 0.0006957421329442854, 'leave': 0.0006954085082209242, 'criminal': 0.0006781801594100936, 'private': 0.0006776537028476842, 'professional': 0.0006612881038720229, 'men': 0.0006608229522270367, 'before': 0.0006603387091621896, 'King': 0.000647278215336264, 'women': 0.0006462239880939337, 'never': 0.0006447196551847641, 'prison': 0.0006412213880125709, 'over': 0.0006356954353569069, 'movie': 0.0006319034107449402, 'murders': 0.0006267633730748538, 'rich': 0.0006250568888723935, 'game': 0.0006239604139422244, "it's": 0.0006183962837741044, 'battle': 0.0006134375802867565, 'London': 0.0006101782338691481, 'turns': 0.0006018704268209774, 'fall': 0.0005990801941607199, 'kidnapped': 0.000598280304703982, 'try': 0.000591111398524686, 'quest': 0.0005864564931900586, 'business': 0.0005832425627971829, 'top': 0.0005782172775520576, 'brings': 0.000576470669392522, 'Dr': 0.0005722134705382059, 'country': 0.0005712366276407316, 'falls': 0.0005659929298746053, 'found': 0.0005655339572351158, 'between': 0.0005651378259844373, 'wild': 0.0005558029067805994, 'home,': 0.0005462272859342156, 'husband': 0.0005331171256866775, 'personal': 0.0005212560121809212, 'Captain': 0.0005179092894987253, 'forced': 0.0005166090221282259, 'becoming': 0.000516008545907147, 'dream': 0.0005115626897886701, 'now': 0.0005115582111307109, 'life,': 0.0005102224440864271, 'help': 0.0005062437869151756, 'every': 0.000503085603773875, 'all': 0.0004971770169862742, '--': 0.0004900751445683792, 'missing': 0.00048820392506671625, 'struggling': 0.0004873418345750869, 'lost': 0.00048547624381005785, 'accidentally': 0.0004838214736466237, 'could': 0.0004820641231848741, 'Mr': 0.00048054000620244834, 'They': 0.00047637497803694744, 'life': 0.0004738302273402244, 'search': 0.00047109500942935784, 'real': 0.00046967881566594, 'remote': 0.00046962187133707436, 'full': 0.0004693445010451289, 'York': 0.00045860845489124446, 'runs': 0.00045829578206730794, 'up': 0.0004462932925571315, 'White': 0.0004371762026944228, 'some': 0.0004350630912086757, 'across': 0.00043222413557156877, 'until': 0.0004299619507813401, 'relationship': 0.00042889339303675024, 'son,': 0.0004262462353688262, 'within': 0.00041822554719950537, 'hired': 0.00041676958341729163, 'order': 0.0004124971773325103, 'former': 0.0004097659523796182, 'James': 0.0004088909940240591, 'plan': 0.0004078880438073808, 'famous': 0.00040673998620343807, 'French': 0.0004050372348131234, 'trapped': 0.00040038697953866787, 'New': 0.00039967126362747693, 'tells': 0.00039841519260909886, 'through': 0.00039544529413887885, 'way': 0.00039165802078548046, 'heist': 0.00038860466193141373, 'want': 0.00038848365972299576, 'killer': 0.0003830421925742478, 'brutal': 0.00037979507136012933, 'crew': 0.00037864454058692725, 'whose': 0.0003779166828777031, 'For': 0.00037758132040465337, 'road': 0.0003730577155848397, "It's": 0.00036544791160872186, 'like': 0.0003648676025212625, 'black': 0.00036034481624068113, 'joins': 0.0003563257857338632, 'control': 0.000353365756327881, 'look': 0.00034788803436312603, 'most': 0.00034756826900734177, 'these': 0.00034464116938824163, 'four': 0.0003423878541099868, 'agent': 0.00034122472433025755, 'college': 0.0003361839164167132, 'bring': 0.0003359161398333628, 'village': 0.0003353209345498132, 'taking': 0.0003338675667072448, 'involved': 0.00032421969565085424, 'later': 0.00032230526973603114, 'cop': 0.00032230192769320436, 'trying': 0.00032190634042148414, 'hero': 0.0003213718182494091, 'perfect': 0.0003192655264488511, 'more': 0.00031727031127515737, 'ends': 0.0003150732276677387, 'gets': 0.00031384529524733996, 'English': 0.0003122174418027145, 'five': 0.00031077618063465623, 'class': 0.00031049766956279797, 'side': 0.0003103303492060927, 'faces': 0.0003076415709141205, 'scientist': 0.00030713299426827806, 'Based': 0.00030676794495259284, 'no': 0.0003006150449044819, 'events': 0.0002970866632328569, 'strange': 0.0002953650087020636, 'characters': 0.0002935838727904503, 'terrifying': 0.00029240206462356874, 'lives': 0.00029191529204065654, 'they': 0.00029160272709057836, 'including': 0.0002891826494323466, 'move': 0.0002883535061925477, 'first': 0.00028373637309942986, 'part': 0.0002832783383765353, 'herself': 0.0002826392899249987, 'once': 0.0002804063607992625, 'drug': 0.00027914329185059814, 'single': 0.00027521081843588223, 'assassin': 0.00027039978321725456, 'pair': 0.0002698089754892257, 'its': 0.0002694825950270733, 'alone': 0.00026869416535884195, 'United': 0.0002679070902589046, 'just': 0.0002655922549460839, 'popular': 0.0002652598530724267, 'eventually': 0.00026321681583556684, 'people': 0.00026149472140082284, 'Los': 0.0002579209646196273, 'At': 0.00025783865037364907, 'everything': 0.00025580959629809333, 'secret': 0.00025130298504955886, 'set': 0.00024917841936392793, 'future': 0.0002467199680550481, 'serial': 0.00024665696409562445, 'Along': 0.0002402705510477633, 'trouble': 0.00023963346353478385, 'power': 0.00023751743152509059, 'guy': 0.0002333786741073645, 'couple': 0.00023317166612745946, 'drama': 0.0002325844271342842, "can't": 0.00023225280892301016, 'form': 0.00022838792818185772, 'figure': 0.0002281383514412975, 'him,': 0.00022583054982669258, 'determined': 0.00022423445161426323, 'British': 0.00022049023735464033, "doesn't": 0.0002185019930200628, 'government': 0.00021611911712828948, 'there': 0.0002156515882484349, 'car': 0.00021440204428240526, 'house': 0.00021163311521824307, 'Meanwhile,': 0.0002099791612111764, 'group': 0.00020985115908943446, 'international': 0.000208621585024637, 'take': 0.0002074253591976514, 'only': 0.0002052587890562712, 'find': 0.00020511826739638846, 'complicated': 0.00020008626300698445, 'based': 0.0001985551586047262, 'career': 0.0001933573051701768, 'music': 0.00019268709093372573, 'those': 0.00019093681208683092, 'dreams': 0.0001901462280021735, 'very': 0.00018958839871884822, 'friendship': 0.00018702698237973254, 'show': 0.00018618328784876693, 'makes': 0.00018539326002410336, 'hit': 0.00018293561492514597, 'student': 0.00018190642900926124, 'His': 0.00018065568614584557, 'fellow': 0.00017322952390105568, 'legendary': 0.0001708662089289896, 'yet': 0.000170806487629841, 'little': 0.00016999334022100708, 'early': 0.00016444029700447716, 'century': 0.0001626009756437574, 'latest': 0.0001619348171151287, 'getting': 0.00016054498753445922, 'officer': 0.00015785040427415301, 'because': 0.0001575745456670017, 'not': 0.00015736476685316412, 'Paul': 0.0001542538900316354, 'himself': 0.0001537568832489298, 'having': 0.0001525178861641101, 'survive': 0.0001518549096041069, 'criminals': 0.00015040379793624306, 'which': 0.00014944774688156195, 'despite': 0.0001483541106957309, 'mother': 0.00014398432462827505, '': 0.00014146067938313015, 'America': 0.00014055046404228038, 'childhood': 0.00014017711230083355, 'making': 0.00013993568969973574, 'army': 0.00013902753241337715, 'Angeles': 0.00013394372703999144, 'special': 0.00012833327521931992, 'He': 0.0001280338914774966, 'summer': 0.0001245843224015098, 'ruthless': 0.00012125593975374088, 'owner': 0.00011914463008995135, 'US': 0.00011839654653330166, 'along': 0.00011777403262164777, 'American': 0.0001171840018679681, 'right': 0.00011656163933448219, 'same': 0.00011650022636287235, 'soldiers': 0.00011249670032958435, 'bond': 0.00011221793607236632, 'work': 0.00011175803147031088, 'and,': 0.00011078310287639588, 'since': 0.00011043423770571387, 'wife': 0.00010727919103632418, 'unlikely': 0.00010702282014485269, 'school': 0.00010601168548087097, 'fights': 0.00010554189991943377, 'both': 0.00010504646359639383, 'escapes': 0.0001050394620582783, 'boy': 0.00010211312905398643, 'boyfriend': 0.00010181693411501697, 'follows': 0.00010125831993084435, 'might': 0.00010112035961531406, 'successful': 9.893721382040992e-05, 'alien': 9.877146629661304e-05, 'greatest': 9.821664664762912e-05, 'great': 9.792703944653584e-05, 'off': 9.750309704453528e-05, 'planet': 9.714784473659812e-05, 'start': 9.66252894356222e-05, 'course': 9.44446760473422e-05, 'fighting': 9.365293793851847e-05, 'old': 9.30335676711209e-05, 'Set': 9.220502313417496e-05, 'Two': 9.214680099662658e-05, 'best': 9.173452455789145e-05, 'War': 8.947466556886966e-05, 'detective': 8.907424643502546e-05, 'It': 8.840788079760974e-05, 'marry': 8.675252242800865e-05, 'told': 8.615338353651231e-05, 'inside': 8.60038503707024e-05, 'another': 8.55996267854946e-05, 'affair': 8.443227998410538e-05, 'were': 8.162262511328777e-05, 'murder': 8.159812036921966e-05, 'force': 8.003927015550026e-05, 'changes': 7.849433815736876e-05, 'still': 7.826055899395166e-05, 'following': 7.76847442284592e-05, 'wants': 7.532795542278481e-05, 'job': 7.459421833107295e-05, 'body': 6.919556215290926e-05, 'years,': 6.904941799206367e-05, 'again': 6.853076442736274e-05, 'visit': 6.836589353835163e-05, 'company': 6.723280331323776e-05, 'town,': 6.652517325874164e-05, 'does': 6.556727919397874e-05, 'romance': 6.530121534357643e-05, 'documentary': 6.455232339259668e-05, 'attack': 6.407747234459425e-05, 'star': 6.364307438595612e-05, 'suddenly': 6.339427990929825e-05, 'kill': 6.333319654461871e-05, 'made': 6.317334287325987e-05, 'deadly': 6.30526023373411e-05, 'stories': 6.285621115393121e-05, 'involving': 6.119768125368942e-05, 'good': 6.116703108572787e-05, 'girl': 6.0679630246698085e-05, 'next': 5.9421373718212165e-05, 'starts': 5.936313747052548e-05, 'wife,': 5.8292696816545034e-05, 'marriage': 5.606348917995388e-05, 'though': 5.552087159491038e-05, 'called': 5.4469370663412835e-05, 'final': 5.3351800767092634e-05, 'really': 5.1216399966811514e-05, 'desert': 5.066265046513263e-05, 'how': 4.9836266160314466e-05, 'return': 4.8654137681784856e-05, 'each': 4.737115472330599e-05, "family's": 4.719903315493493e-05, 'older': 4.7012908880609096e-05, 'David': 4.68495605759426e-05, 'big': 4.623995591098539e-05, 'many': 4.547119025181797e-05, 'love,': 4.519874359303312e-05, 'violent': 4.478389836838271e-05, 'During': 4.432445950748794e-05, 'wedding': 4.429178024013631e-05, 'boss': 4.133317174608269e-05, 'high': 4.050737243591626e-05, 'long': 3.943892287061399e-05, 'finding': 3.675361660064021e-05, 'may': 3.5976269253448465e-05, 'decide': 3.569157991269337e-05, 'rise': 3.507140275600793e-05, 'going': 3.4897211284577793e-05, 'deep': 3.4863387664968535e-05, 'works': 3.384534129692212e-05, 'local': 3.3792324966055067e-05, 'travels': 3.377651891616711e-05, 'town': 3.361971137172364e-05, 'better': 3.347581904436333e-05, 'quickly': 3.303030601965902e-05, 'girls': 3.2596996767282466e-05, 'convinces': 3.2393228328941264e-05, 'comedy': 3.18695352109982e-05, 'seemingly': 3.1855608067234205e-05, 'reporter': 3.130966139353279e-05, 'Tom': 3.106171749471902e-05, 'her,': 3.0636617983858534e-05, 'bank': 3.0282468432032664e-05, 'party': 3.0282244882640125e-05, 'friends,': 2.997878117938684e-05, 'however,': 2.983066623663876e-05, 'She': 2.9790061033485765e-05, 'killed': 2.9114697165388532e-05, 'night': 2.889407673195614e-05, 'finally': 2.887678427488413e-05, 'Paris': 2.8694091395531575e-05, 'reality': 2.767801157278872e-05, 'beauty': 2.72790820395727e-05, 'action': 2.667695967239376e-05, 'small': 2.5547032545322968e-05, 'use': 2.5278924193736674e-05, 'corrupt': 2.4365054884405836e-05, 'close': 2.4067472555387275e-05, 'stop': 2.384457065113884e-05, 'bad': 2.3683549617535945e-05, 'looking': 2.3352752847996654e-05, 'law': 2.2583547786352578e-05, 'On': 2.2540780257911715e-05, 'puts': 2.1867633979874233e-05, 'leaves': 2.1589022381985716e-05, 'various': 2.1426510351921586e-05, 'teenage': 2.0402302706236984e-05, 'thinks': 2.0344241281578173e-05, 'ever': 1.9456193642509987e-05, 'follow': 1.9318027487633762e-05, 'political': 1.916889662419943e-05, 'writer': 1.906141117901446e-05, 'parents': 1.8936261761435582e-05, 'island': 1.875309848351491e-05, 'money': 1.8277491466461276e-05, 'Her': 1.8017159351469974e-05, 'sister': 1.7589918313132053e-05, 'middle': 1.7129101426813784e-05, 'run': 1.6614465589340635e-05, 'shows': 1.64853859559288e-05, 'begin': 1.5790491700342798e-05, 'However,': 1.5603556713218544e-05, 'TV': 1.5322964951887614e-05, 'play': 1.4729637003680738e-05, 'free': 1.4122158878130476e-05, 'among': 1.3802972703600669e-05, 'encounters': 1.3571290364772839e-05, 'experiences': 1.3333876837724347e-05, 'common': 1.2883735362167597e-05, 'This': 1.2840587411726178e-05, 'stay': 1.271333666865269e-05, 'man,': 1.2637014648444978e-05, 'you': 1.0209716921362418e-05, 'apartment': 1.0123559409806692e-05, 'hard': 9.753044567412863e-06, 'Their': 9.722269225711621e-06, 'trip': 9.692460536451281e-06, 'daughter': 9.205278842201037e-06, 'band': 8.524200215307934e-06, 'would': 8.091459650814031e-06, 'history': 7.914946382096993e-06, 'taken': 7.359456283210349e-06, 'whom': 7.2968146125063176e-06, 'six': 6.316852456160365e-06, 'believes': 6.003326742153314e-06, 'without': 5.726240840987287e-06, 'Three': 5.381423552599085e-06, 'moves': 5.105996065624237e-06, 'brothers': 4.844301321180465e-06, 'everyone': 4.392577556228644e-06, 'deal': 4.278000455290069e-06, 'friend,': 4.243413694745292e-06, 'keep': 3.788086223333113e-06, 'sees': 3.110946439437873e-06, 'point': 2.8576947865794846e-06, 'uses': 2.0781376453188322e-06, 'sexual': 1.9666103858306105e-06, 'working': 1.4390315935955978e-06, 'call': 8.84590055559038e-07, 'several': 7.954735611262576e-07, 'brother,': 5.228625936322173e-07, 'share': 4.302663108348751e-07, 'wrong': 4.25777683128505e-07, 'Texas': 3.6987554907650023e-07, 'dies': 3.2085300542102547e-07, 'boys': 1.6032137271691198e-07, 'killing': 1.1273122846254205e-07, "isn't": 9.92732359000178e-08, 'Hollywood': 3.938071242498251e-08, 'mother,': 2.0095495717300823e-08, 'grows': 1.4789298572294252e-08}
# tag_dict = scores_from_text('tagline')
tag_dict = {'New': 0.03620693451600272, 'Vengeance': 0.03116531124173893, 'Be': 0.02500518021858832, 'An': 0.02157006813014948, 'A': 0.01838938081866782, 'adventure': 0.017583193421964002, 'Revenge': 0.017189681488235277, 'Live': 0.017036352405648426, 'is': 0.01523541027167244, 'of': 0.01509646136664354, 'So': 0.014488944589428973, 'From': 0.013976882889342606, 'the': 0.012993961813664554, 'Home': 0.011838431028526126, 'we': 0.010815957543901132, 'Has': 0.010353356699317829, 'For': 0.00988209949569636, 'to': 0.009571756900179525, 'very': 0.00905544607453493, '': 0.009014554909843606, 'All': 0.008800074063242852, 'come': 0.008531034623051655, 'you': 0.008471723985630961, 'Power': 0.008378715467960117, 'Love': 0.008120981651841597, 'danger': 0.00800571671850881, 'a': 0.007867533187199866, 'Witness': 0.007706794961538331, 'out': 0.007445054298816698, 'has': 0.007250346416197645, 'fire': 0.006861633625847423, 'heroes': 0.006830226756567248, 'believe': 0.006657771344583896, 'million': 0.00641100188882075, 'jungle': 0.006263029533205576, "You're": 0.006152944051856903, 'will': 0.006063178678429792, 'our': 0.0059589119393204625, 'friends': 0.005940417032258893, 'beginning': 0.005764006770809394, 'them': 0.005550066022046772, 'begins': 0.0054241389587627396, 'Welcome': 0.005260928020402225, 'always': 0.005196549933832258, 'his': 0.005148305307114558, 'The': 0.0050913252257838515, "He's": 0.005055557355908562, 'talk': 0.004861242710240592, "Don't": 0.004859510342510808, 'in': 0.004777268419237977, 'journey': 0.004463501034564081, 'remember': 0.004251355873232954, 'choice': 0.004238603967362016, 'Get': 0.004174743638467524, 'world': 0.004169818868132959, 'making': 0.0041620456758191485, 'Why': 0.004042971026541473, 'Your': 0.004026177588636709, 'Will': 0.003996339389138126, 'date': 0.003936849994874444, 'Legend': 0.003931711445065682, 'And': 0.0038431268778598452, 'Forever': 0.0038319095676050325, 'Never': 0.0037715549503521692, 'go': 0.0037230632043602696, 'Begins': 0.00371997107537935, 'lies': 0.003718969543027746, 'back': 0.0037050756955215288, 'When': 0.003669615790507593, 'No': 0.003656805737385265, 'coming': 0.003638344362947611, 'Life': 0.0035067373204179103, "won't": 0.003462330756528845, 'Our': 0.0034385681691695993, 'Believe': 0.00337044786017711, 'At': 0.0033518630842076583, 'looking': 0.0033399668746615222, 'end': 0.003328957473366608, 'are': 0.003315697736700872, 'happens': 0.003313282332839257, 'age': 0.0032128941242482918, 'away': 0.003202441893967663, 'alone': 0.0031962139056128944, 'history': 0.0031636562168490777, 'all': 0.0031503522427421445, 'Their': 0.0030638122704341646, 'finding': 0.0030472594288760155, 'love,': 0.0030158065355390348, 'It': 0.003007080190654036, 'To': 0.0029978537144863815, 'just': 0.002988510064656909, 'it': 0.002941328286613378, 'One': 0.0028106805253901505, 'past': 0.0028051531682790993, 'greatest': 0.0028048389858787823, 'Let': 0.00279716795285605, "can't": 0.0027585991786314014, 'Hell': 0.002737783878670046, 'were': 0.0027312749870475406, 'Game': 0.0027134583970041598, 'You': 0.0026865225374253085, 'life': 0.0026687300572603803, 'on': 0.002637447861007304, 'and': 0.0026181421870635113, 'got': 0.0025066297585749665, 'be': 0.0024828562903396706, 'most': 0.0024448321586615367, 'Go': 0.002421232107432236, 'legend': 0.002414085050202184, 'step': 0.002404677714872223, 'conquered': 0.002384818074875063, 'last': 0.0023538917932216575, 'future': 0.0023413853472911634, 'courage': 0.002310232188182393, 'matter': 0.002253151829398447, 'Trust': 0.002176007285388633, 'Everyone': 0.002156336355279988, 'family': 0.0021546984255798388, "Can't": 0.00215128682801736, 'who': 0.0021307789322876506, 'next': 0.002124139644815584, 'war': 0.0020929702439925063, 'With': 0.002083323599798481, 'waiting': 0.00204216072824447, 'revenge': 0.0020135738610501555, 'other': 0.0020102231582467362, "it's": 0.0019951455955924094, 'must': 0.00198000891297929, 'for': 0.0019247102009776382, 'as': 0.0019013221052186142, 'an': 0.001886731074686778, 'unforgettable': 0.0018860895060413465, 'not': 0.0018748072264341438, 'Hope': 0.001861967592062148, 'trust': 0.0018542458416941545, 'forever': 0.0018284427246870482, 'His': 0.0018192779703152199, 'comes': 0.001807705186132143, 'fall': 0.001805146529795363, 'over': 0.0017979710272999713, 'What': 0.0017873959218033281, 'from': 0.0017844199378373535, 'going': 0.0017837134460557298, 'terror': 0.0017829065481352352, 'Beginning': 0.0017543176910125937, 'Earth': 0.0017445365646408203, 'those': 0.0017342779334529074, 'people': 0.0017316567631505864, 'History': 0.0017270085745810086, 'at': 0.0017240079552129125, 'us': 0.0017014703457907895, 'Some': 0.0016529272566197244, 'justice': 0.0016474774905604697, 'place': 0.0016384916276347857, 'But': 0.0016374752293529353, 'make': 0.00162000485218236, 'As': 0.0016116560875844264, 'die': 0.001604951939836685, 'girl': 0.0015937132966643958, 'By': 0.0015931724350701144, 'between': 0.0015697543843039153, 'good': 0.001531235431792672, 'one': 0.0015295650785703313, 'true': 0.0015187280080893523, 'Evil': 0.0014873288704357846, 'Prepare': 0.0014621793127718372, 'have': 0.0014570152735695005, 'get': 0.0014363584352689574, 'Not': 0.0014320325238654041, 'time,': 0.001417105837125606, 'can': 0.0013905381157272836, 'dangerous': 0.001387497716297502, 'little': 0.0013544932848549114, 'someone': 0.0013223714299684557, 'new': 0.0013137389258436887, 'beyond': 0.0013091998704652453, 'Ever': 0.0013089823263346396, 'Is': 0.0012990244439623128, 'Story': 0.0012989144243746348, 'face': 0.001292256761288817, 'second': 0.0012630375354665533, 'man': 0.0012618094193376423, "she's": 0.0012543067483697218, 'no': 0.0012503528941579605, 'they': 0.0012424725616328898, 'challenge': 0.00123675949968891, 'everything': 0.0012218080225541554, "he's": 0.0012043409917253925, 'that': 0.0011980246669788968, 'house': 0.0011940026820716025, "you've": 0.0011903335951292462, 'like': 0.001166504081423653, 'your': 0.0011538629903218407, "It's": 0.0011487558345793752, 'Discover': 0.0011366217586762156, 'only': 0.0011118904877950651, 'her': 0.0011084084693920593, 'Look': 0.0011009942275548512, 'guys': 0.0010622298454134008, 'Someone': 0.0010600524242153902, 'back!': 0.001057987626165825, 'He': 0.0010524580902535193, 'better': 0.0010490936108236508, 'Law': 0.0010327032110654432, "don't": 0.0010273962926566264, 'getting': 0.0010256107843400982, 'with': 0.0010206595010002935, 'Born': 0.0010151740824376882, 'Everything': 0.0009957083645727252, 'extraordinary': 0.0009934527419362738, 'This': 0.0009790601210442685, 'nothing': 0.0009677083223560805, 'mean': 0.0009497240962612145, 'far': 0.0009485136663555712, 'Real': 0.0009478213607304158, 'First': 0.0009390049295164356, 'darkness': 0.0009333418419314041, 'Comedy': 0.0009309402254001348, 'never': 0.0009306962651410754, 'Where': 0.0009253790137543347, 'story': 0.0009163148876813653, 'Yourself': 0.0008999981070507262, 'death': 0.0008791744669483246, 'Back': 0.0008711974247666952, 'stands': 0.0008689136876001227, 'freedom': 0.0008630191978361845, 'game': 0.0008619899648240247, 'THE': 0.0008566224251368387, 'than': 0.0008457382059766144, 'their': 0.0008453605596147769, 'yours': 0.0008392672661408649, 'perfect': 0.0008370253761833726, 'Up': 0.0008320516129269488, 'Men': 0.0008255254627175452, 'There': 0.0008139845295986074, 'planet': 0.0008078410311755088, 'so': 0.0008023983690349824, 'love': 0.0007989289975232434, 'hell': 0.0007973099892783145, 'stay': 0.0007817194885990734, 'by': 0.0007816906444444499, 'stand': 0.0007685720534546256, 'Rules': 0.0007623246805625833, 'was': 0.0007525210313170566, 'miles': 0.0007525059782241659, 'They': 0.0007504037746425953, "what's": 0.0007464199280772152, 'about': 0.0007410254880765885, 'found': 0.0007393834293195097, 'Fear': 0.000737652074088648, 'tale': 0.0007289911413762133, 'Every': 0.0007274376131779245, 'now': 0.0007201887051809005, 'Adventure': 0.0007159895914833217, 'Only': 0.0007147443541182483, 'Today': 0.0007091617016731834, 'Honor': 0.0007056140830853208, 'On': 0.0006907473649769838, 'another': 0.0006901496845742499, 'he': 0.0006893603937134674, 'cop': 0.0006857561752425551, 'People': 0.0006836941700087789, 'change': 0.0006754146827383391, 'Can': 0.0006618027284570585, 'Save': 0.000659267697967567, 'here': 0.0006574430494663687, 'having': 0.0006538419371394757, 'About': 0.0006506127872164598, 'Most': 0.0006491414521010425, 'Have': 0.0006465610400741718, 'this': 0.0006423860070247523, 'Justice': 0.0006387336213636205, 'power': 0.0006321570265789498, 'number': 0.0006319614418239647, 'old': 0.0006254406215316511, 'without': 0.0006241370748361964, 'limits': 0.0006229628698782264, 'Big': 0.0006097004655829543, 'Little': 0.0005908666736574239, 'up': 0.000589865861287134, 'Take': 0.0005836010168286861, 'if': 0.0005768502168157016, 'Something': 0.0005654618930034021, 'Man': 0.0005613281135866396, 'Fight': 0.0005591356049243673, 'men,': 0.0005448594028670983, 'after': 0.0005441564592110421, 'Are': 0.0005405046082659573, 'America': 0.0005374169269575918, 'race': 0.0005342133255888344, 'Courage': 0.0005267957481047472, 'Dream': 0.0005255777027050756, 'ever': 0.0005219471090427769, 'need': 0.0005149931277405677, 'fighting': 0.0005136285514123891, 'wants': 0.0005071640396095957, 'In': 0.0005062645208338576, 'Four': 0.0005011478074690971, 'there': 0.0004994259529840327, 'been': 0.0004968256911555162, 'mission': 0.0004951431436005995, 'gone': 0.0004917245573930468, 'thing': 0.0004910259822191567, 'first': 0.0004812102647868675, 'truth': 0.00048072573211584873, 'anything': 0.0004775235677766116, 'heart': 0.00047503943437246493, 'King': 0.00046909585026099176, '3': 0.0004668485062955842, 'Just': 0.0004662750074400332, 'saw': 0.00045593732743430093, 'hero': 0.00045505300565412893, 'We': 0.00045339152639031993, 'met': 0.0004501663719974726, 'best': 0.0004453026085126864, 'American': 0.00044204563135954045, 'any': 0.00044173420438080237, 'time': 0.0004350600078334709, 'ready': 0.000432837402470828, 'needed': 0.0004313184787089106, 'some': 0.0004249850334870382, 'woman': 0.000417074593720355, "Who's": 0.00041637496178654283, 'War': 0.0004162593467694524, 'Meet': 0.00041285220318730087, 'Make': 0.000405465855531142, 'Different': 0.00040414156339708154, 'anywhere': 0.0004034333155415439, 'She': 0.00040340870445188215, 'dead': 0.0004009598033020822, 'impossible': 0.0003999552133240279, "isn't": 0.00039878770742611243, 'Start': 0.0003980246631329823, 'find': 0.0003969077140446303, 'possible': 0.00039011291186193955, 'too': 0.00038791804854901486, 'Now': 0.0003851208266968641, 'ordinary': 0.0003804302287001352, 'case': 0.00037714760422605763, 'rules': 0.00037617809731912055, 'hear': 0.0003730366397824289, 'count': 0.00036903537514544713, '4': 0.0003672769665014267, 'hit': 0.00036679486149646427, 'but': 0.0003666095209649337, 'Death': 0.0003657866838219325, 'few': 0.0003601820160690075, 'school': 0.00035825007428240624, 'live': 0.0003565510396240943, 'light': 0.00035579418955103946, 'Who': 0.00035171977843926765, 'Come': 0.00034648104927293533, 'Nothing': 0.0003462177128178142, 'keep': 0.00034186109977115065, 'lot': 0.0003379230843617351, 'run': 0.0003364399919219784, 'could': 0.0003325235891896556, 'comedy': 0.0003313405986729473, 'mistake': 0.0003311357141986826, 'way': 0.0003307531688312332, 'still': 0.00032841514102497857, 'enemies': 0.0003240789838600674, 'incredible': 0.0003227117776038622, 'film': 0.00032057326323221374, 'World': 0.0003166123279399795, 'call': 0.0003151461586744238, 'lifetime': 0.0003134670347883621, 'Danger': 0.0003131597049330733, 'law': 0.00031256377516660806, 'happening': 0.000312312842258932, "doesn't": 0.0003116879859735617, 'do': 0.0003031674156887206, 'crime': 0.0002998724823605275, 'lives': 0.00029885439311780793, 'Truth': 0.0002984114095318157, 'may': 0.00029787546604910677, 'powerful': 0.00029778969487812874, 'town': 0.0002976783812490754, 'more': 0.0002971048297849041, 'dream': 0.0002966038747354166, 'real': 0.00029263498275718943, 'murder': 0.0002903142149817214, 'hide': 0.0002862271721120207, 'Find': 0.0002836755195933848, 'him': 0.00028277506494615065, 'you?': 0.00028205918033302106, 'let': 0.0002804626014823061, 'even': 0.0002780535147893638, 'save': 0.0002756384802016284, 'years': 0.0002748859094241378, 'out,': 0.00027275368374324446, 'women': 0.00027186104906240895, 'battle': 0.0002708152906594194, "You'll": 0.00026898189001250906, 'until': 0.0002672219783592628, 'friendship': 0.000266437417310287, 'killer': 0.00026432572603210004, 'wife': 0.0002641111032678291, 'deadly': 0.0002636115878188251, "world's": 0.00026230895269348513, 'More': 0.00026149492991178183, 'look': 0.000261016689987873, 'Play': 0.00025870709777275414, 'finally': 0.00025806045239729286, 'gonna': 0.0002552231797625694, 'Last': 0.0002551874201740687, 'Of': 0.0002509171123581659, 'into': 0.0002490047159397535, 'Based': 0.0002468361088958246, 'or': 0.0002453404039304658, 'black': 0.00024076740000349394, 'nowhere': 0.0002407498601353281, 'secret': 0.00023997661555694166, 'plan': 0.00023962535874750743, 'himself': 0.00023894289761194993, 'being': 0.00023668053802976757, 'Before': 0.00023572025854751472, 'bad': 0.00023467252883618302, 'tell': 0.00023217219645610765, 'cost': 0.00023180051385748206, 'name': 0.00023136022115218213, 'friend': 0.00023126090429637577, 'How': 0.00022239868822900534, 'shot': 0.00021767409169847109, 'gets': 0.00021761707612223078, 'safe': 0.00021542433365807334, 'body': 0.00021479983909070398, 'would': 0.0002145767070542683, 'fought': 0.0002141353155789207, 'times': 0.00021396507721528987, 'Two': 0.00021267909547672837, 'down': 0.00021225227276478945, 'long': 0.00021135265968677666, 'dies': 0.00021013065123415346, 'day': 0.000210108269900602, 'should': 0.00020975577786258988, 'party': 0.00020881937827429295, 'country': 0.00020794777860718718, 'True': 0.00020607856225816676, 'much': 0.00020324794788070845, 'loves': 0.0002000763309588735, 'everyone': 0.00019911803094117657, 'things': 0.00019768301417672605, 'imagination': 0.00019405894348236506, 'seems': 0.0001922509285554299, 'secrets': 0.00018604593004625873, 'easy': 0.00017889143552720952, 'goes': 0.0001764266891434222, 'nation': 0.00017520425973322498, 'own': 0.00017472241096587476, 'star': 0.00017463935618389177, "man's": 0.00017410811731219427, 'miracle': 0.0001698101089110326, 'did': 0.00016980728379492157, 'when': 0.00016971806116025724, 'survival': 0.00016931428352537684, 'behind': 0.0001672063612750055, 'either': 0.0001657647139381968, 'life,': 0.0001560994076798502, 'rise': 0.00015259139504308512, 'knows': 0.00015195102286385694, 'how': 0.00015077628078368982, "They're": 0.00014995205583056434, "We're": 0.00014889228925239823, "There's": 0.00014546214598625315, 'lead': 0.00014394188881274532, 'others': 0.00014372053120245335, 'Inside': 0.00014350746159932953, 'home': 0.00014200199966357278, 'came': 0.00014102478790352867, 'Hero': 0.00013891270614518983, 'left': 0.0001386702621915228, 'Leave': 0.00013825892430965729, 'If': 0.00013724289367236282, 'lived': 0.00013659707717369598, 'buried': 0.00013524641426491683, 'think': 0.00013497056141191407, 'whole': 0.0001304887243739022, "they're": 0.00012574088141616943, 'year,': 0.00012447075534908895, 'knew': 0.00012272238785699736, 'hope': 0.0001225832603707832, 'final': 0.0001204576034366667, 'I': 0.00011984620871450559, 'every': 0.00011936226204002057, 'where': 0.00011915182848939442, 'father': 0.00011853614851679156, 'Sex': 0.00011836945102499596, 'seen': 0.00011754506452606331, 'city': 0.00011748722975044696, 'high': 0.0001171858516673637, '-': 0.000117088570825431, 'That': 0.0001118185772130992, 'trouble': 0.00010998051784474522, 'Murder': 0.00010629464618356608, 'ride': 0.00010512341684225503, "you're": 0.00010377110215368631, 'Out': 0.00010299363227468307, 'made': 0.00010145965229195334, 'what': 0.00010044571908314445, 'mystery': 9.929312670898576e-05, 'dreams': 9.514634145191386e-05, 'human': 9.294713298801637e-05, 'fight': 9.11717669198345e-05, 'After': 9.043986774002914e-05, 'summer': 8.978746260000772e-05, 'alive': 8.92867565178194e-05, 'fly': 8.787860981188698e-05, 'Once': 8.732408944952738e-05, 'needs': 8.644598939070385e-05, 'Sometimes': 8.479857079743107e-05, 'fear': 8.470942879957607e-05, 'big': 8.461177906107068e-05, 'motion': 8.44475432777187e-05, 'three': 8.381857550581479e-05, 'road': 8.305520934993528e-05, 'Her': 8.196266902635198e-05, 'had': 8.056717047778467e-05, "She's": 7.965736182296331e-05, 'Survival': 7.958662837476655e-05, 'dog': 7.884002500359411e-05, 'right': 7.799271687570308e-05, '1': 7.538566699520616e-05, 'she': 7.397246066145091e-05, 'Was': 7.173432644424726e-05, 'fun': 7.146198314546262e-05, 'Survive': 6.981664929486745e-05, 'Betrayal': 6.847168445343728e-05, 'against': 6.795478251874229e-05, 'many': 6.79454539604956e-05, 'began': 6.75066902719749e-05, 'runs': 6.733161780493752e-05, 'kill': 6.653300123052205e-05, 'take': 6.560241557476587e-05, 'movies': 6.50227529483906e-05, 'Do': 6.44242981839428e-05, 'gave': 6.438518055150063e-05, "you'll": 6.373405438693404e-05, 'movie': 6.368060530508119e-05, 'Three': 6.308318079876453e-05, 'favorite': 6.302449560805549e-05, 'search': 6.26050656110406e-05, 'dying': 6.143224481780689e-05, 'means': 6.077741086643661e-05, 'know': 5.888161259070792e-05, 'James': 5.8840399395292e-05, 'moment': 5.8147058047053544e-05, 'Beyond': 5.749806223241814e-05, 'apart': 5.734410191610228e-05, 'chance': 5.711729065701756e-05, 'two': 5.689387430706136e-05, 'happen': 5.6579635531661234e-05, 'Behind': 5.577047846645584e-05, 'God': 5.480947799560851e-05, 'High': 5.470158365826201e-05, 'stole': 5.3817003941606336e-05, 'give': 5.317441137741483e-05, 'kid': 5.130488926194886e-05, 'hold': 5.08060572389578e-05, 'Time': 4.8976315704396666e-05, 'play': 4.818196692211223e-05, 'picture': 4.752875061386119e-05, 'yourself': 4.684162619937974e-05, '!': 4.626448522246993e-05, 'its': 4.433164160185862e-05, 'romantic': 4.384546365482421e-05, 'girls': 4.28806431073154e-05, 'close': 4.228126150100119e-05, "wasn't": 4.1495185636896177e-05, 'changed': 4.124183863666234e-05, 'deep': 3.9085942895786835e-05, 'book': 3.8803947299711344e-05, 'earth': 3.85368033721114e-05, 'does': 3.824878220514372e-05, 'Terror': 3.824011287647497e-05, 'again': 3.685770139406576e-05, 'crazy': 3.463277828540275e-05, 'each': 3.453562726724695e-05, 'meet': 3.4210033298583644e-05, 'Mercy': 3.4107328482752314e-05, 'forget': 3.3745886058045663e-05, 'soul': 3.3420234643183994e-05, 'young': 3.324611420436313e-05, 'such': 3.236631580431148e-05, 'Best': 3.192808535560171e-05, 'ultimate': 2.9659875268243844e-05, 'my': 2.7763480418396313e-05, 'want': 2.7514637104959332e-05, 'trying': 2.6759443175249505e-05, 'Even': 2.657133650432234e-05, 'year': 2.6241976810326854e-05, 'because': 2.580125087449911e-05, 'trip': 2.579964246367937e-05, 'Obsession': 2.5721345894856828e-05, 'Experience': 2.529949724380828e-05, 'course': 2.4659931571548236e-05, 'price': 2.455810327140772e-05, 'streets': 2.4548677048817414e-05, 'side': 2.393716455776168e-05, '12': 2.3780983975134397e-05, 'both': 2.33428679240074e-05, 'John': 2.121177470222141e-05, 'men': 2.0840184100609124e-05, 'night': 2.080287066285254e-05, 'funny': 2.0444695763404907e-05, 'see': 1.9098618564912662e-05, 'thought': 1.833108187746387e-05, 'lose': 1.8054623591320014e-05, 'evil': 1.7831236690178435e-05, 'edge': 1.705506265446577e-05, 'free': 1.5322207818873126e-05, 'cops': 1.4958162032988431e-05, 'manhunt': 1.487560326503432e-05, 'together': 1.4860372648035756e-05, 'four': 1.4806989305942204e-05, 'took': 1.4751435930994856e-05, 'forbidden': 1.4367256677232201e-05, 'put': 1.3712653457537857e-05, 'reality': 1.3508413924466551e-05, 'door': 1.3235157712427043e-05, 'Mother': 1.1792157671662395e-05, 'inside': 1.1665608522956113e-05, 'great': 1.1080858548618177e-05, 'Then': 1.0951320321865094e-05, 'living': 1.0900221738089943e-05, 'say': 1.0874411307169011e-05, 'Family': 1.076193416722271e-05, 'belong': 1.0442068890902102e-05, 'weekend': 1.0360439623861326e-05, 'takes': 1.028627394677168e-05, 'strong': 1.008759780798499e-05, 'through': 9.857067836455067e-06, 'See': 9.701456494761916e-06, 'protect': 9.163600885551738e-06, 'really': 9.096197339622386e-06, 'turn': 8.747654443842874e-06, 'before': 8.343523568682185e-06, 'days': 7.82951450037734e-06, 'boy': 7.610369427486912e-06, 'it!': 7.422663643061076e-06, 'hard': 7.337324010384851e-06, 'biggest': 7.194224772625323e-06, "didn't": 7.0529492059073e-06, "there's": 6.53009426723086e-06, 'spectacle': 6.24843625633146e-06, 'set': 6.198578241097471e-06, 'target': 6.06839255866178e-06, 'try': 5.5671109277458955e-06, 'Fighting': 5.56160577339351e-06, 'world,': 5.464930916028693e-06, 'killed': 5.325071153865425e-06, 'Like': 5.28119962354059e-06, 'survive': 4.839145259146183e-06, 'Heroes': 4.71641537869352e-06, 'passion': 4.251380963007932e-06, "couldn't": 4.23740281734405e-06, 'wrong': 4.1906445366637815e-06, 'Being': 4.082125603431591e-06, 'choose': 3.7701474027577285e-06, 'years,': 3.5999278178406603e-06, 'home,': 3.588058909318712e-06, 'wanted': 3.491715864683279e-06, 'mind': 3.4668425800356197e-06, 'classic': 3.4571082406869473e-06, 'space': 3.410345741367822e-06, 'loved': 3.3912226248246676e-06, 'weapon': 3.326508727136684e-06, 'con': 3.2196603653721735e-06, 'Until': 3.13758842511474e-06, 'guy': 2.957959641653703e-06, "who's": 2.7535907618427343e-06, 'lost': 2.1035764381236295e-06, 'became': 2.092660124174814e-06, 'anyone': 2.0656844210914526e-06, "America's": 2.0432902231558606e-06, 'City': 1.9583758771297093e-06, 'romance': 1.94496577555691e-06, 'enemy': 1.8022885372946844e-06, 'called': 1.4275563782917483e-06, 'friends,': 7.914503249853476e-07, 'started': 7.242742498160318e-07, 'once': 7.152538148698693e-07, 'magic': 6.208336863127885e-07, 'York': 5.201692741569738e-07, 'desire': 4.570699937225715e-07, 'tough': 4.1595242784417615e-07, 'stop': 3.1300938749811797e-07, 'hours': 3.031092820156608e-07, 'murderer': 2.7042704656775766e-07, 'entire': 2.1114203752709402e-07, 'present': 1.339467846492312e-07, 'become': 1.1924715544128377e-07, 'police': 8.570268772963181e-08, 'laughter': 3.273067614985543e-08, 'Tonight,': 2.936277364122439e-08, 'violence': 2.2386017752896857e-08, 'something': 1.7699332285346148e-08, 'upon': 8.747988047227817e-09, 'beat': 2.636927569400668e-10, 'these': 0.0}
score_dicts = {
    'cast': cast_dict,
    'production_companies': com_dict,
    'production_countries': country_dict,
    'crew': crew_dict,
    'overview': over_dict,
    'tagline': tag_dict,
    'Keywords': keys_dict,
    'genres': genre_dict,
}

{'Peter Jackson': 0.3607585833952881, 'Michael Bay': 0.2278850807583257, 'Steven Spielberg': 0.19870629923784916, 'Jon Turteltaub': 0.03256032241596181, 'Ridley Scott': 0.02978942131662294, 'Todd Phillips': 0.02665669126247886, 'Tim Burton': 0.025348163208237934, 'Ron Howard': 0.02052850887908686, 'Paul W.S. Anderson': 0.012870197832332269, 'Wolfgang Petersen': 0.012063284091476762, 'Tony Scott': 0.0075079252521017366, 'Ivan Reitman': 0.006701125863214141, 'John Glen': 0.005124794246014205, 'Billy Wilder': 0.003161555869235804, 'Alfred Hitchcock': 0.0030549866547323417, 'Garry Marshall': 0.0027003887450844856, 'Blake Edwards': 0.002610201807169538, 'Michael Mann': 0.0022647740408132623, 'Ralph Bakshi': 0.001978492390377613, 'Richard Linklater': 0.001970996732781956, 'Luis Mandoki': 0.0017744945539599633, 'John Frankenheimer': 0.0017513420262906371, 'Sidney Lumet': 0.0015331505757762705, 'John Huston': 0.0014378253057946249, 'Lasse Hallstr√∂m': 0.0013841503688306251, 'Wes Anderson': 0.0

In [0]:
def proc_score(df, header):
    name_lists = df[header].apply(get_name_list)
    counts = name_lists.apply(lambda x: len(x))
    scores = name_lists.apply(lambda x: get_score(x, score_dicts[header]))
    after = df.drop(header, axis = 1)
    after[header+"_score"] = scores
#     after[header+'_count'] = counts
    return after

def proc_text_score(df, header):
    name_lists = df[header].apply(
        lambda x:
            set() if isinstance(x, float) else
            set(x.replace('.', '').replace(';', '').replace(':', '').split(' ')))
    scores = name_lists.apply(lambda x: get_score(x, score_dicts[header]))
    after = df.drop(header, axis = 1)
    after[header+"_score"] = scores
    return after

def proc_cast2(df):
    return proc_score(df, 'cast')
def proc_dir(df):
    return proc_score(df, 'crew')
def proc_genre(df):
    return proc_score(df, 'genres')
def proc_com(df):
    return proc_score(df, 'production_companies')
def proc_country(df):
    return proc_score(df, 'production_countries')
def proc_crew(df):
    return proc_score(df, 'crew')
def proc_keys(df):
    return proc_score(df, 'Keywords')
def proc_over(df):
    return proc_text_score(df, 'overview')
def proc_tag(df):
    return proc_text_score(df, 'tagline')

In [0]:
%%time

X_train, X_test = process_date(df_train), process_date(df_test)
X_train, X_test = proc_budget(X_train), proc_budget(X_test)
X_train, X_test = proc_coll(X_train), proc_coll(X_test)
X_train, X_test = proc_time(X_train), proc_time(X_test)
X_train, X_test = proc_cast2(X_train), proc_cast2(X_test)
X_train, X_test = proc_dir(X_train), proc_dir(X_test)
X_train, X_test = proc_genre(X_train), proc_genre(X_test)
X_train, X_test = proc_com(X_train), proc_com(X_test)
X_train, X_test = proc_home(X_train), proc_home(X_test)
X_train, X_test = proc_country(X_train), proc_country(X_test)
# X_train, X_test = proc_crew(X_train), proc_crew(X_test)
X_train, X_test = proc_over(X_train), proc_over(X_test)
X_train, X_test = proc_tag(X_train), proc_tag(X_test)
X_train, X_test = proc_keys(X_train), proc_keys(X_test)
print_score(X_train, X_test = X_test)

Accuracy: 2.12 (+/- 0.73)
                            importance
log_budget                    0.375787
popularity                    0.103229
production_companies_score    0.077737
release_year                  0.064705
runtime                       0.058540
production_countries_score    0.046462
genres_score                  0.040759
overview_score                0.040147
tagline_score                 0.039414
release_season                0.037088
Keywords_score                0.035453
belongs_to_collection_bool    0.034694
homepage_bool                 0.020351
cast_score                    0.019272
crew_score                    0.006362
2019-02-17 04:58:25.194619
CPU times: user 22.1 s, sys: 581 ms, total: 22.7 s
Wall time: 29.3 s
